# Cleaning Notebook: County Health Ranking Data sets 2015-2020

### This file calls the directory for each year of data, loops through the files (which represent different states) within each year's folder and combines various sheets of metrics into a single dataframe for the year finally appending each state in a consistent format.  Additional steps are made to review nulls, convert values to consistent categories where necessary and then address data types align for future use. 

Ultimately converts to 6 csv files: 
- US data in its entirety for all years and counties
- US data for training (based on all ranked counties and removes tooltip related entries from main data), and
- US data for testing (based on unranked counties)
- Texas only data (filters US full data set to just Texas counties for all years)
- Texas data for training

### Import Dependencies
(included pymongo in case we eventually link code to directly connect to DB)

In [1]:
import pandas as pd
from datetime import datetime
import os
import numpy as np
import pymongo

### Loop to move through files within 2020 Data Folder
- join 3 worksheets within each state file together
- rename required columns
- drop unnecessary columns
- append all state data to single dataframe for 2020

In [2]:
appended_data2020 = []
path = os.path.join("Original_Files","CountyHealthRankingsData","2020")
for WorkingFile in os.listdir(path):
        #identifies the list of files to be grouped together from designated folder
        print(WorkingFile)
        
        # Import the excel file by joining the path directory and file name together and call Ranked Data and Additonal Measure tabs
        file = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Ranked Measure Data',header=(0,1))
        file2 = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Additional Measure Data',header=(0,1))
        file3 = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Outcomes & Factors Rankings',header=(0,1))
       
        # Load the excel files as two dataframes
        df = pd.DataFrame(file)
        df2 = pd.DataFrame(file2)
        df3 = pd.DataFrame(file3)
        
        # Code from Data_cleaning
       
        # Clean Ranked Data First by addressing multiindex column names
        df.columns = df.columns.map('_'.join)
        
        # Rename Columns
        df.rename(columns = {'Unnamed: 0_level_0_FIPS' : 'FIPS',
         'Unnamed: 1_level_0_State' :'State',
         'Unnamed: 2_level_0_County':'County/Borough/Parish',
         'Unnamed: 2_level_0_Borough':'County/Borough/Parish',
         'Unnamed: 2_level_0_Parish':'County/Borough/Parish',
         'Premature death_Years of Potential Life Lost Rate':'Yrs_Potential_Life_Lost_per100K(YPLL)',
         'Poor or fair health_% Fair or Poor Health':'%_Fair/Poor_Health',
         'Low birthweight_% Low Birthweight':'%_Low_Birthweight_Births',
         'Adult smoking_% Smokers':'%_Adult_Smokers',
         'Adult obesity_% Adults with Obesity':'%_Adult_Obesity',
         'Food environment index_Food Environment Index':'Healthy_Food_Access(0=Worst,10=Best)',
         'Physical inactivity_% Physically Inactive':'%_Adults_Physical_Inactivity',
         'Access to exercise opportunities_% With Access to Exercise Opportunities':'%_Access_to_Excercise_Opportunities',
         'Excessive drinking_% Excessive Drinking':'%_Adults_Excessive_Drinking',
         'Alcohol-impaired driving deaths_% Driving Deaths with Alcohol Involvement':'%_Driving_Deaths_with_Alcohol_Involvement',
         'Sexually transmitted infections_Chlamydia Rate':'Chlamydia_Cases_per_100K',
         'Teen births_Teen Birth Rate':'Teen_Birth_Rate',
         'Uninsured_% Uninsured':'%_Ppl_Under65_w/o_Insurance',
         'Primary care physicians_Primary Care Physicians Rate':'PCP_per_100K',
         'Dentists_Dentist Rate':'Dentists_per_100K',
         'Mental health providers_Mental Health Provider Rate':'MHP_per_100K',
         'Preventable hospital stays_Preventable Hospitalization Rate':'Preventable_Hospitalization_Rate(per100K_Medicare_Enrollees)',
         'High school graduation_High School Graduation Rate':'Graduation_Rate',
         'Some college_% Some College':'%_Adults(25-44)_with_Some_Post-secondary_Ed',
         'Unemployment_% Unemployed':'%_Ppl(16+)_Unemployed_&_Looking_for_Work',
         'Children in poverty_% Children in Poverty':'%_Children(under18)_Living_in_Poverty',
         'Income inequality_80th Percentile Income':'80th_Percentile_MHI',
         'Income inequality_20th Percentile Income':'20th_Percentile_MHI',
         'Children in single-parent households_% Single-Parent Households':'%_Children_Living_in_Single_Parent_Household',
         'Social associations_Social Association Rate':'Associations_per_10K',
         'Violent crime_Violent Crime Rate':'Violent_Crimes_per_100K',
         'Injury deaths_Injury Death Rate':'Injury_Death_Rate_per_100K',
         'Air pollution - particulate matter_Average Daily PM2.5':'Avg_Daily_Particulate_Matter_in_Micrograms_per_Cubic_Meter',
         'Drinking water violations_Presence of Water Violation':'County_Affected_by_Water_Violation(1=Yes,0=No)',
         'Severe housing problems_% Severe Housing Problems':'%_Households_with_One_or_More Problems(overcrowding,high-housing-cost,lack-kitchen,lack-plumbing-facilities)',
         'Long commute - driving alone_% Long Commute - Drives Alone':'%_Lone_Commuters_who_Commute_More_Than_30Min'}, 
                  inplace = True)
        
        # Drop unnecessary columns
        df = df.drop(['Premature death_Deaths',
         'Premature death_95% CI - Low',
         'Premature death_95% CI - High',
         'Premature death_YPLL Rate (AIAN) 95% CI - Low',
         'Premature death_YPLL Rate (AIAN) 95% CI - High',
         'Premature death_YPLL Rate (Asian) 95% CI - Low',
         'Premature death_YPLL Rate (Asian) 95% CI - High',
         'Premature death_YPLL Rate (Black) 95% CI - Low',
         'Premature death_YPLL Rate (Black) 95% CI - High',
         'Premature death_YPLL Rate (Hispanic) 95% CI - Low',
         'Premature death_YPLL Rate (Hispanic) 95% CI - High',
         'Premature death_YPLL Rate (White) 95% CI - Low',
         'Premature death_YPLL Rate (White) 95% CI - High',
         'Premature death_Z-Score',
         'Poor or fair health_95% CI - Low',
         'Poor or fair health_95% CI - High',
         'Poor physical health days_Average Number of Physically Unhealthy Days',
         'Poor mental health days_Average Number of Mentally Unhealthy Days',
         'Low birthweight_Unreliable',
         'Poor physical health days_95% CI - Low',
         'Poor physical health days_95% CI - High',
         'Poor mental health days_95% CI - Low',
         'Poor mental health days_95% CI - High',
         'Low birthweight_95% CI - Low',
         'Low birthweight_95% CI - High',
         'Low birthweight_% LBW (AIAN) 95% CI - Low',
         'Low birthweight_% LBW (AIAN) 95% CI - High',
         'Low birthweight_% LBW (Asian) 95% CI - Low',
         'Low birthweight_% LBW (Asian) 95% CI - High',
         'Low birthweight_% LBW (Black) 95% CI - Low',
         'Low birthweight_% LBW (Black) 95% CI - High',
         'Low birthweight_% LBW (Hispanic) 95% CI - Low',
         'Low birthweight_% LBW (Hispanic) 95% CI - High',
         'Low birthweight_% LBW (White) 95% CI - Low',
         'Low birthweight_% LBW (White) 95% CI - High',
         'Adult smoking_95% CI - Low',
         'Adult smoking_95% CI - High',
         'Adult obesity_95% CI - Low',
         'Adult obesity_95% CI - High',
         'Physical inactivity_95% CI - Low',
         'Physical inactivity_95% CI - High',
         'Excessive drinking_95% CI - Low',
         'Excessive drinking_95% CI - High',
         'Alcohol-impaired driving deaths_95% CI - Low',
         'Alcohol-impaired driving deaths_95% CI - High',
         'Teen births_95% CI - Low',
         'Teen births_95% CI - High',
         'Teen births_Teen Birth Rate (AIAN) 95% CI - Low',
         'Teen births_Teen Birth Rate (AIAN) 95% CI - High',
         'Teen births_Teen Birth Rate (Asian) 95% CI - Low',
         'Teen births_Teen Birth Rate (Asian) 95% CI - High',
         'Teen births_Teen Birth Rate (Black) 95% CI - Low',
         'Teen births_Teen Birth Rate (Black) 95% CI - High',
         'Teen births_Teen Birth Rate (Hispanic) 95% CI - Low',
         'Teen births_Teen Birth Rate (Hispanic) 95% CI - High',
         'Teen births_Teen Birth Rate (White) 95% CI - Low',
         'Teen births_Teen Birth Rate (White) 95% CI - High',
         'Uninsured_95% CI - Low',
         'Uninsured_95% CI - High',
         'Some college_95% CI - Low',
         'Some college_95% CI - High',
         'Children in poverty_95% CI - Low',
         'Children in poverty_95% CI - High',
         'Children in single-parent households_95% CI - Low',
         'Children in single-parent households_95% CI - High',
         'Injury deaths_95% CI - Low',
         'Injury deaths_95% CI - High',
         'Injury deaths_Injury Death Rate (AIAN) 95% CI - Low',
         'Injury deaths_Injury Death Rate (AIAN) 95% CI - High',
         'Injury deaths_Injury Death Rate (Asian) 95% CI - Low',
         'Injury deaths_Injury Death Rate (Asian) 95% CI - High',
         'Injury deaths_Injury Death Rate (Black) 95% CI - Low',
         'Injury deaths_Injury Death Rate (Black) 95% CI - High',
         'Injury deaths_Injury Death Rate (Hispanic) 95% CI - Low',
         'Injury deaths_Injury Death Rate (Hispanic) 95% CI - High',
         'Injury deaths_Injury Death Rate (White) 95% CI - Low',
         'Injury deaths_Injury Death Rate (White) 95% CI - High',
         'Severe housing problems_95% CI - Low',
         'Severe housing problems_95% CI - High',
         'Severe housing problems_Severe Housing Cost Burden 95% CI - Low',
         'Severe housing problems_Severe Housing Cost Burden 95% CI - High',
         'Severe housing problems_Overcrowding 95% CI - Low',
         'Severe housing problems_Overcrowding 95% CI - High',
         'Severe housing problems_Inadequate Facilities 95% CI - Low',
         'Severe housing problems_Inadequate Facilities 95% CI - High',
         'Driving alone to work_95% CI - Low',
         'Driving alone to work_95% CI - High',
         'Driving alone to work_% Drive Alone (AIAN) 95% CI - Low',
         'Driving alone to work_% Drive Alone (AIAN) 95% CI - High',
         'Driving alone to work_% Drive Alone (Asian) 95% CI - Low',
         'Driving alone to work_% Drive Alone (Asian) 95% CI - High',
         'Driving alone to work_% Drive Alone (Black) 95% CI - Low',
         'Driving alone to work_% Drive Alone (Black) 95% CI - High',
         'Driving alone to work_% Drive Alone (Hispanic) 95% CI - Low',
         'Driving alone to work_% Drive Alone (Hispanic) 95% CI - High',
         'Driving alone to work_% Drive Alone (White) 95% CI - Low',
         'Driving alone to work_% Drive Alone (White) 95% CI - High',
         'Long commute - driving alone_95% CI - Low',
         'Long commute - driving alone_95% CI - High',
         'Poor or fair health_Z-Score',
         'Poor physical health days_Z-Score',
         'Poor mental health days_Z-Score',
         'Low birthweight_Z-Score',
         'Adult smoking_Z-Score',
         'Adult obesity_Z-Score',
         'Food environment index_Z-Score',
         'Physical inactivity_Z-Score',
         'Access to exercise opportunities_Z-Score',
         'Excessive drinking_Z-Score',
         'Alcohol-impaired driving deaths_Z-Score',
         'Sexually transmitted infections_Z-Score',
         'Teen births_Z-Score',
         'Uninsured_Z-Score',
         'Primary care physicians_Z-Score',
         'Dentists_Z-Score',
         'Mental health providers_Z-Score',
         'Preventable hospital stays_Z-Score',
         'Mammography screening_Z-Score',
         'Flu vaccinations_Z-Score',
         'High school graduation_Z-Score',
         'Some college_Z-Score',
         'Unemployment_Z-Score',
         'Children in poverty_Z-Score',
         'Income inequality_Z-Score',
         'Children in single-parent households_Z-Score',
         'Social associations_Z-Score',
         'Violent crime_Z-Score',
         'Injury deaths_Z-Score',
         'Drinking water violations_Z-Score',
         'Severe housing problems_Z-Score',
         'Driving alone to work_Z-Score',
         'Long commute - driving alone_Z-Score',
         'Premature death_YPLL Rate (AIAN)',
         'Premature death_YPLL Rate (Asian)',
         'Premature death_YPLL Rate (Black)',
         'Premature death_YPLL Rate (Hispanic)',
         'Premature death_YPLL Rate (White)',
         'Low birthweight_% LBW (AIAN)',
         'Low birthweight_% LBW (Asian)',
         'Low birthweight_% LBW (Black)',
         'Low birthweight_% LBW (Hispanic)',
         'Low birthweight_% LBW (White)',
         'Teen births_Teen Birth Rate (AIAN)',
         'Teen births_Teen Birth Rate (Asian)',
         'Teen births_Teen Birth Rate (Black)',
         'Teen births_Teen Birth Rate (Hispanic)',
         'Teen births_Teen Birth Rate (White)',
         'Preventable hospital stays_Preventable Hosp. Rate (AIAN)',
         'Preventable hospital stays_Preventable Hosp. Rate (Asian)',
         'Preventable hospital stays_Preventable Hosp. Rate (Black)',
         'Preventable hospital stays_Preventable Hosp. Rate (Hispanic)',
         'Preventable hospital stays_Preventable Hosp. Rate (White)',
         'Mammography screening_% Screened (AIAN)',
         'Mammography screening_% Screened (Asian)',
         'Mammography screening_% Screened (Black)',
         'Mammography screening_% Screened (Hispanic)',
         'Mammography screening_% Screened (White)',
         'Flu vaccinations_% Vaccinated (AIAN)',
         'Flu vaccinations_% Vaccinated (Asian)',
         'Flu vaccinations_% Vaccinated (Black)',
         'Flu vaccinations_% Vaccinated (Hispanic)',
         'Flu vaccinations_% Vaccinated (White)',
         'Children in poverty_% Children in Poverty (AIAN)',
         'Children in poverty_% Children in Poverty (Asian)',
         'Children in poverty_% Children in Poverty (Black)',
         'Children in poverty_% Children in Poverty (Hispanic)',
         'Children in poverty_% Children in Poverty (White)',
         'Injury deaths_Injury Death Rate (AIAN)',
         'Injury deaths_Injury Death Rate (Asian)',
         'Injury deaths_Injury Death Rate (Black)',
         'Injury deaths_Injury Death Rate (Hispanic)',
         'Injury deaths_Injury Death Rate (White)',
         'Driving alone to work_% Drive Alone (AIAN)',
         'Driving alone to work_% Drive Alone (Asian)',
         'Driving alone to work_% Drive Alone (Black)',
         'Driving alone to work_% Drive Alone (Hispanic)',
         'Driving alone to work_% Drive Alone (White)',
         'Flu vaccinations_% Vaccinated',
         'Air pollution - particulate matter_Z-Score',
         'Alcohol-impaired driving deaths_# Alcohol-Impaired Driving Deaths',
         'Alcohol-impaired driving deaths_# Driving Deaths',
         'Sexually transmitted infections_# Chlamydia Cases',
         'Uninsured_# Uninsured',
         'Primary care physicians_# Primary Care Physicians',
         'Primary care physicians_Primary Care Physicians Ratio',
         'Dentists_# Dentists',
         'Dentists_Dentist Ratio',
         'Mental health providers_# Mental Health Providers',
         'Mental health providers_Mental Health Provider Ratio',
         'Mammography screening_% With Annual Mammogram',
         'High school graduation_Cohort Size',
         'Some college_# Some College',
         'Some college_Population',
         'Unemployment_# Unemployed',
         'Unemployment_Labor Force',
         'Income inequality_Income Ratio',
         'Children in single-parent households_# Single-Parent Households',
         'Children in single-parent households_# Households',
         'Social associations_# Associations',
         'Injury deaths_# Injury Deaths',
         'Severe housing problems_Severe Housing Cost Burden',
         'Severe housing problems_Overcrowding',
         'Severe housing problems_Inadequate Facilities',
         'Driving alone to work_% Drive Alone to Work',
         'Long commute - driving alone_# Workers who Drive Alone',
         'Violent crime_Annual Average Violent Crimes']
                     ,axis=1)
        
        
        # Clean Additional Data Second by addressing multiindex column names
        df2.columns = df2.columns.map('_'.join)
        
        # Rename Columns
        df2.rename(columns = {'Unnamed: 0_level_0_FIPS':'FIPS',
         'Unnamed: 1_level_0_State':'State',
         'Unnamed: 2_level_0_County':'County/Borough/Parish',
         'Unnamed: 2_level_0_Borough':'County/Borough/Parish',
         'Unnamed: 2_level_0_Parish':'County/Borough/Parish',
         'Premature age-adjusted mortality_# Deaths':'Premature_Deaths',
         'Premature age-adjusted mortality_Age-Adjusted Death Rate':'Premature(under75)_Age_Adjusted_Mortality_per_100K',
         'Child mortality_Child Mortality Rate':'Child(under18)_Mortality_per_100K',
         'Infant mortality_Infant Mortality Rate':'Infant(within1yr)_Mortality_per_1K_live_births',
         'Diabetes prevalence_% Adults with Diabetes':'%_Adults(20+)_Diagnosed_Diabetes',
         'HIV prevalence_HIV Prevalence Rate':'#_Ppl(Age13+)_diagnosed_HIV_per_100K',
         'Food insecurity_% Food Insecure':'%_Pop_w/o_Adequate_Access_to_Food',
         'Drug overdose deaths_Drug Overdose Mortality Rate':'Drug_Overdose_Deaths_per_100K',
         'Motor vehicle crash deaths_Motor Vehicle Mortality Rate':'Motor_Vehicle_Deaths_per_100K',
         'Uninsured children_% Uninsured':'%_Children(under19)_w/o_Health_Insurance',
         'Other primary care providers_Other Primary Care Provider Rate':'Other_Primary_Care_Providers(non_physician)_per_100K',
         'Median household income_Median Household Income':'Median_Household_Income',
         'Children eligible for free or reduced price lunch_% Enrolled in Free or Reduced Lunch':'%_Children_Enrolled_in_Free/Reduced_Lunch',
         'Homicides_Homicide Rate':'#_Deaths_due_to_Homicide_per_100K',
         'Demographics_Population':'Resident_Population',
         'Demographics_% less than 18 years of age':'%_under_18_years_of_age',
         'Demographics_% 65 and over':'%_Age_65_and_over',
         'Demographics_# Black':'#_Non-Hispanic_Black_or_African_American',
         'Demographics_% Black':'%_Non-Hispanic_Black_or_African_American',
         'Demographics_# American Indian & Alaska Native':'#_American_Indian_or_Alaskan_Native',
         'Demographics_% American Indian & Alaska Native':'%_American_Indian_or_Alaskan_Native',
         'Demographics_# Asian':'#_Asian',
         'Demographics_% Asian':'%_Asian',
         'Demographics_# Native Hawaiian/Other Pacific Islander':'#_Native_Hawaiian/Other_Pacific_Islander',
         'Demographics_% Native Hawaiian/Other Pacific Islander':'%_Native_Hawaiian/Other_Pacific_Islander',
         'Demographics_# Hispanic':'#_Hispanic',
         'Demographics_% Hispanic':'%_Hispanic',
         'Demographics_# Non-Hispanic White':'#_Non-Hispanic_White',
         'Demographics_% Non-Hispanic White':'%_Non-Hispanic_White',
         'Demographics_# Not Proficient in English':'#_Not_Proficient_in_English',
         'Demographics_% Not Proficient in English':'%_Not_Proficient_in_English',
         'Demographics_% Female':'%_Female',
         'Demographics_# Rural':'#_Pop_Living_in_Rural_Area',
         'Demographics_% Rural':'%_Rural'}, 
                   inplace = True)
        
        # Drop unnecessary columns
        df2 = df2.drop(['Life expectancy_Life Expectancy',
         'Life expectancy_95% CI - Low',
         'Life expectancy_95% CI - High',
         'Life expectancy_Life Expectancy (AIAN)',
         'Life expectancy_Life Expectancy (AIAN) 95% CI - Low',
         'Life expectancy_Life Expectancy (AIAN) 95% CI - High',
         'Life expectancy_Life Expectancy (Asian)',
         'Life expectancy_Life Expectancy (Asian) 95% CI - Low',
         'Life expectancy_Life Expectancy (Asian) 95% CI - High',
         'Life expectancy_Life Expectancy (Black)',
         'Life expectancy_Life Expectancy (Black) 95% CI - Low',
         'Life expectancy_Life Expectancy (Black) 95% CI - High',
         'Life expectancy_Life Expectancy (Hispanic)',
         'Life expectancy_Life Expectancy (Hispanic) 95% CI - Low',
         'Life expectancy_Life Expectancy (Hispanic) 95% CI - High',
         'Life expectancy_Life Expectancy (White)',
         'Life expectancy_Life Expectancy (White) 95% CI - Low',
         'Life expectancy_Life Expectancy (White) 95% CI - High',
         'Premature age-adjusted mortality_95% CI - Low',
         'Premature age-adjusted mortality_95% CI - High',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (AIAN)',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (AIAN) 95% CI - Low',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (AIAN) 95% CI - High',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (Asian)',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (Asian) 95% CI - Low',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (Asian) 95% CI - High',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (Black)',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (Black) 95% CI - Low',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (Black) 95% CI - High',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (Hispanic)',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (Hispanic) 95% CI - Low',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (Hispanic) 95% CI - High',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (White)',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (White) 95% CI - Low',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (White) 95% CI - High',
         'Child mortality_95% CI - Low',
         'Child mortality_95% CI - High',
         'Child mortality_Child Mortality Rate (AIAN)',
         'Child mortality_Child Mortality Rate (AIAN) 95% CI - Low',
         'Child mortality_Child Mortality Rate (AIAN) 95% CI - High',
         'Child mortality_Child Mortality Rate (Asian)',
         'Child mortality_Child Mortality Rate (Asian) 95% CI - Low',
         'Child mortality_Child Mortality Rate (Asian) 95% CI - High',
         'Child mortality_Child Mortality Rate (Black)',
         'Child mortality_Child Mortality Rate (Black) 95% CI - Low',
         'Child mortality_Child Mortality Rate (Black) 95% CI - High',
         'Child mortality_Child Mortality Rate (Hispanic)',
         'Child mortality_Child Mortality Rate (Hispanic) 95% CI - Low',
         'Child mortality_Child Mortality Rate (Hispanic) 95% CI - High',
         'Child mortality_Child Mortality Rate (White)',
         'Child mortality_Child Mortality Rate (White) 95% CI - Low',
         'Child mortality_Child Mortality Rate (White) 95% CI - High',
         'Infant mortality_95% CI - Low',
         'Infant mortality_95% CI - High',
         'Infant mortality_Infant Mortality Rate (AIAN)',
         'Infant mortality_Infant Mortality Rate (AIAN) 95% CI - Low',
         'Infant mortality_Infant Mortality Rate (AIAN) 95% CI - High',
         'Infant mortality_Infant Mortality Rate (Asian)',
         'Infant mortality_Infant Mortality Rate (Asian) 95% CI - Low',
         'Infant mortality_Infant Mortality Rate (Asian) 95% CI - High',
         'Infant mortality_Infant Mortality Rate (Black)',
         'Infant mortality_Infant Mortality Rate (Black) 95% CI - Low',
         'Infant mortality_Infant Mortality Rate (Black) 95% CI - High',
         'Infant mortality_Infant Mortality Rate (Hispanic)',
         'Infant mortality_Infant Mortality Rate (Hispanic) 95% CI - Low',
         'Infant mortality_Infant Mortality Rate (Hispanic) 95% CI - High',
         'Infant mortality_Infant Mortality Rate (White)',
         'Infant mortality_Infant Mortality Rate (White) 95% CI - Low',
         'Infant mortality_Infant Mortality Rate (White) 95% CI - High',
         'Frequent physical distress_% Frequent Physical Distress',
         'Frequent physical distress_95% CI - Low',
         'Frequent physical distress_95% CI - High',
         'Frequent mental distress_% Frequent Mental Distress',
         'Frequent mental distress_95% CI - Low',
         'Frequent mental distress_95% CI - High',
         'Diabetes prevalence_95% CI - Low',
         'Diabetes prevalence_95% CI - High',
         'Drug overdose deaths_95% CI - Low',
         'Drug overdose deaths_95% CI - High',
         'Drug overdose deaths_Drug Overdose Mortality Rate (AIAN)',
         'Drug overdose deaths_Drug Overdose Mortality Rate (AIAN) 95% CI - Low',
         'Drug overdose deaths_Drug Overdose Mortality Rate (AIAN) 95% CI - High',
         'Drug overdose deaths_Drug Overdose Mortality Rate (Asian)',
         'Drug overdose deaths_Drug Overdose Mortality Rate (Asian) 95% CI - Low',
         'Drug overdose deaths_Drug Overdose Mortality Rate (Asian) 95% CI - High',
         'Drug overdose deaths_Drug Overdose Mortality Rate (Black)',
         'Drug overdose deaths_Drug Overdose Mortality Rate (Black) 95% CI - Low',
         'Drug overdose deaths_Drug Overdose Mortality Rate (Black) 95% CI - High',
         'Drug overdose deaths_Drug Overdose Mortality Rate (Hispanic)',
         'Drug overdose deaths_Drug Overdose Mortality Rate (Hispanic) 95% CI - Low',
         'Drug overdose deaths_Drug Overdose Mortality Rate (Hispanic) 95% CI - High',
         'Drug overdose deaths_Drug Overdose Mortality Rate (White)',
         'Drug overdose deaths_Drug Overdose Mortality Rate (White) 95% CI - Low',
         'Drug overdose deaths_Drug Overdose Mortality Rate (White) 95% CI - High',
         'Motor vehicle crash deaths_95% CI - Low',
         'Motor vehicle crash deaths_95% CI - High',
         'Motor vehicle crash deaths_MV Mortality Rate (AIAN)',
         'Motor vehicle crash deaths_MV Mortality Rate (AIAN) 95% CI - Low',
         'Motor vehicle crash deaths_MV Mortality Rate (AIAN) 95% CI - High',
         'Motor vehicle crash deaths_MV Mortality Rate (Asian)',
         'Motor vehicle crash deaths_MV Mortality Rate (Asian) 95% CI - Low',
         'Motor vehicle crash deaths_MV Mortality Rate (Asian) 95% CI - High',
         'Motor vehicle crash deaths_MV Mortality Rate (Black)',
         'Motor vehicle crash deaths_MV Mortality Rate (Black) 95% CI - Low',
         'Motor vehicle crash deaths_MV Mortality Rate (Black) 95% CI - High',
         'Motor vehicle crash deaths_MV Mortality Rate (Hispanic)',
         'Motor vehicle crash deaths_MV Mortality Rate (Hispanic) 95% CI - Low',
         'Motor vehicle crash deaths_MV Mortality Rate (Hispanic) 95% CI - High',
         'Motor vehicle crash deaths_MV Mortality Rate (White)',
         'Motor vehicle crash deaths_MV Mortality Rate (White) 95% CI - Low',
         'Motor vehicle crash deaths_MV Mortality Rate (White) 95% CI - High',
         'Insufficient sleep_% Insufficient Sleep',
         'Insufficient sleep_95% CI - Low',
         'Insufficient sleep_95% CI - High',
         'Uninsured adults_95% CI - Low',
         'Uninsured adults_95% CI - High',
         'Uninsured children_95% CI - Low',
         'Uninsured children_95% CI - High',
         'Disconnected youth_% Disconnected Youth',
         'Disconnected youth_95% CI - Low',
         'Disconnected youth_95% CI - High',
         'Reading scores_Average Grade Performance',
         'Reading scores_Average Grade Performance (Asian)',
         'Reading scores_Average Grade Performance (Black)',
         'Reading scores_Average Grade Performance (Hispanic)',
         'Reading scores_Average Grade Performance (White)',
         'Math scores_Average Grade Performance',
         'Math scores_Average Grade Performance (Asian)',
         'Math scores_Average Grade Performance (Black)',
         'Math scores_Average Grade Performance (Hispanic)',
         'Math scores_Average Grade Performance (White)',
         'Median household income_95% CI - Low',
         'Median household income_95% CI - High',
         'Median household income_Household Income (AIAN)',
         'Median household income_Household Income (AIAN) 95% CI - Low',
         'Median household income_Household Income (AIAN) 95% CI - High',
         'Median household income_Household Income (Asian)',
         'Median household income_Household Income (Asian) 95% CI - Low',
         'Median household income_Household Income (Asian) 95% CI - High',
         'Median household income_Household Income (Black)',
         'Median household income_Household Income (Black) 95% CI - Low',
         'Median household income_Household Income (Black) 95% CI - High',
         'Median household income_Household Income (Hispanic)',
         'Median household income_Household Income (Hispanic) 95% CI - Low',
         'Median household income_Household Income (Hispanic) 95% CI - High',
         'Median household income_Household Income (White)',
         'Median household income_Household Income (White) 95% CI - Low',
         'Median household income_Household Income (White) 95% CI - High',
         'Residential segregation - Black/White_Segregation index',
         'Residential segregation - non-White/White_Segregation Index',
         'Homicides_95% CI - Low',
         'Homicides_95% CI - High',
         'Homicides_Homicide Rate (AIAN)',
         'Homicides_Homicide Rate (AIAN) 95% CI - Low',
         'Homicides_Homicide Rate (AIAN) 95% CI - High',
         'Homicides_Homicide Rate (Asian)',
         'Homicides_Homicide Rate (Asian) 95% CI - Low',
         'Homicides_Homicide Rate (Asian) 95% CI - High',
         'Homicides_Homicide Rate (Black)',
         'Homicides_Homicide Rate (Black) 95% CI - Low',
         'Homicides_Homicide Rate (Black) 95% CI - High',
         'Homicides_Homicide Rate (Hispanic)',
         'Homicides_Homicide Rate (Hispanic) 95% CI - Low',
         'Homicides_Homicide Rate (Hispanic) 95% CI - High',
         'Homicides_Homicide Rate (White)',
         'Homicides_Homicide Rate (White) 95% CI - Low',
         'Homicides_Homicide Rate (White) 95% CI - High',
         'Suicides_# Deaths',
         'Suicides_Suicide Rate (Age-Adjusted)',
         'Suicides_95% CI - Low',
         'Suicides_95% CI - High',
         'Suicides_Crude Rate',
         'Suicides_Suicide Rate (AIAN)',
         'Suicides_Suicide Rate (AIAN) 95% CI - Low',
         'Suicides_Suicide Rate (AIAN) 95% CI - High',
         'Suicides_Suicide Rate (Asian)',
         'Suicides_Suicide Rate (Asian) 95% CI - Low',
         'Suicides_Suicide Rate (Asian) 95% CI - High',
         'Suicides_Suicide Rate (Black)',
         'Suicides_Suicide Rate (Black) 95% CI - Low',
         'Suicides_Suicide Rate (Black) 95% CI - High',
         'Suicides_Suicide Rate (Hispanic)',
         'Suicides_Suicide Rate (Hispanic) 95% CI - Low',
         'Suicides_Suicide Rate (Hispanic) 95% CI - High',
         'Suicides_Suicide Rate (White)',
         'Suicides_Suicide Rate (White) 95% CI - Low',
         'Suicides_Suicide Rate (White) 95% CI - High',
         'Firearm fatalities_# Firearm Fatalities',
         'Firearm fatalities_Firearm Fatalities Rate',
         'Firearm fatalities_95% CI - Low',
         'Firearm fatalities_95% CI - High',
         'Firearm fatalities_Firearm Fatalities Rate (AIAN)',
         'Firearm fatalities_Firearm Fatalities Rate (AIAN) 95% CI - Low',
         'Firearm fatalities_Firearm Fatalities Rate (AIAN) 95% CI - High',
         'Firearm fatalities_Firearm Fatalities Rate (Asian)',
         'Firearm fatalities_Firearm Fatalities Rate (Asian) 95% CI - Low',
         'Firearm fatalities_Firearm Fatalities Rate (Asian) 95% CI - High',
         'Firearm fatalities_Firearm Fatalities Rate (Black)',
         'Firearm fatalities_Firearm Fatalities Rate (Black) 95% CI - Low',
         'Firearm fatalities_Firearm Fatalities Rate (Black) 95% CI - High',
         'Firearm fatalities_Firearm Fatalities Rate (Hispanic)',
         'Firearm fatalities_Firearm Fatalities Rate (Hispanic) 95% CI - Low',
         'Firearm fatalities_Firearm Fatalities Rate (Hispanic) 95% CI - High',
         'Firearm fatalities_Firearm Fatalities Rate (White)',
         'Firearm fatalities_Firearm Fatalities Rate (White) 95% CI - Low',
         'Firearm fatalities_Firearm Fatalities Rate (White) 95% CI - High',
         'Juvenile arrests_Non-Petitioned Cases',
         'Juvenile arrests_Petitioned Cases',
         'Juvenile arrests_Denominator',
         'Juvenile arrests_Juvenile Arrest Rate',
         'Traffic volume_Average Traffic Volume per Meter of Major Roadways',
         'Homeownership_# Homeowners',
         'Homeownership_% Homeowners',
         'Homeownership_95% CI - Low',
         'Homeownership_95% CI - High',
         'Severe housing cost burden_# Households with Severe Cost Burden',
         'Severe housing cost burden_% Severe Housing Cost Burden',
         'Severe housing cost burden_95% CI - Low',
         'Severe housing cost burden_95% CI - High',
         'Demographics_95% CI - Low',
         'Demographics_95% CI - High',
         'Child mortality_# Deaths',
         'Infant mortality_# Deaths',
         'HIV prevalence_# HIV Cases',
         'Food insecurity_# Food Insecure',
         'Limited access to healthy foods_# Limited Access',
         'Limited access to healthy foods_% Limited Access to Healthy Foods',
         'Drug overdose deaths_# Drug Overdose Deaths',
         'Motor vehicle crash deaths_# Motor Vehicle Deaths',
         'Uninsured adults_# Uninsured',
         'Uninsured children_# Uninsured',
         'Other primary care providers_Other Primary Care Provider Ratio',
         'Uninsured adults_% Uninsured']
                       ,axis=1)
        
        df3.columns = df3.columns.map('_'.join)
        df3.rename(columns = {
         'Unnamed: 0_level_0_FIPS':'FIPS',
         'Unnamed: 1_level_0_State':'State',
         'Unnamed: 2_level_0_County':'County/Borough/Parish',
         'Unnamed: 2_level_0_Borough':'County/Borough/Parish',
         'Unnamed: 2_level_0_Parish':'County/Borough/Parish'}, 
                   inplace = True)
        df3 = df3.drop(['Health Outcomes_Z-Score','Health Factors_Z-Score','Health Factors_Rank'], axis=1)
        
        # Join two dataframes
        joined_df = pd.merge(df, df2, on='FIPS', how='outer')
        joined_df = joined_df.drop(['State_y','County/Borough/Parish_y'],axis=1)
        joined_df.rename(columns = {'State_x':'State','County/Borough/Parish_x':'County/Borough/Parish'},inplace = True)
        
        joined_df = pd.merge(joined_df, df3, on='FIPS', how='outer')
        joined_df = joined_df.drop(['State_y','County/Borough/Parish_y'],axis=1)
        joined_df.rename(columns = {'State_x':'State','County/Borough/Parish_x':'County/Borough/Parish'},inplace = True)
        
        joined_df['Year'] = '2020'
        
        # append to list
        appended_data2020.append(joined_df)
        
#convert to dataframe
appended_data2020 = pd.concat(appended_data2020)

2020 County Health Rankings Alabama Data - v1_1.xlsx
2020 County Health Rankings Alaska Data - v1_0.xlsx
2020 County Health Rankings Arizona Data - v1_0.xlsx
2020 County Health Rankings Arkansas Data - v1_0.xlsx
2020 County Health Rankings California Data - v1_0.xlsx
2020 County Health Rankings Colorado Data - v1_0.xlsx
2020 County Health Rankings Connecticut Data - v1_0.xlsx
2020 County Health Rankings Delaware Data - v1.xlsx
2020 County Health Rankings District of Columbia Data - v1_0.xlsx
2020 County Health Rankings Florida Data - v1_0.xlsx
2020 County Health Rankings Georgia Data - v1_0.xlsx
2020 County Health Rankings Hawaii Data - v1_0.xlsx
2020 County Health Rankings Idaho Data - v1_0.xlsx
2020 County Health Rankings Illinois Data - v1_0.xlsx
2020 County Health Rankings Indiana Data - v1_0.xlsx
2020 County Health Rankings Iowa Data - v1_0.xlsx
2020 County Health Rankings Kansas Data - v1_0.xlsx
2020 County Health Rankings Kentucky Data - v1_0.xlsx
2020 County Health Rankings Lou

In [3]:
appended_data2020.head(20)

,FIPS,State,County/Borough/Parish,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,%_Low_Birthweight_Births,%_Adult_Smokers,%_Adult_Obesity,"Healthy_Food_Access(0=Worst,10=Best)",%_Adults_Physical_Inactivity,...,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Health Outcomes_Rank,Year
0,1000,Alabama,NaN,9942.794666,22.028703,10.254871,20.927353,35.5,5.8,29.8,...,4.443264,3197324,65.413428,48517,1.061048,51.633032,1957932.0,40.963183,NaN,2020
1,1001,Alabama,Autauga,8128.591190,20.882987,8.619529,18.081557,33.3,7.2,34.7,...,2.965774,41316,74.308016,426,0.820225,51.448715,22921.0,42.002162,6,2020
2,1003,Alabama,Baldwin,7354.122530,17.509134,8.345003,17.489033,31.0,8.0,26.5,...,4.646779,181201,83.111337,1068,0.543517,51.538377,77060.0,42.279099,2,2020
3,1005,Alabama,Barbour,10253.573403,29.591802,11.474559,21.999985,41.7,5.6,23.5,...,4.276355,11356,45.641252,398,1.631683,47.216752,18613.0,67.789635,45,2020
4,1007,Alabama,Bibb,11977.539484,19.439724,10.308710,19.114200,37.6,7.8,33.5,...,2.625000,16708,74.589286,57,0.268210,46.781250,15663.0,68.352607,34,2020
5,1009,Alabama,Blount,11335.071134,21.745293,7.604563,19.208672,33.8,8.4,30.3,...,9.571231,50255,86.886238,934,1.724520,50.726141,51562.0,89.951502,24,2020
6,1011,Alabama,Bullock,11679.558981,30.999102,15.688488,22.894664,37.2,4.3,24.6,...,7.960150,2151,21.217203,43,0.440754,45.482344,5607.0,51.374382,56,2020
7,1013,Alabama,Butler,14359.939103,27.910673,12.680288,21.765335,43.3,6.6,39.5,...,1.509146,10087,51.255081,93,0.494155,53.429878,14921.0,71.232157,64,2020
8,1015,Alabama,Calhoun,12078.616145,23.107733,9.161374,20.612560,38.5,6.9,31.7,...,3.910673,82308,72.024992,1076,0.991376,51.946586,39955.0,33.696826,42,2020
9,1017,Alabama,Chambers,11112.536614,23.995474,12.213740,19.392927,40.1,6.4,30.1,...,2.561357,18566,55.231296,36,0.112938,52.125539,16816.0,49.148034,44,2020


In [4]:
list(appended_data2020.columns.values)

['FIPS',
 'State',
 'County/Borough/Parish',
 'Yrs_Potential_Life_Lost_per100K(YPLL)',
 '%_Fair/Poor_Health',
 '%_Low_Birthweight_Births',
 '%_Adult_Smokers',
 '%_Adult_Obesity',
 'Healthy_Food_Access(0=Worst,10=Best)',
 '%_Adults_Physical_Inactivity',
 '%_Access_to_Excercise_Opportunities',
 '%_Adults_Excessive_Drinking',
 '%_Driving_Deaths_with_Alcohol_Involvement',
 'Chlamydia_Cases_per_100K',
 'Teen_Birth_Rate',
 '%_Ppl_Under65_w/o_Insurance',
 'PCP_per_100K',
 'Dentists_per_100K',
 'MHP_per_100K',
 'Preventable_Hospitalization_Rate(per100K_Medicare_Enrollees)',
 'Graduation_Rate',
 '%_Adults(25-44)_with_Some_Post-secondary_Ed',
 '%_Ppl(16+)_Unemployed_&_Looking_for_Work',
 '%_Children(under18)_Living_in_Poverty',
 '80th_Percentile_MHI',
 '20th_Percentile_MHI',
 '%_Children_Living_in_Single_Parent_Household',
 'Associations_per_10K',
 'Violent_Crimes_per_100K',
 'Injury_Death_Rate_per_100K',
 'Avg_Daily_Particulate_Matter_in_Micrograms_per_Cubic_Meter',
 'County_Affected_by_Water_V

In [5]:
appended_data2020.count()

FIPS                                                            3193
State                                                           3193
County/Borough/Parish                                           3142
Yrs_Potential_Life_Lost_per100K(YPLL)                           2900
%_Fair/Poor_Health                                              3193
%_Low_Birthweight_Births                                        3086
%_Adult_Smokers                                                 3193
%_Adult_Obesity                                                 3193
Healthy_Food_Access(0=Worst,10=Best)                            3174
%_Adults_Physical_Inactivity                                    3193
%_Access_to_Excercise_Opportunities                             3187
%_Adults_Excessive_Drinking                                     3193
%_Driving_Deaths_with_Alcohol_Involvement                       3158
Chlamydia_Cases_per_100K                                        3041
Teen_Birth_Rate                   

In [6]:
appended_data2020.isnull().any()

FIPS                                                            False
State                                                           False
County/Borough/Parish                                            True
Yrs_Potential_Life_Lost_per100K(YPLL)                            True
%_Fair/Poor_Health                                              False
%_Low_Birthweight_Births                                         True
%_Adult_Smokers                                                 False
%_Adult_Obesity                                                 False
Healthy_Food_Access(0=Worst,10=Best)                             True
%_Adults_Physical_Inactivity                                    False
%_Access_to_Excercise_Opportunities                              True
%_Adults_Excessive_Drinking                                     False
%_Driving_Deaths_with_Alcohol_Involvement                        True
Chlamydia_Cases_per_100K                                         True
Teen_Birth_Rate     

In [7]:
appended_data2020.loc[appended_data2020.isnull().any(axis=1)]

,FIPS,State,County/Borough/Parish,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,%_Low_Birthweight_Births,%_Adult_Smokers,%_Adult_Obesity,"Healthy_Food_Access(0=Worst,10=Best)",%_Adults_Physical_Inactivity,...,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Health Outcomes_Rank,Year
0,1000,Alabama,NaN,9942.794666,22.028703,10.254871,20.927353,35.5,5.8,29.8,...,4.443264,3197324,65.413428,48517,1.061048,51.633032,1957932.0,40.963183,NaN,2020
3,1005,Alabama,Barbour,10253.573403,29.591802,11.474559,21.999985,41.7,5.6,23.5,...,4.276355,11356,45.641252,398,1.631683,47.216752,18613.0,67.789635,45,2020
6,1011,Alabama,Bullock,11679.558981,30.999102,15.688488,22.894664,37.2,4.3,24.6,...,7.960150,2151,21.217203,43,0.440754,45.482344,5607.0,51.374382,56,2020
7,1013,Alabama,Butler,14359.939103,27.910673,12.680288,21.765335,43.3,6.6,39.5,...,1.509146,10087,51.255081,93,0.494155,53.429878,14921.0,71.232157,64,2020
10,1019,Alabama,Cherokee,12349.707594,20.698471,8.272506,17.498674,35.0,8.3,31.3,...,1.621082,23860,91.656423,99,0.401639,50.460971,22282.0,85.736273,37,2020
12,1023,Alabama,Choctaw,12479.822147,24.863531,11.018711,18.793248,41.3,6.4,36.2,...,1.043532,7241,56.389689,51,0.411158,52.729538,13859.0,100.000000,52,2020
13,1025,Alabama,Clarke,11144.037371,25.235085,13.108039,20.579655,34.1,5.4,34.6,...,1.362876,12560,52.508361,30,0.130304,52.642140,19628.0,75.980335,49,2020
14,1027,Alabama,Clay,10461.834320,23.795917,9.356137,19.620354,40.6,7.8,32.6,...,3.013183,10690,80.527307,51,0.401322,51.096045,13932.0,100.000000,25,2020
15,1029,Alabama,Cleburne,11087.984454,19.284408,8.520900,18.374466,37.3,8.0,31.3,...,2.482151,13907,92.793755,100,0.708366,50.483753,14972.0,100.000000,21,2020
18,1035,Alabama,Conecuh,10822.452428,28.608748,14.064116,21.107513,43.0,5.6,40.7,...,2.150362,6176,50.305449,0,0.000000,52.097418,10708.0,80.949501,53,2020


In [8]:
appended_data2020 = appended_data2020.fillna(value=0)
appended_data2020.head(20)

,FIPS,State,County/Borough/Parish,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,%_Low_Birthweight_Births,%_Adult_Smokers,%_Adult_Obesity,"Healthy_Food_Access(0=Worst,10=Best)",%_Adults_Physical_Inactivity,...,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Health Outcomes_Rank,Year
0,1000,Alabama,0,9942.794666,22.028703,10.254871,20.927353,35.5,5.8,29.8,...,4.443264,3197324,65.413428,48517,1.061048,51.633032,1957932.0,40.963183,0,2020
1,1001,Alabama,Autauga,8128.591190,20.882987,8.619529,18.081557,33.3,7.2,34.7,...,2.965774,41316,74.308016,426,0.820225,51.448715,22921.0,42.002162,6,2020
2,1003,Alabama,Baldwin,7354.122530,17.509134,8.345003,17.489033,31.0,8.0,26.5,...,4.646779,181201,83.111337,1068,0.543517,51.538377,77060.0,42.279099,2,2020
3,1005,Alabama,Barbour,10253.573403,29.591802,11.474559,21.999985,41.7,5.6,23.5,...,4.276355,11356,45.641252,398,1.631683,47.216752,18613.0,67.789635,45,2020
4,1007,Alabama,Bibb,11977.539484,19.439724,10.308710,19.114200,37.6,7.8,33.5,...,2.625000,16708,74.589286,57,0.268210,46.781250,15663.0,68.352607,34,2020
5,1009,Alabama,Blount,11335.071134,21.745293,7.604563,19.208672,33.8,8.4,30.3,...,9.571231,50255,86.886238,934,1.724520,50.726141,51562.0,89.951502,24,2020
6,1011,Alabama,Bullock,11679.558981,30.999102,15.688488,22.894664,37.2,4.3,24.6,...,7.960150,2151,21.217203,43,0.440754,45.482344,5607.0,51.374382,56,2020
7,1013,Alabama,Butler,14359.939103,27.910673,12.680288,21.765335,43.3,6.6,39.5,...,1.509146,10087,51.255081,93,0.494155,53.429878,14921.0,71.232157,64,2020
8,1015,Alabama,Calhoun,12078.616145,23.107733,9.161374,20.612560,38.5,6.9,31.7,...,3.910673,82308,72.024992,1076,0.991376,51.946586,39955.0,33.696826,42,2020
9,1017,Alabama,Chambers,11112.536614,23.995474,12.213740,19.392927,40.1,6.4,30.1,...,2.561357,18566,55.231296,36,0.112938,52.125539,16816.0,49.148034,44,2020


In [9]:
# appended_data2020.to_csv(os.path.join("Cleaned_Files","2020_County_Health_Ranking_Data.csv"),index=False)

### Loop to move through files within 2019 Data Folder
- join 3 worksheets within each state file together
- rename required columns
- drop unnecessary columns
- append all state data to single dataframe for 2019

In [10]:
appended_data2019 = []
path = os.path.join("Original_Files","CountyHealthRankingsData","2019")
for WorkingFile in os.listdir(path):
        #identifies the list of files to be grouped together from designated folder
        print(WorkingFile)
        
        # Import the excel file by joining the path directory and file name together and call Ranked Data and Additonal Measure tabs
        file = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Ranked Measure Data',header=(0,1))
        file2 = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Additional Measure Data',header=(0,1))
        file3 = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Outcomes & Factors Rankings',header=(0,1))
       
        # Load the excel files as two dataframes
        df = pd.DataFrame(file)
        df2 = pd.DataFrame(file2)
        df3 = pd.DataFrame(file3)
        
        # Code from Data_cleaning
       
        # Clean Ranked Data First by addressing multiindex column names
        df.columns = df.columns.map('_'.join)
        
        # Rename Columns
        df.rename(columns = {'Unnamed: 0_level_0_FIPS' : 'FIPS',
         'Unnamed: 1_level_0_State' :'State',
         'Unnamed: 2_level_0_County':'County/Borough/Parish',
         'Unnamed: 2_level_0_Borough':'County/Borough/Parish',
         'Unnamed: 2_level_0_Parish':'County/Borough/Parish',
         'Premature death_Years of Potential Life Lost Rate':'Yrs_Potential_Life_Lost_per100K(YPLL)',
         'Poor or fair health_% Fair/Poor':'%_Fair/Poor_Health',
         'Low birthweight_% LBW':'%_Low_Birthweight_Births',
         'Adult smoking_% Smokers':'%_Adult_Smokers',
         'Adult obesity_% Obese':'%_Adult_Obesity',
         'Food environment index_Food Environment Index':'Healthy_Food_Access(0=Worst,10=Best)',
         'Physical inactivity_% Physically Inactive':'%_Adults_Physical_Inactivity',
         'Access to exercise opportunities_% With Access':'%_Access_to_Excercise_Opportunities',
         'Excessive drinking_% Excessive Drinking':'%_Adults_Excessive_Drinking',
         'Alcohol-impaired driving deaths_% Alcohol-Impaired':'%_Driving_Deaths_with_Alcohol_Involvement',
         'Sexually transmitted infections_Chlamydia Rate':'Chlamydia_Cases_per_100K',
         'Teen births_Teen Birth Rate':'Teen_Birth_Rate',
         'Uninsured_% Uninsured':'%_Ppl_Under65_w/o_Insurance',                            
         'Primary care physicians_PCP Rate':'PCP_per_100K',
         'Dentists_Dentist Rate':'Dentists_per_100K',
         'Mental health providers_MHP Rate':'MHP_per_100K',
         'Preventable hospital stays_Preventable Hosp. Rate':'Preventable_Hospitalization_Rate(per100K_Medicare_Enrollees)',
         'High school graduation_Graduation Rate':'Graduation_Rate',
         'Some college_% Some College':'%_Adults(25-44)_with_Some_Post-secondary_Ed',
         'Unemployment_% Unemployed':'%_Ppl(16+)_Unemployed_&_Looking_for_Work',
         'Children in poverty_% Children in Poverty':'%_Children(under18)_Living_in_Poverty',
         'Income inequality_80th Percentile Income':'80th_Percentile_MHI',
         'Income inequality_20th Percentile Income':'20th_Percentile_MHI',
         'Children in single-parent households_% Single-Parent Households':'%_Children_Living_in_Single_Parent_Household',
         'Social associations_Association Rate':'Associations_per_10K',
         'Violent crime_Violent Crime Rate':'Violent_Crimes_per_100K',
         'Injury deaths_Injury Death Rate':'Injury_Death_Rate_per_100K',
         'Air pollution - particulate matter_Average Daily PM2.5':'Avg_Daily_Particulate_Matter_in_Micrograms_per_Cubic_Meter',
         'Drinking water violations_Presence of violation':'County_Affected_by_Water_Violation(1=Yes,0=No)',
         'Severe housing problems_% Severe Housing Problems':'%_Households_with_One_or_More Problems(overcrowding,high-housing-cost,lack-kitchen,lack-plumbing-facilities)',
         'Long commute - driving alone_% Long Commute - Drives Alone':'%_Lone_Commuters_who_Commute_More_Than_30Min'}, 
                  inplace = True)
        
        # Drop unnecessary columns
        df = df.drop([
         'Premature death_95% CI - Low',
         'Premature death_95% CI - High',
         'Premature death_Z-Score',
         'Premature death_YPLL Rate (Black)',
         'Premature death_YPLL Rate (Hispanic)',
         'Premature death_YPLL Rate (White)',
         'Poor or fair health_95% CI - Low',
         'Poor or fair health_95% CI - High',
         'Poor or fair health_Z-Score',
         'Poor physical health days_95% CI - Low',
         'Poor physical health days_95% CI - High',
         'Poor physical health days_Z-Score',
         'Poor mental health days_95% CI - Low',
         'Poor mental health days_95% CI - High',
         'Poor mental health days_Z-Score',
         'Low birthweight_95% CI - Low',
         'Low birthweight_95% CI - High',
         'Low birthweight_Z-Score',
         'Low birthweight_% LBW (Black)',
         'Low birthweight_% LBW (Hispanic)',
         'Low birthweight_% LBW (White)',
         'Adult smoking_95% CI - Low',
         'Adult smoking_95% CI - High',
         'Adult smoking_Z-Score',
         'Adult obesity_95% CI - Low',
         'Adult obesity_95% CI - High',
         'Adult obesity_Z-Score',
         'Food environment index_Z-Score',
         'Physical inactivity_95% CI - Low',
         'Physical inactivity_95% CI - High',
         'Physical inactivity_Z-Score',
         'Access to exercise opportunities_Z-Score',
         'Excessive drinking_95% CI - Low',
         'Excessive drinking_95% CI - High',
         'Excessive drinking_Z-Score',
         'Alcohol-impaired driving deaths_95% CI - Low',
         'Alcohol-impaired driving deaths_95% CI - High',
         'Alcohol-impaired driving deaths_Z-Score',
         'Sexually transmitted infections_Z-Score',
         'Teen births_95% CI - Low',
         'Teen births_95% CI - High',
         'Teen births_Z-Score',
         'Teen births_Teen Birth Rate (Black)',
         'Teen births_Teen Birth Rate (Hispanic)',
         'Teen births_Teen Birth Rate (White)',
         'Uninsured_95% CI - Low',
         'Uninsured_95% CI - High',
         'Uninsured_Z-Score',
         'Primary care physicians_Z-Score',
         'Dentists_Z-Score',
         'Mental health providers_Z-Score',
         'Preventable hospital stays_Z-Score',
         'Preventable hospital stays_Preventable Hosp. Rate (Black)',
         'Preventable hospital stays_Preventable Hosp. Rate (Hispanic)',
         'Preventable hospital stays_Preventable Hosp. Rate (White)',
         'Mammography screening_Z-Score',
         'Mammography screening_% Screened (Black)',
         'Mammography screening_% Screened (Hispanic)',
         'Mammography screening_% Screened (White)',
         'Flu vaccinations_% Vaccinated',
         'Flu vaccinations_Z-Score',
         'Flu vaccinations_% Vaccinated (Black)',
         'Flu vaccinations_% Vaccinated (Hispanic)',
         'Flu vaccinations_% Vaccinated (White)',
         'High school graduation_Z-Score',
         'Some college_95% CI - Low',
         'Some college_95% CI - High',
         'Some college_Z-Score',
         'Unemployment_Z-Score',
         'Children in poverty_95% CI - Low',
         'Children in poverty_95% CI - High',
         'Children in poverty_Z-Score',
         'Children in poverty_% Children in Poverty (Black)',
         'Children in poverty_% Children in Poverty (Hispanic)',
         'Children in poverty_% Children in Poverty (White)',
         'Income inequality_Z-Score',
         'Children in single-parent households_95% CI - Low',
         'Children in single-parent households_95% CI - High',
         'Children in single-parent households_Z-Score',
         'Social associations_Z-Score',
         'Violent crime_Z-Score',
         'Injury deaths_95% CI - Low',
         'Injury deaths_95% CI - High',
         'Injury deaths_Z-Score',
         'Air pollution - particulate matter_Z-Score',
         'Drinking water violations_Z-Score',
         'Severe housing problems_95% CI - Low',
         'Severe housing problems_95% CI - High',
         'Severe housing problems_Z-Score',
         'Driving alone to work_95% CI - Low',
         'Driving alone to work_95% CI - High',
         'Driving alone to work_Z-Score',
         'Driving alone to work_% Drive Alone (Black)',
         'Driving alone to work_% Drive Alone (Hispanic)',
         'Driving alone to work_% Drive Alone (White)',
         'Long commute - driving alone_95% CI - Low',
         'Long commute - driving alone_95% CI - High',
         'Long commute - driving alone_Z-Score',
         'Poor physical health days_Physically Unhealthy Days',
         'Poor mental health days_Mentally Unhealthy Days',
         'Low birthweight_Unreliable',
         'Alcohol-impaired driving deaths_# Alcohol-Impaired Driving Deaths',
         'Alcohol-impaired driving deaths_# Driving Deaths',
         'Sexually transmitted infections_# Chlamydia Cases',
         'Uninsured_# Uninsured',
         'Primary care physicians_# Primary Care Physicians',
         'Primary care physicians_PCP Ratio',                           
         'Dentists_# Dentists',
         'Dentists_Dentist Ratio',
         'Mental health providers_# Mental Health Providers',
         'Mental health providers_MHP Ratio',
         'Mammography screening_% Screened',
         'High school graduation_Cohort Size',
         'Some college_# Some College',
         'Some college_Population',
         'Unemployment_# Unemployed',
         'Unemployment_Labor Force',
         'Income inequality_Income Ratio',
         'Children in single-parent households_# Single-Parent Households',
         'Children in single-parent households_# Households',
         'Social associations_# Associations',
         'Injury deaths_# Injury Deaths',
         'Severe housing problems_Severe Housing Cost Burden',
         'Severe housing problems_Overcrowding',
         'Severe housing problems_Inadequate Facilities',
         'Driving alone to work_% Drive Alone',
         'Long commute - driving alone_# Workers who Drive Alone',
         'Violent crime_Annual Average Violent Crimes']
                             ,axis=1)
        
        # Clean Additional Data Second by addressing multiindex column names
        df2.columns = df2.columns.map('_'.join)
        
        # Rename Columns
        df2.rename(columns = {'Unnamed: 0_level_0_FIPS':'FIPS',
         'Unnamed: 1_level_0_State':'State',
         'Unnamed: 2_level_0_County':'County/Borough/Parish',
         'Unnamed: 2_level_0_Borough':'County/Borough/Parish',
         'Unnamed: 2_level_0_Parish':'County/Borough/Parish',
         'Premature age-adjusted mortality_Age-Adjusted Mortality':'Premature(under75)_Age_Adjusted_Mortality_per_100K',
         'Child mortality_Child Mortality Rate':'Child(under18)_Mortality_per_100K',
         'Infant mortality_Infant Mortality Rate':'Infant(within1yr)_Mortality_per_1K_live_births',
         'Diabetes prevalence_% Diabetic':'%_Adults(20+)_Diagnosed_Diabetes',
         'HIV prevalence_HIV Prevalence Rate':'#_Ppl(Age13+)_diagnosed_HIV_per_100K',
         'Food insecurity_% Food Insecure':'%_Pop_w/o_Adequate_Access_to_Food',
         'Drug overdose deaths_Drug Overdose Mortality Rate':'Drug_Overdose_Deaths_per_100K',
         'Motor vehicle crash deaths_MV Mortality Rate':'Motor_Vehicle_Deaths_per_100K',
         'Uninsured children_% Uninsured':'%_Children(under19)_w/o_Health_Insurance',
         'Other primary care providers_Other PCP Rate':'Other_Primary_Care_Providers(non_physician)_per_100K',
         'Median household income_Household Income':'Median_Household_Income',
         'Children eligible for free or reduced price lunch_% Free or Reduced Lunch':'%_Children_Enrolled_in_Free/Reduced_Lunch',
         'Homicides_Homicide Rate':'#_Deaths_due_to_Homicide_per_100K',
         'Demographics_Population':'Resident_Population',
         'Demographics_% < 18':'%_under_18_years_of_age',
         'Demographics_% 65 and over':'%_Age_65_and_over',
         'Demographics_# African American':'#_Non-Hispanic_Black_or_African_American',
         'Demographics_% African American':'%_Non-Hispanic_Black_or_African_American',
         'Demographics_# American Indian/Alaskan Native':'#_American_Indian_or_Alaskan_Native',
         'Demographics_% American Indian/Alaskan Native':'%_American_Indian_or_Alaskan_Native',
         'Demographics_# Asian':'#_Asian',
         'Demographics_% Asian':'%_Asian',
         'Demographics_# Native Hawaiian/Other Pacific Islander':'#_Native_Hawaiian/Other_Pacific_Islander',
         'Demographics_% Native Hawaiian/Other Pacific Islander':'%_Native_Hawaiian/Other_Pacific_Islander',
         'Demographics_# Hispanic':'#_Hispanic',
         'Demographics_% Hispanic':'%_Hispanic',
         'Demographics_# Non-Hispanic White':'#_Non-Hispanic_White',
         'Demographics_% Non-Hispanic White':'%_Non-Hispanic_White',
         'Demographics_# Not Proficient in English':'#_Not_Proficient_in_English',
         'Demographics_% Not Proficient in English':'%_Not_Proficient_in_English',
         'Demographics_% Female':'%_Female',
         'Demographics_# Rural':'#_Pop_Living_in_Rural_Area',
         'Demographics_% Rural':'%_Rural',
         'Premature age-adjusted mortality_# Deaths':'Premature_Deaths'}, 
           inplace = True)
        
        # Drop unnecessary columns
        df2 = df2.drop(['Life expectancy_Life Expectancy',
         'Life expectancy_95% CI - Low',
         'Life expectancy_95% CI - High',
         'Life expectancy_Life Expectancy (Black)',
         'Life expectancy_Life Expectancy (Hispanic)',
         'Life expectancy_Life Expectancy (White)',
         'Premature age-adjusted mortality_95% CI - Low',
         'Premature age-adjusted mortality_95% CI - High',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (Black)',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (Hispanic)',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (White)',
         'Child mortality_95% CI - Low',
         'Child mortality_95% CI - High',
         'Child mortality_Child Mortality Rate (Black)',
         'Child mortality_Child Mortality Rate (Hispanic)',
         'Child mortality_Child Mortality Rate (White)',
         'Infant mortality_95% CI - Low',
         'Infant mortality_95% CI - High',
         'Infant mortality_Infant Mortality Rate (Black)',
         'Infant mortality_Infant Mortality Rate (Hispanic)',
         'Infant mortality_Infant Mortality Rate (White)',
         'Frequent physical distress_% Frequent Physical Distress',
         'Frequent physical distress_95% CI - Low',
         'Frequent physical distress_95% CI - High',
         'Frequent mental distress_% Frequent Mental Distress',
         'Frequent mental distress_95% CI - Low',
         'Frequent mental distress_95% CI - High',
         'Diabetes prevalence_95% CI - Low',
         'Diabetes prevalence_95% CI - High',
         'Motor vehicle crash deaths_95% CI - Low',
         'Motor vehicle crash deaths_95% CI - High',
         'Insufficient sleep_% Insufficient Sleep',
         'Insufficient sleep_95% CI - Low',
         'Insufficient sleep_95% CI - High',
         'Uninsured adults_95% CI - Low',
         'Uninsured adults_95% CI - High',
         'Uninsured children_95% CI - Low',
         'Uninsured children_95% CI - High',
         'Disconnected youth_% Disconnected Youth',
         'Median household income_95% CI - Low',
         'Median household income_95% CI - High',
         'Median household income_Household income (Black)',
         'Median household income_Household income (Hispanic)',
         'Median household income_Household income (White)',
         'Residential segregation - black/white_Segregation index',
         'Residential segregation - non-white/white_Segregation Index',
         'Homicides_95% CI - Low',
         'Homicides_95% CI - High',
         'Firearm fatalities_# Firearm Fatalities',
         'Firearm fatalities_Firearm Fatalities Rate',
         'Firearm fatalities_95% CI - Low',
         'Firearm fatalities_95% CI - High',
         'Homeownership_# Homeowners',
         'Homeownership_% Homeowners',
         'Homeownership_95% CI - Low',
         'Homeownership_95% CI - High',
         'Severe housing cost burden_# Households with Severe Cost Burden',
         'Severe housing cost burden_% Severe Housing Cost Burden',
         'Severe housing cost burden_95% CI - Low',
         'Severe housing cost burden_95% CI - High',
         'Demographics_95% CI - Low',
         'Demographics_95% CI - High',
         'Child mortality_# Deaths',
         'Infant mortality_# Deaths',
         'HIV prevalence_# HIV Cases',
         'Food insecurity_# Food Insecure',
         'Limited access to healthy foods_# Limited Access',
         'Limited access to healthy foods_% Limited Access',
         'Drug overdose deaths_# Drug Overdose Deaths',
         'Motor vehicle crash deaths_# Motor Vehicle Deaths',
         'Uninsured adults_# Uninsured',
         'Uninsured adults_% Uninsured',
         'Uninsured children_# Uninsured',
         'Other primary care providers_Other PCP Ratio']
                          ,axis=1)
        
        df3.columns = df3.columns.map('_'.join)
        df3.rename(columns = {
         'Unnamed: 0_level_0_FIPS':'FIPS',
         'Unnamed: 1_level_0_State':'State',
         'Unnamed: 2_level_0_County':'County/Borough/Parish',
         'Unnamed: 2_level_0_Borough':'County/Borough/Parish',
         'Unnamed: 2_level_0_Parish':'County/Borough/Parish'}, 
                   inplace = True)
        df3 = df3.drop(['Health Outcomes_Z-Score','Health Factors_Z-Score','Health Factors_Rank'], axis=1)
        
        # Join two dataframes
        joined_df = pd.merge(df, df2, on='FIPS', how='outer')
        joined_df = joined_df.drop(['State_y','County/Borough/Parish_y'],axis=1)
        joined_df.rename(columns = {'State_x':'State','County/Borough/Parish_x':'County/Borough/Parish'},inplace = True)
        
        joined_df = pd.merge(joined_df, df3, on='FIPS', how='outer')
        joined_df = joined_df.drop(['State_y','County/Borough/Parish_y'],axis=1)
        joined_df.rename(columns = {'State_x':'State','County/Borough/Parish_x':'County/Borough/Parish'},inplace = True)
        
        joined_df['Year'] = '2019'
        
        # append to list
        appended_data2019.append(joined_df)
        
#convert to dataframe
appended_data2019 = pd.concat(appended_data2019)

2019 County Health Rankings Alabama Data - v1_0.xls
2019 County Health Rankings Alaska Data - v1_0.xls
2019 County Health Rankings Arizona Data - v1_0.xls
2019 County Health Rankings Arkansas Data - v1_0.xls
2019 County Health Rankings California Data - v1_0.xls
2019 County Health Rankings Colorado Data - v1_0.xls
2019 County Health Rankings Connecticut Data - v1_0.xls
2019 County Health Rankings Delaware Data - v1_0.xls
2019 County Health Rankings District of Columbia Data - v1_0.xls
2019 County Health Rankings Florida Data - v1_0.xls
2019 County Health Rankings Georgia Data - v1_0.xls
2019 County Health Rankings Hawaii Data - v1_0.xls
2019 County Health Rankings Idaho Data - v1_0.xls
2019 County Health Rankings Illinois Data - v1_0.xls
2019 County Health Rankings Indiana Data - v1_0.xls
2019 County Health Rankings Iowa Data - v1_0.xls
2019 County Health Rankings Kansas Data - v2.xls
2019 County Health Rankings Kentucky Data - v1_0.xls
2019 County Health Rankings Louisiana Data - v1_0

In [11]:
appended_data2019.head(20)

,FIPS,State,County/Borough/Parish,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,%_Low_Birthweight_Births,%_Adult_Smokers,%_Adult_Obesity,"Healthy_Food_Access(0=Worst,10=Best)",%_Adults_Physical_Inactivity,...,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Health Outcomes_Rank,Year
0,1000,Alabama,NaN,9917.232898,21.402406,10.145493,21.538154,35.1,5.8,28.2,...,4.329620,3196852,65.579855,48998,1.074846,51.590595,1957932.0,40.963183,NaN,2019
1,1001,Alabama,Autauga,8824.057123,18.411124,8.475719,19.124658,37.5,7.2,31.1,...,2.857452,41336,74.473912,430,0.828548,51.342246,22921.0,42.002162,7,2019
2,1003,Alabama,Baldwin,7224.632160,18.060458,8.338683,16.795485,31.0,8.0,23.8,...,4.550200,176582,83.047388,872,0.454512,51.452772,77060.0,42.279099,2,2019
3,1005,Alabama,Barbour,9586.165037,25.773416,10.952624,21.540878,44.3,5.6,28.2,...,4.206569,11613,45.955679,297,1.199273,47.229917,18613.0,67.789635,33,2019
4,1007,Alabama,Bibb,11783.543675,19.996912,11.105003,19.916404,37.8,7.6,34.9,...,2.638080,16842,74.298571,84,0.394348,46.453150,15663.0,68.352607,40,2019
5,1009,Alabama,Blount,10908.101822,21.095323,7.926829,19.652158,34.4,8.5,28.5,...,9.565097,50439,86.944306,1013,1.870004,50.688639,51562.0,89.951502,22,2019
6,1011,Alabama,Bullock,12066.910263,29.407714,13.732004,23.478291,39.4,4.2,29.2,...,8.245223,2196,21.301775,109,1.104692,45.533029,5607.0,51.374382,58,2019
7,1013,Alabama,Butler,14018.606262,26.502613,12.411765,22.197656,40.2,6.5,34.7,...,1.437579,10229,51.596469,135,0.712702,53.432535,14921.0,71.232157,62,2019
8,1015,Alabama,Calhoun,12217.759984,19.584701,9.002330,20.462232,37.1,6.9,30.2,...,3.749739,82961,72.311031,960,0.880968,51.930653,39955.0,33.696826,38,2019
9,1017,Alabama,Chambers,11273.170980,25.219836,12.446043,22.040370,40.3,6.3,32.4,...,2.438229,18710,55.497879,49,0.153894,52.125293,16816.0,49.148034,49,2019


In [12]:
list(appended_data2019.columns.values)

['FIPS',
 'State',
 'County/Borough/Parish',
 'Yrs_Potential_Life_Lost_per100K(YPLL)',
 '%_Fair/Poor_Health',
 '%_Low_Birthweight_Births',
 '%_Adult_Smokers',
 '%_Adult_Obesity',
 'Healthy_Food_Access(0=Worst,10=Best)',
 '%_Adults_Physical_Inactivity',
 '%_Access_to_Excercise_Opportunities',
 '%_Adults_Excessive_Drinking',
 '%_Driving_Deaths_with_Alcohol_Involvement',
 'Chlamydia_Cases_per_100K',
 'Teen_Birth_Rate',
 '%_Ppl_Under65_w/o_Insurance',
 'PCP_per_100K',
 'Dentists_per_100K',
 'MHP_per_100K',
 'Preventable_Hospitalization_Rate(per100K_Medicare_Enrollees)',
 'Graduation_Rate',
 '%_Adults(25-44)_with_Some_Post-secondary_Ed',
 '%_Ppl(16+)_Unemployed_&_Looking_for_Work',
 '%_Children(under18)_Living_in_Poverty',
 '80th_Percentile_MHI',
 '20th_Percentile_MHI',
 '%_Children_Living_in_Single_Parent_Household',
 'Associations_per_10K',
 'Violent_Crimes_per_100K',
 'Injury_Death_Rate_per_100K',
 'Avg_Daily_Particulate_Matter_in_Micrograms_per_Cubic_Meter',
 'County_Affected_by_Water_V

In [13]:
appended_data2019.count()

FIPS                                                            3193
State                                                           3193
County/Borough/Parish                                           3142
Yrs_Potential_Life_Lost_per100K(YPLL)                           2959
%_Fair/Poor_Health                                              3193
%_Low_Birthweight_Births                                        3086
%_Adult_Smokers                                                 3193
%_Adult_Obesity                                                 3193
Healthy_Food_Access(0=Worst,10=Best)                            3174
%_Adults_Physical_Inactivity                                    3193
%_Access_to_Excercise_Opportunities                             3187
%_Adults_Excessive_Drinking                                     3193
%_Driving_Deaths_with_Alcohol_Involvement                       3160
Chlamydia_Cases_per_100K                                        3035
Teen_Birth_Rate                   

In [14]:
appended_data2019.isnull().any()

FIPS                                                            False
State                                                           False
County/Borough/Parish                                            True
Yrs_Potential_Life_Lost_per100K(YPLL)                            True
%_Fair/Poor_Health                                              False
%_Low_Birthweight_Births                                         True
%_Adult_Smokers                                                 False
%_Adult_Obesity                                                 False
Healthy_Food_Access(0=Worst,10=Best)                             True
%_Adults_Physical_Inactivity                                    False
%_Access_to_Excercise_Opportunities                              True
%_Adults_Excessive_Drinking                                     False
%_Driving_Deaths_with_Alcohol_Involvement                        True
Chlamydia_Cases_per_100K                                         True
Teen_Birth_Rate     

In [15]:
appended_data2019.loc[appended_data2019.isnull().any(axis=1)]

,FIPS,State,County/Borough/Parish,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,%_Low_Birthweight_Births,%_Adult_Smokers,%_Adult_Obesity,"Healthy_Food_Access(0=Worst,10=Best)",%_Adults_Physical_Inactivity,...,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Health Outcomes_Rank,Year
0,1000,Alabama,NaN,9917.232898,21.402406,10.145493,21.538154,35.1,5.8,28.2,...,4.329620,3196852,65.579855,48998,1.074846,51.590595,1957932.0,40.963183,NaN,2019
3,1005,Alabama,Barbour,9586.165037,25.773416,10.952624,21.540878,44.3,5.6,28.2,...,4.206569,11613,45.955679,297,1.199273,47.229917,18613.0,67.789635,33,2019
6,1011,Alabama,Bullock,12066.910263,29.407714,13.732004,23.478291,39.4,4.2,29.2,...,8.245223,2196,21.301775,109,1.104692,45.533029,5607.0,51.374382,58,2019
7,1013,Alabama,Butler,14018.606262,26.502613,12.411765,22.197656,40.2,6.5,34.7,...,1.437579,10229,51.596469,135,0.712702,53.432535,14921.0,71.232157,62,2019
9,1017,Alabama,Chambers,11273.170980,25.219836,12.446043,22.040370,40.3,6.3,32.4,...,2.438229,18710,55.497879,49,0.153894,52.125293,16816.0,49.148034,49,2019
10,1019,Alabama,Cherokee,11294.410540,19.291593,8.232446,19.144853,36.3,8.5,31.8,...,1.643655,23707,91.685037,24,0.097225,50.319063,22282.0,85.736273,23,2019
12,1023,Alabama,Choctaw,12047.187077,24.111285,11.004274,19.062060,41.3,6.4,32.6,...,0.973349,7309,56.461954,0,0.000000,52.715334,13859.0,100.000000,47,2019
13,1025,Alabama,Clarke,9688.373843,27.700529,12.954429,20.933323,39.6,5.4,32.3,...,1.324586,12703,52.746751,46,0.197773,52.784122,19628.0,75.980335,43,2019
14,1027,Alabama,Clay,9666.041236,20.587635,9.552846,19.139404,37.7,7.9,30.6,...,3.119623,10761,80.504227,57,0.447761,51.051096,13932.0,100.000000,15,2019
15,1029,Alabama,Cleburne,11985.555305,18.300984,8.881579,19.710512,34.5,8.0,28.7,...,2.422819,13818,92.738255,125,0.888920,50.416107,14972.0,100.000000,36,2019


In [16]:
appended_data2019 = appended_data2019.fillna(value=0)
appended_data2019.head(20)

,FIPS,State,County/Borough/Parish,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,%_Low_Birthweight_Births,%_Adult_Smokers,%_Adult_Obesity,"Healthy_Food_Access(0=Worst,10=Best)",%_Adults_Physical_Inactivity,...,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Health Outcomes_Rank,Year
0,1000,Alabama,0,9917.232898,21.402406,10.145493,21.538154,35.1,5.8,28.2,...,4.329620,3196852,65.579855,48998,1.074846,51.590595,1957932.0,40.963183,0,2019
1,1001,Alabama,Autauga,8824.057123,18.411124,8.475719,19.124658,37.5,7.2,31.1,...,2.857452,41336,74.473912,430,0.828548,51.342246,22921.0,42.002162,7,2019
2,1003,Alabama,Baldwin,7224.632160,18.060458,8.338683,16.795485,31.0,8.0,23.8,...,4.550200,176582,83.047388,872,0.454512,51.452772,77060.0,42.279099,2,2019
3,1005,Alabama,Barbour,9586.165037,25.773416,10.952624,21.540878,44.3,5.6,28.2,...,4.206569,11613,45.955679,297,1.199273,47.229917,18613.0,67.789635,33,2019
4,1007,Alabama,Bibb,11783.543675,19.996912,11.105003,19.916404,37.8,7.6,34.9,...,2.638080,16842,74.298571,84,0.394348,46.453150,15663.0,68.352607,40,2019
5,1009,Alabama,Blount,10908.101822,21.095323,7.926829,19.652158,34.4,8.5,28.5,...,9.565097,50439,86.944306,1013,1.870004,50.688639,51562.0,89.951502,22,2019
6,1011,Alabama,Bullock,12066.910263,29.407714,13.732004,23.478291,39.4,4.2,29.2,...,8.245223,2196,21.301775,109,1.104692,45.533029,5607.0,51.374382,58,2019
7,1013,Alabama,Butler,14018.606262,26.502613,12.411765,22.197656,40.2,6.5,34.7,...,1.437579,10229,51.596469,135,0.712702,53.432535,14921.0,71.232157,62,2019
8,1015,Alabama,Calhoun,12217.759984,19.584701,9.002330,20.462232,37.1,6.9,30.2,...,3.749739,82961,72.311031,960,0.880968,51.930653,39955.0,33.696826,38,2019
9,1017,Alabama,Chambers,11273.170980,25.219836,12.446043,22.040370,40.3,6.3,32.4,...,2.438229,18710,55.497879,49,0.153894,52.125293,16816.0,49.148034,49,2019


In [17]:
# appended_data2019.to_csv(os.path.join("Cleaned_Files","2019_County_Health_Ranking_Data.csv"),index=False)

### Loop to move through files within 2018 Data Folder
- join 3 worksheets within each state file together
- rename required columns
- drop unnecessary columns
- append all state data to single dataframe for 2018

In [18]:
appended_data2018 = []
path = os.path.join("Original_Files","CountyHealthRankingsData","2018")
for WorkingFile in os.listdir(path):
        #identifies the list of files to be grouped together from designated folder
        print(WorkingFile)
        
        if WorkingFile == '2018 County Health Rankings Ohio Data - v3.xls':
            # Import the excel file by joining the path directory and file name together and call Ranked Data and Additonal Measure tabs
            file = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Ranked Measure Data',header=(0,1))
            file2 = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Additional Measure Data',header=(0,1))
            file3 = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Outcomes & Factors Rankings',header=(0,1))
       
            # Load the excel files as two dataframes
            df = pd.DataFrame(file)
            df2 = pd.DataFrame(file2)
            df3 = pd.DataFrame(file3)
        
            # Code from Data_cleaning
       
            # Clean Ranked Data First by addressing multiindex column names
            df.columns = df.columns.map('_'.join)
        
            # Rename Columns
            df.rename(columns = {'Unnamed: 0_level_0_FIPS' : 'FIPS',
             'Unnamed: 1_level_0_State' :'State',
             'Unnamed: 2_level_0_County':'County/Borough/Parish',
             'Unnamed: 2_level_0_Borough':'County/Borough/Parish',
             'Unnamed: 2_level_0_Parish':'County/Borough/Parish',
             'Premature death_Years of Potential Life Lost Rate':'Yrs_Potential_Life_Lost_per100K(YPLL)',
             'Poor or fair health_% Fair/Poor':'%_Fair/Poor_Health',
             'Low birthweight_% LBW':'%_Low_Birthweight_Births',
             'Adult smoking_% Smokers':'%_Adult_Smokers',
             'Adult obesity_% Obese':'%_Adult_Obesity',
             'Food environment index_Food Environment Index':'Healthy_Food_Access(0=Worst,10=Best)',
             'Physical inactivity_% Physically Inactive':'%_Adults_Physical_Inactivity',
             'Access to exercise opportunities_% With Access':'%_Access_to_Excercise_Opportunities',
             'Excessive drinking_% Excessive Drinking':'%_Adults_Excessive_Drinking',
             'Alcohol-impaired driving deaths_% Alcohol-Impaired':'%_Driving_Deaths_with_Alcohol_Involvement',
             'Sexually transmitted infections_Chlamydia Rate':'Chlamydia_Cases_per_100K',
             'Teen births_Teen Birth Rate':'Teen_Birth_Rate',
             'Uninsured_% Uninsured':'%_Ppl_Under65_w/o_Insurance',                            
             'Primary care physicians_PCP Rate':'PCP_per_100K',
             'Dentists_Dentist Rate':'Dentists_per_100K',
             'Mental health providers_MHP Rate':'MHP_per_100K',
             'Preventable hospital stays_Preventable Hosp. Rate':'Preventable_Hospitalization_Rate(per100K_Medicare_Enrollees)',
             'High school graduation_Graduation Rate':'Graduation_Rate',
             'Some college_% Some College':'%_Adults(25-44)_with_Some_Post-secondary_Ed',
             'Unemployment_% Unemployed':'%_Ppl(16+)_Unemployed_&_Looking_for_Work',
             'Children in poverty_% Children in Poverty':'%_Children(under18)_Living_in_Poverty',
             'Income inequality_80th Percentile Income':'80th_Percentile_MHI',
             'Income inequality_20th Percentile Income':'20th_Percentile_MHI',
             'Children in single-parent households_% Single-Parent Households':'%_Children_Living_in_Single_Parent_Household',
             'Social associations_Association Rate':'Associations_per_10K',
             'Violent crime_Annual Average Violent Crimes':'Annual_Avg_Violent_Crimes',
             'Violent crime_Violent Crime Rate':'Violent_Crimes_per_100K',
             'Injury deaths_Injury Death Rate':'Injury_Death_Rate_per_100K',
             'Air pollution - particulate matter_Average Daily PM2.5':'Avg_Daily_Particulate_Matter_in_Micrograms_per_Cubic_Meter',
             'Drinking water violations_Presence of violation':'County_Affected_by_Water_Violation(1=Yes,0=No)',
             'Severe housing problems_% Severe Housing Problems':'%_Households_with_One_or_More Problems(overcrowding,high-housing-cost,lack-kitchen,lack-plumbing-facilities)',
             'Long commute - driving alone_% Long Commute - Drives Alone':'%_Lone_Commuters_who_Commute_More_Than_30Min'}, 
                      inplace = True)
        
            # Drop unnecessary columns
            df = df.drop([
             'Premature death_95% CI - Low',
             'Premature death_95% CI - High',
             'Premature death_Z-Score',
             'Premature death_Years of Potential Life Lost Rate (Black)',
             'Premature death_Years of Potential Life Lost Rate (Hispanic)',
             'Premature death_Years of Potential Life Lost Rate (White)',
             'Poor or fair health_95% CI - Low',
             'Poor or fair health_95% CI - High',
             'Poor or fair health_Z-Score',
             'Poor physical health days_Physically Unhealthy Days',
             'Poor physical health days_95% CI - Low',
             'Poor physical health days_95% CI - High',
             'Poor physical health days_Z-Score',
             'Poor mental health days_Mentally Unhealthy Days',
             'Poor mental health days_95% CI - Low',
             'Poor mental health days_95% CI - High',
             'Poor mental health days_Z-Score',
             'Low birthweight_Unreliable',
             'Low birthweight_95% CI - Low',
             'Low birthweight_95% CI - High',
             'Low birthweight_Z-Score',
             'Low birthweight_% LBW (Black)',
             'Low birthweight_% LBW (Hispanic)',
             'Low birthweight_% LBW (White)',
             'Adult smoking_95% CI - Low',
             'Adult smoking_95% CI - High',
             'Adult smoking_Z-Score',
             'Adult obesity_95% CI - Low',
             'Adult obesity_95% CI - High',
             'Adult obesity_Z-Score',
             'Food environment index_Z-Score',
             'Physical inactivity_95% CI - Low',
             'Physical inactivity_95% CI - High',
             'Physical inactivity_Z-Score',
             'Access to exercise opportunities_Z-Score',
             'Excessive drinking_95% CI - Low',
             'Excessive drinking_95% CI - High',
             'Excessive drinking_Z-Score',
             'Alcohol-impaired driving deaths_# Alcohol-Impaired Driving Deaths',
             'Alcohol-impaired driving deaths_# Driving Deaths',
             'Alcohol-impaired driving deaths_95% CI - Low',
             'Alcohol-impaired driving deaths_95% CI - High',
             'Alcohol-impaired driving deaths_Z-Score',
             'Sexually transmitted infections_# Chlamydia Cases',
             'Sexually transmitted infections_Z-Score',
             'Teen births_95% CI - Low',
             'Teen births_95% CI - High',
             'Teen births_Z-Score',
             'Teen births_Teen Birth Rate (Black)',
             'Teen births_Teen Birth Rate (Hispanic)',
             'Teen births_Teen Birth Rate (White)',
             'Uninsured_# Uninsured',
             'Uninsured_95% CI - Low',
             'Uninsured_95% CI - High',
             'Uninsured_Z-Score',
             'Primary care physicians_# Primary Care Physicians',
             'Primary care physicians_PCP Ratio',
             'Primary care physicians_Z-Score',
             'Dentists_# Dentists',
             'Dentists_Dentist Ratio',
             'Dentists_Z-Score',
             'Mental health providers_# Mental Health Providers',
             'Mental health providers_MHP Ratio',
             'Mental health providers_Z-Score',
             'Preventable hospital stays_# Medicare Enrollees',
             'Preventable hospital stays_95% CI - Low',
             'Preventable hospital stays_95% CI - High',
             'Preventable hospital stays_Z-Score',
             'Diabetes monitoring_# Diabetics',
             'Diabetes monitoring_% Receiving HbA1c',
             'Diabetes monitoring_95% CI - Low',
             'Diabetes monitoring_95% CI - High',
             'Diabetes monitoring_Z-Score',
             'Diabetes monitoring_% Receiving HbA1c (Black)',
             'Diabetes monitoring_% Receiving HbA1c (White)',
             'Mammography screening_# Medicare Enrollees',
             'Mammography screening_% Mammography',
             'Mammography screening_95% CI - Low',
             'Mammography screening_95% CI - High',
             'Mammography screening_Z-Score',
             'Mammography screening_% Mammography (Black)',
             'Mammography screening_% Mammography (White)',
             'High school graduation_Cohort Size',
             'High school graduation_Z-Score',
             'Some college_# Some College',
             'Some college_Population',
             'Some college_95% CI - Low',
             'Some college_95% CI - High',
             'Some college_Z-Score',
             'Unemployment_# Unemployed',
             'Unemployment_Labor Force',
             'Unemployment_Z-Score',
             'Children in poverty_95% CI - Low',
             'Children in poverty_95% CI - High',
             'Children in poverty_Z-Score',
             'Children in poverty_% Children in Poverty (Black)',
             'Children in poverty_% Children in Poverty (Hispanic)',
             'Children in poverty_% Children in Poverty (White)',
             'Income inequality_Income Ratio',
             'Income inequality_Z-Score',
             'Children in single-parent households_# Single-Parent Households',
             'Children in single-parent households_# Households',
             'Children in single-parent households_95% CI - Low',
             'Children in single-parent households_95% CI - High',
             'Children in single-parent households_Z-Score',
             'Social associations_# Associations',
             'Social associations_Z-Score',
             'Violent crime_# Violent Crimes',
             'Violent crime_Z-Score',
             'Injury deaths_# Injury Deaths',
             'Injury deaths_95% CI - Low',
             'Injury deaths_95% CI - High',
             'Injury deaths_Z-Score',
             'Air pollution - particulate matter_Z-Score',
             'Drinking water violations_Z-Score',
             'Severe housing problems_# Households with Severe Problems',
             'Severe housing problems_95% CI - Low',
             'Severe housing problems_95% CI - High',
             'Severe housing problems_Z-Score',
             'Driving alone to work_% Drive Alone',
             'Driving alone to work_95% CI - Low',
             'Driving alone to work_95% CI - High',
             'Driving alone to work_Z-Score',
             'Driving alone to work_% Drive Alone (Black)',
             'Driving alone to work_% Drive Alone (Hispanic)',
             'Driving alone to work_% Drive Alone (White)',
             'Long commute - driving alone_# Workers who Drive Alone',
             'Long commute - driving alone_95% CI - Low',
             'Long commute - driving alone_95% CI - High',
             'Long commute - driving alone_Z-Score',
             'High school graduation_Error Flag']
                                 ,axis=1)
        
            # Clean Additional Data Second by addressing multiindex column names
            df2.columns = df2.columns.map('_'.join)
        
            # Rename Columns
            df2.rename(columns = {'Unnamed: 0_level_0_FIPS':'FIPS',
             'Unnamed: 1_level_0_State':'State',
             'Unnamed: 2_level_0_County':'County/Borough/Parish',
             'Unnamed: 2_level_0_Borough':'County/Borough/Parish',
             'Unnamed: 2_level_0_Parish':'County/Borough/Parish',
             'Premature age-adjusted mortality_# Deaths':'Premature_Deaths',
             'Premature age-adjusted mortality_Age-Adjusted Mortality':'Premature(under75)_Age_Adjusted_Mortality_per_100K',
             'Child mortality_Child Mortality Rate':'Child(under18)_Mortality_per_100K',
             'Infant mortality_Infant Mortality Rate':'Infant(within1yr)_Mortality_per_1K_live_births',
             'Diabetes prevalence_% Diabetic':'%_Adults(20+)_Diagnosed_Diabetes',
             'HIV prevalence_HIV Prevalence Rate':'#_Ppl(Age13+)_diagnosed_HIV_per_100K',
             'Food insecurity_% Food Insecure':'%_Pop_w/o_Adequate_Access_to_Food',
             'Drug overdose deaths_Drug Overdose Mortality Rate':'Drug_Overdose_Deaths_per_100K',
             'Motor vehicle crash deaths_MV Mortality Rate':'Motor_Vehicle_Deaths_per_100K',
             'Uninsured children_% Uninsured':'%_Children(under19)_w/o_Health_Insurance',
             'Other primary care providers_Other PCP Rate':'Other_Primary_Care_Providers(non_physician)_per_100K',
             'Median household income_Household Income':'Median_Household_Income',
             'Children eligible for free or reduced price lunch_% Free or Reduced Lunch':'%_Children_Enrolled_in_Free/Reduced_Lunch',
             'Homicides_Homicide Rate':'#_Deaths_due_to_Homicide_per_100K',
             'Demographics_Population':'Resident_Population',
             'Demographics_% < 18':'%_under_18_years_of_age',
             'Demographics_% 65 and over':'%_Age_65_and_over',
             'Demographics_# African American':'#_Non-Hispanic_Black_or_African_American',
             'Demographics_% African American':'%_Non-Hispanic_Black_or_African_American',
             'Demographics_# American Indian/Alaskan Native':'#_American_Indian_or_Alaskan_Native',
             'Demographics_% American Indian/Alaskan Native':'%_American_Indian_or_Alaskan_Native',
             'Demographics_# Asian':'#_Asian',
             'Demographics_% Asian':'%_Asian',
             'Demographics_# Native Hawaiian/Other Pacific Islander':'#_Native_Hawaiian/Other_Pacific_Islander',
             'Demographics_% Native Hawaiian/Other Pacific Islander':'%_Native_Hawaiian/Other_Pacific_Islander',
             'Demographics_# Hispanic':'#_Hispanic',
             'Demographics_% Hispanic':'%_Hispanic',
             'Demographics_# Non-Hispanic White':'#_Non-Hispanic_White',
             'Demographics_% Non-Hispanic White':'%_Non-Hispanic_White',
             'Demographics_# Not Proficient in English':'#_Not_Proficient_in_English',
             'Demographics_% Not Proficient in English':'%_Not_Proficient_in_English',
             'Demographics_% Female':'%_Female',
             'Demographics_# Rural':'#_Pop_Living_in_Rural_Area',
             'Demographics_% Rural':'%_Rural'}, 
               inplace = True)
        
            # Drop unnecessary columns
            df2 = df2.drop(['Premature age-adjusted mortality_95% CI - Low',
             'Premature age-adjusted mortality_95% CI - High',
             'Premature age-adjusted mortality_Age-Adjusted Mortality (Black)',
             'Premature age-adjusted mortality_Age-Adjusted Mortality (Hispanic)',
             'Premature age-adjusted mortality_Age-Adjusted Mortality (White)',
             'Child mortality_# Deaths',
             'Child mortality_95% CI - Low',
             'Child mortality_95% CI - High',
             'Child mortality_Child Mortality Rate (Black)',
             'Child mortality_Child Mortality Rate (Hispanic)',
             'Child mortality_Child Mortality Rate (White)',
             'Infant mortality_# Deaths',
             'Infant mortality_95% CI - Low',
             'Infant mortality_95% CI - High',
             'Infant mortality_Infant Mortality Rate (Black)',
             'Infant mortality_Infant Mortality Rate (Hispanic)',
             'Infant mortality_Infant Mortality Rate (White)',
             'Frequent physical distress_% Frequent Physical Distress',
             'Frequent physical distress_95% CI - Low',
             'Frequent physical distress_95% CI - High',
             'Frequent mental distress_% Frequent Mental Distress',
             'Frequent mental distress_95% CI - Low',
             'Frequent mental distress_95% CI - High',
             'Diabetes prevalence_95% CI - Low',
             'Diabetes prevalence_95% CI - High',
             'HIV prevalence_# HIV Cases',
             'Food insecurity_# Food Insecure',
             'Limited access to healthy foods_# Limited Access',
             'Limited access to healthy foods_% Limited Access',
             'Drug overdose deaths_# Drug Overdose Deaths',
             'Drug overdose deaths - modeled_Range Drug Overdose Mortality Rate',
             'Motor vehicle crash deaths_# Motor Vehicle Deaths',
             'Motor vehicle crash deaths_95% CI - Low',
             'Motor vehicle crash deaths_95% CI - High',
             'Insufficient sleep_% Insufficient Sleep',
             'Insufficient sleep_95% CI - Low',
             'Insufficient sleep_95% CI - High',
             'Uninsured adults_# Uninsured',
             'Uninsured adults_% Uninsured',
             'Uninsured adults_95% CI - Low',
             'Uninsured adults_95% CI - High',
             'Uninsured children_# Uninsured',
             'Uninsured children_95% CI - Low',
             'Uninsured children_95% CI - High',
             'Health care costs_Costs',
             'Other primary care providers_Other PCP Ratio',
             'Disconnected youth_% Disconnected Youth',
             'Median household income_95% CI - Low',
             'Median household income_95% CI - High',
             'Median household income_Household income (Black)',
             'Median household income_Household income (Hispanic)',
             'Median household income_Household income (White)',
             'Residential segregation - black/white_Segregation index',
             'Residential segregation - non-white/white_Segregation Index',
             'Homicides_95% CI - Low',
             'Homicides_95% CI - High',
             'Firearm fatalities_# Firearm Fatalities',
             'Firearm fatalities_Firearm Fatalities Rate',
             'Firearm fatalities_95% CI - Low',
             'Firearm fatalities_95% CI - High',
             'Demographics_95% CI - Low',
             'Demographics_95% CI - High']
                         ,axis=1)

            df3.columns = df3.columns.map('_'.join)
            df3.rename(columns = {
             'Unnamed: 0_level_0_FIPS':'FIPS',
             'Unnamed: 1_level_0_State':'State',
             'Unnamed: 2_level_0_County':'County/Borough/Parish',
             'Unnamed: 2_level_0_Borough':'County/Borough/Parish',
             'Unnamed: 2_level_0_Parish':'County/Borough/Parish'}, 
                   inplace = True)
            df3 = df3.drop(['Health Outcomes_Z-Score','Health Factors_Z-Score','Health Factors_Rank'], axis=1)
        
            # Join two dataframes
            joined_df = pd.merge(df, df2, on='FIPS', how='outer')
            joined_df = joined_df.drop(['State_y','County/Borough/Parish_y'],axis=1)
            joined_df.rename(columns = {'State_x':'State','County/Borough/Parish_x':'County/Borough/Parish'},inplace = True)
        
            joined_df = pd.merge(joined_df, df3, on='FIPS', how='outer')
            joined_df = joined_df.drop(['State_y','County/Borough/Parish_y'],axis=1)
            joined_df.rename(columns = {'State_x':'State','County/Borough/Parish_x':'County/Borough/Parish'},inplace = True)
        
            joined_df['Year'] = '2018'
            
            # append to list
            appended_data2018.append(joined_df)
            
        else:
            # Import the excel file by joining the path directory and file name together and call Ranked Data and Additonal Measure tabs
            file = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Ranked Measure Data',header=(0,1))
            file2 = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Additional Measure Data',header=(0,1))
            file3 = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Outcomes & Factors Rankings',header=(0,1))
       
            # Load the excel files as two dataframes
            df = pd.DataFrame(file)
            df2 = pd.DataFrame(file2)
            df3 = pd.DataFrame(file3)
        
            # Code from Data_cleaning
       
            # Clean Ranked Data First by addressing multiindex column names
            df.columns = df.columns.map('_'.join)
        
            # Rename Columns
            df.rename(columns = {'Unnamed: 0_level_0_FIPS' : 'FIPS',
             'Unnamed: 1_level_0_State' :'State',
             'Unnamed: 2_level_0_County':'County/Borough/Parish',
             'Unnamed: 2_level_0_Borough':'County/Borough/Parish',
             'Unnamed: 2_level_0_Parish':'County/Borough/Parish',
             'Premature death_Years of Potential Life Lost Rate':'Yrs_Potential_Life_Lost_per100K(YPLL)',
             'Poor or fair health_% Fair/Poor':'%_Fair/Poor_Health',
             'Low birthweight_% LBW':'%_Low_Birthweight_Births',
             'Adult smoking_% Smokers':'%_Adult_Smokers',
             'Adult obesity_% Obese':'%_Adult_Obesity',
             'Food environment index_Food Environment Index':'Healthy_Food_Access(0=Worst,10=Best)',
             'Physical inactivity_% Physically Inactive':'%_Adults_Physical_Inactivity',
             'Access to exercise opportunities_% With Access':'%_Access_to_Excercise_Opportunities',
             'Excessive drinking_% Excessive Drinking':'%_Adults_Excessive_Drinking',
             'Alcohol-impaired driving deaths_% Alcohol-Impaired':'%_Driving_Deaths_with_Alcohol_Involvement',
             'Sexually transmitted infections_Chlamydia Rate':'Chlamydia_Cases_per_100K',
             'Teen births_Teen Birth Rate':'Teen_Birth_Rate',
             'Uninsured_% Uninsured':'%_Ppl_Under65_w/o_Insurance',                            
             'Primary care physicians_PCP Rate':'PCP_per_100K',
             'Dentists_Dentist Rate':'Dentists_per_100K',
             'Mental health providers_MHP Rate':'MHP_per_100K',
             'Preventable hospital stays_Preventable Hosp. Rate':'Preventable_Hospitalization_Rate(per100K_Medicare_Enrollees)',
             'High school graduation_Graduation Rate':'Graduation_Rate',
             'Some college_% Some College':'%_Adults(25-44)_with_Some_Post-secondary_Ed',
             'Unemployment_% Unemployed':'%_Ppl(16+)_Unemployed_&_Looking_for_Work',
             'Children in poverty_% Children in Poverty':'%_Children(under18)_Living_in_Poverty',
             'Income inequality_80th Percentile Income':'80th_Percentile_MHI',
             'Income inequality_20th Percentile Income':'20th_Percentile_MHI',
             'Children in single-parent households_% Single-Parent Households':'%_Children_Living_in_Single_Parent_Household',
             'Social associations_Association Rate':'Associations_per_10K',
             'Violent crime_Annual Average Violent Crimes':'Annual_Avg_Violent_Crimes',
             'Violent crime_Violent Crime Rate':'Violent_Crimes_per_100K',
             'Injury deaths_Injury Death Rate':'Injury_Death_Rate_per_100K',
             'Air pollution - particulate matter_Average Daily PM2.5':'Avg_Daily_Particulate_Matter_in_Micrograms_per_Cubic_Meter',
             'Drinking water violations_Presence of violation':'County_Affected_by_Water_Violation(1=Yes,0=No)',
             'Severe housing problems_% Severe Housing Problems':'%_Households_with_One_or_More Problems(overcrowding,high-housing-cost,lack-kitchen,lack-plumbing-facilities)',
             'Long commute - driving alone_% Long Commute - Drives Alone':'%_Lone_Commuters_who_Commute_More_Than_30Min'}, 
                      inplace = True)
        
            # Drop unnecessary columns
            df = df.drop([
             'Premature death_95% CI - Low',
             'Premature death_95% CI - High',
             'Premature death_Z-Score',
             'Premature death_Years of Potential Life Lost Rate (Black)',
             'Premature death_Years of Potential Life Lost Rate (Hispanic)',
             'Premature death_Years of Potential Life Lost Rate (White)',
             'Poor or fair health_95% CI - Low',
             'Poor or fair health_95% CI - High',
             'Poor or fair health_Z-Score',
             'Poor physical health days_Physically Unhealthy Days',
             'Poor physical health days_95% CI - Low',
             'Poor physical health days_95% CI - High',
             'Poor physical health days_Z-Score',
             'Poor mental health days_Mentally Unhealthy Days',
             'Poor mental health days_95% CI - Low',
             'Poor mental health days_95% CI - High',
             'Poor mental health days_Z-Score',
             'Low birthweight_Unreliable',
             'Low birthweight_95% CI - Low',
             'Low birthweight_95% CI - High',
             'Low birthweight_Z-Score',
             'Low birthweight_% LBW (Black)',
             'Low birthweight_% LBW (Hispanic)',
             'Low birthweight_% LBW (White)',
             'Adult smoking_95% CI - Low',
             'Adult smoking_95% CI - High',
             'Adult smoking_Z-Score',
             'Adult obesity_95% CI - Low',
             'Adult obesity_95% CI - High',
             'Adult obesity_Z-Score',
             'Food environment index_Z-Score',
             'Physical inactivity_95% CI - Low',
             'Physical inactivity_95% CI - High',
             'Physical inactivity_Z-Score',
             'Access to exercise opportunities_Z-Score',
             'Excessive drinking_95% CI - Low',
             'Excessive drinking_95% CI - High',
             'Excessive drinking_Z-Score',
             'Alcohol-impaired driving deaths_# Alcohol-Impaired Driving Deaths',
             'Alcohol-impaired driving deaths_# Driving Deaths',
             'Alcohol-impaired driving deaths_95% CI - Low',
             'Alcohol-impaired driving deaths_95% CI - High',
             'Alcohol-impaired driving deaths_Z-Score',
             'Sexually transmitted infections_# Chlamydia Cases',
             'Sexually transmitted infections_Z-Score',
             'Teen births_95% CI - Low',
             'Teen births_95% CI - High',
             'Teen births_Z-Score',
             'Teen births_Teen Birth Rate (Black)',
             'Teen births_Teen Birth Rate (Hispanic)',
             'Teen births_Teen Birth Rate (White)',
             'Uninsured_# Uninsured',
             'Uninsured_95% CI - Low',
             'Uninsured_95% CI - High',
             'Uninsured_Z-Score',
             'Primary care physicians_# Primary Care Physicians',
             'Primary care physicians_PCP Ratio',
             'Primary care physicians_Z-Score',
             'Dentists_# Dentists',
             'Dentists_Dentist Ratio',
             'Dentists_Z-Score',
             'Mental health providers_# Mental Health Providers',
             'Mental health providers_MHP Ratio',
             'Mental health providers_Z-Score',
             'Preventable hospital stays_# Medicare Enrollees',
             'Preventable hospital stays_95% CI - Low',
             'Preventable hospital stays_95% CI - High',
             'Preventable hospital stays_Z-Score',
             'Diabetes monitoring_# Diabetics',
             'Diabetes monitoring_% Receiving HbA1c',
             'Diabetes monitoring_95% CI - Low',
             'Diabetes monitoring_95% CI - High',
             'Diabetes monitoring_Z-Score',
             'Diabetes monitoring_% Receiving HbA1c (Black)',
             'Diabetes monitoring_% Receiving HbA1c (White)',
             'Mammography screening_# Medicare Enrollees',
             'Mammography screening_% Mammography',
             'Mammography screening_95% CI - Low',
             'Mammography screening_95% CI - High',
             'Mammography screening_Z-Score',
             'Mammography screening_% Mammography (Black)',
             'Mammography screening_% Mammography (White)',
             'High school graduation_Cohort Size',
             'High school graduation_Z-Score',
             'Some college_# Some College',
             'Some college_Population',
             'Some college_95% CI - Low',
             'Some college_95% CI - High',
             'Some college_Z-Score',
             'Unemployment_# Unemployed',
             'Unemployment_Labor Force',
             'Unemployment_Z-Score',
             'Children in poverty_95% CI - Low',
             'Children in poverty_95% CI - High',
             'Children in poverty_Z-Score',
             'Children in poverty_% Children in Poverty (Black)',
             'Children in poverty_% Children in Poverty (Hispanic)',
             'Children in poverty_% Children in Poverty (White)',
             'Income inequality_Income Ratio',
             'Income inequality_Z-Score',
             'Children in single-parent households_# Single-Parent Households',
             'Children in single-parent households_# Households',
             'Children in single-parent households_95% CI - Low',
             'Children in single-parent households_95% CI - High',
             'Children in single-parent households_Z-Score',
             'Social associations_# Associations',
             'Social associations_Z-Score',
             'Violent crime_# Violent Crimes',
             'Violent crime_Z-Score',
             'Injury deaths_# Injury Deaths',
             'Injury deaths_95% CI - Low',
             'Injury deaths_95% CI - High',
             'Injury deaths_Z-Score',
             'Air pollution - particulate matter_Z-Score',
             'Drinking water violations_Z-Score',
             'Severe housing problems_# Households with Severe Problems',
             'Severe housing problems_95% CI - Low',
             'Severe housing problems_95% CI - High',
             'Severe housing problems_Z-Score',
             'Driving alone to work_% Drive Alone',
             'Driving alone to work_95% CI - Low',
             'Driving alone to work_95% CI - High',
             'Driving alone to work_Z-Score',
             'Driving alone to work_% Drive Alone (Black)',
             'Driving alone to work_% Drive Alone (Hispanic)',
             'Driving alone to work_% Drive Alone (White)',
             'Long commute - driving alone_# Workers who Drive Alone',
             'Long commute - driving alone_95% CI - Low',
             'Long commute - driving alone_95% CI - High',
             'Long commute - driving alone_Z-Score']
                                 ,axis=1)
        
            # Clean Additional Data Second by addressing multiindex column names
            df2.columns = df2.columns.map('_'.join)
        
            # Rename Columns
            df2.rename(columns = {'Unnamed: 0_level_0_FIPS':'FIPS',
             'Unnamed: 1_level_0_State':'State',
             'Unnamed: 2_level_0_County':'County/Borough/Parish',
             'Unnamed: 2_level_0_Borough':'County/Borough/Parish',
             'Unnamed: 2_level_0_Parish':'County/Borough/Parish',
             'Premature age-adjusted mortality_# Deaths':'Premature_Deaths',
             'Premature age-adjusted mortality_Age-Adjusted Mortality':'Premature(under75)_Age_Adjusted_Mortality_per_100K',
             'Child mortality_Child Mortality Rate':'Child(under18)_Mortality_per_100K',
             'Infant mortality_Infant Mortality Rate':'Infant(within1yr)_Mortality_per_1K_live_births',
             'Diabetes prevalence_% Diabetic':'%_Adults(20+)_Diagnosed_Diabetes',
             'HIV prevalence_HIV Prevalence Rate':'#_Ppl(Age13+)_diagnosed_HIV_per_100K',
             'Food insecurity_% Food Insecure':'%_Pop_w/o_Adequate_Access_to_Food',
             'Drug overdose deaths_Drug Overdose Mortality Rate':'Drug_Overdose_Deaths_per_100K',
             'Motor vehicle crash deaths_MV Mortality Rate':'Motor_Vehicle_Deaths_per_100K',
             'Uninsured children_% Uninsured':'%_Children(under19)_w/o_Health_Insurance',
             'Other primary care providers_Other PCP Rate':'Other_Primary_Care_Providers(non_physician)_per_100K',
             'Median household income_Household Income':'Median_Household_Income',
             'Children eligible for free or reduced price lunch_% Free or Reduced Lunch':'%_Children_Enrolled_in_Free/Reduced_Lunch',
             'Homicides_Homicide Rate':'#_Deaths_due_to_Homicide_per_100K',
             'Demographics_Population':'Resident_Population',
             'Demographics_% < 18':'%_under_18_years_of_age',
             'Demographics_% 65 and over':'%_Age_65_and_over',
             'Demographics_# African American':'#_Non-Hispanic_Black_or_African_American',
             'Demographics_% African American':'%_Non-Hispanic_Black_or_African_American',
             'Demographics_# American Indian/Alaskan Native':'#_American_Indian_or_Alaskan_Native',
             'Demographics_% American Indian/Alaskan Native':'%_American_Indian_or_Alaskan_Native',
             'Demographics_# Asian':'#_Asian',
             'Demographics_% Asian':'%_Asian',
             'Demographics_# Native Hawaiian/Other Pacific Islander':'#_Native_Hawaiian/Other_Pacific_Islander',
             'Demographics_% Native Hawaiian/Other Pacific Islander':'%_Native_Hawaiian/Other_Pacific_Islander',
             'Demographics_# Hispanic':'#_Hispanic',
             'Demographics_% Hispanic':'%_Hispanic',
             'Demographics_# Non-Hispanic White':'#_Non-Hispanic_White',
             'Demographics_% Non-Hispanic White':'%_Non-Hispanic_White',
             'Demographics_# Not Proficient in English':'#_Not_Proficient_in_English',
             'Demographics_% Not Proficient in English':'%_Not_Proficient_in_English',
             'Demographics_% Female':'%_Female',
             'Demographics_# Rural':'#_Pop_Living_in_Rural_Area',
             'Demographics_% Rural':'%_Rural'}, 
               inplace = True)
        
            # Drop unnecessary columns
            df2 = df2.drop(['Premature age-adjusted mortality_95% CI - Low',
             'Premature age-adjusted mortality_95% CI - High',
             'Premature age-adjusted mortality_Age-Adjusted Mortality (Black)',
             'Premature age-adjusted mortality_Age-Adjusted Mortality (Hispanic)',
             'Premature age-adjusted mortality_Age-Adjusted Mortality (White)',
             'Child mortality_# Deaths',
             'Child mortality_95% CI - Low',
             'Child mortality_95% CI - High',
             'Child mortality_Child Mortality Rate (Black)',
             'Child mortality_Child Mortality Rate (Hispanic)',
             'Child mortality_Child Mortality Rate (White)',
             'Infant mortality_# Deaths',
             'Infant mortality_95% CI - Low',
             'Infant mortality_95% CI - High',
             'Infant mortality_Infant Mortality Rate (Black)',
             'Infant mortality_Infant Mortality Rate (Hispanic)',
             'Infant mortality_Infant Mortality Rate (White)',
             'Frequent physical distress_% Frequent Physical Distress',
             'Frequent physical distress_95% CI - Low',
             'Frequent physical distress_95% CI - High',
             'Frequent mental distress_% Frequent Mental Distress',
             'Frequent mental distress_95% CI - Low',
             'Frequent mental distress_95% CI - High',
             'Diabetes prevalence_95% CI - Low',
             'Diabetes prevalence_95% CI - High',
             'HIV prevalence_# HIV Cases',
             'Food insecurity_# Food Insecure',
             'Limited access to healthy foods_# Limited Access',
             'Limited access to healthy foods_% Limited Access',
             'Drug overdose deaths_# Drug Overdose Deaths',
             'Drug overdose deaths - modeled_Range Drug Overdose Mortality Rate',
             'Motor vehicle crash deaths_# Motor Vehicle Deaths',
             'Motor vehicle crash deaths_95% CI - Low',
             'Motor vehicle crash deaths_95% CI - High',
             'Insufficient sleep_% Insufficient Sleep',
             'Insufficient sleep_95% CI - Low',
             'Insufficient sleep_95% CI - High',
             'Uninsured adults_# Uninsured',
             'Uninsured adults_% Uninsured',
             'Uninsured adults_95% CI - Low',
             'Uninsured adults_95% CI - High',
             'Uninsured children_# Uninsured',
             'Uninsured children_95% CI - Low',
             'Uninsured children_95% CI - High',
             'Health care costs_Costs',
             'Other primary care providers_Other PCP Ratio',
             'Disconnected youth_% Disconnected Youth',
             'Median household income_95% CI - Low',
             'Median household income_95% CI - High',
             'Median household income_Household income (Black)',
             'Median household income_Household income (Hispanic)',
             'Median household income_Household income (White)',
             'Residential segregation - black/white_Segregation index',
             'Residential segregation - non-white/white_Segregation Index',
             'Homicides_95% CI - Low',
             'Homicides_95% CI - High',
             'Firearm fatalities_# Firearm Fatalities',
             'Firearm fatalities_Firearm Fatalities Rate',
             'Firearm fatalities_95% CI - Low',
             'Firearm fatalities_95% CI - High',
             'Demographics_95% CI - Low',
             'Demographics_95% CI - High']
                         ,axis=1)

            df3.columns = df3.columns.map('_'.join)
            df3.rename(columns = {
             'Unnamed: 0_level_0_FIPS':'FIPS',
             'Unnamed: 1_level_0_State':'State',
             'Unnamed: 2_level_0_County':'County/Borough/Parish',
             'Unnamed: 2_level_0_Borough':'County/Borough/Parish',
             'Unnamed: 2_level_0_Parish':'County/Borough/Parish'}, 
                   inplace = True)
            df3 = df3.drop(['Health Outcomes_Z-Score','Health Factors_Z-Score','Health Factors_Rank'], axis=1)
        
            # Join two dataframes
            joined_df = pd.merge(df, df2, on='FIPS', how='outer')
            joined_df = joined_df.drop(['State_y','County/Borough/Parish_y'],axis=1)
            joined_df.rename(columns = {'State_x':'State','County/Borough/Parish_x':'County/Borough/Parish'},inplace = True)
        
            joined_df = pd.merge(joined_df, df3, on='FIPS', how='outer')
            joined_df = joined_df.drop(['State_y','County/Borough/Parish_y'],axis=1)
            joined_df.rename(columns = {'State_x':'State','County/Borough/Parish_x':'County/Borough/Parish'},inplace = True)
        
            joined_df['Year'] = '2018'
            # append to list
            appended_data2018.append(joined_df)            
        
#convert to dataframe
appended_data2018 = pd.concat(appended_data2018)

2018 County Health Rankings Alabama Data - v3.xls
2018 County Health Rankings Alaska Data - v3.xls
2018 County Health Rankings Arizona Data - v3.xls
2018 County Health Rankings Arkansas Data - v3.xls
2018 County Health Rankings California Data - v3.xls
2018 County Health Rankings Colorado Data - v3.xls
2018 County Health Rankings Connecticut Data - v3.xls
2018 County Health Rankings Delaware Data - v3.xls
2018 County Health Rankings District of Columbia Data - v3.xls
2018 County Health Rankings Florida Data - v3.xls
2018 County Health Rankings Georgia Data - v3.xls
2018 County Health Rankings Hawaii Data - v3.xls
2018 County Health Rankings Idaho Data - v3.xls
2018 County Health Rankings Illinois Data - v3.xls
2018 County Health Rankings Indiana Data - v3.xls
2018 County Health Rankings Iowa Data - v3.xls
2018 County Health Rankings Kansas Data - v3.xls
2018 County Health Rankings Kentucky Data - v3.xls
2018 County Health Rankings Louisiana Data - v3.xls
2018 County Health Rankings Mai

In [19]:
appended_data2018.head(20)

,FIPS,State,County/Borough/Parish,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,%_Low_Birthweight_Births,%_Adult_Smokers,%_Adult_Obesity,"Healthy_Food_Access(0=Worst,10=Best)",%_Adults_Physical_Inactivity,...,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Health Outcomes_Rank,Year
0,1000.0,Alabama,NaN,9642.412033,21.402406,10.148697,21.538154,34.7,5.6,29.0,...,4.191495,3201937.0,65.838772,51759.0,1.137962,51.564041,1957932.0,40.963183,NaN,2018
1,1001.0,Alabama,Autauga,9409.294766,18.411124,8.594631,19.124658,36.4,7.1,30.9,...,2.688754,41562.0,75.000000,432.0,0.832739,51.288437,22921.0,42.002162,11,2018
2,1003.0,Alabama,Baldwin,7467.596664,18.060458,8.480290,16.795485,29.3,7.9,24.5,...,4.416411,173529.0,83.202198,1540.0,0.818809,51.457833,77060.0,42.279099,3,2018
3,1005.0,Alabama,Barbour,8929.474982,25.773416,11.741294,21.540878,44.2,5.5,30.6,...,4.213364,12061.0,46.450992,382.0,1.519370,46.932409,18613.0,67.789635,34,2018
4,1007.0,Alabama,Bibb,11741.938889,19.996912,11.258278,19.916404,38.4,7.6,37.5,...,2.499669,16850.0,74.415934,96.0,0.448137,46.363114,15663.0,68.352607,41,2018
5,1009.0,Alabama,Blount,9359.122343,21.095323,7.791667,19.652158,35.8,8.5,29.0,...,9.068695,50472.0,87.467073,1018.0,1.878506,50.613476,51562.0,89.951502,14,2018
6,1011.0,Alabama,Bullock,12343.052871,29.407714,14.727085,23.478291,40.2,4.5,29.8,...,7.788072,2231.0,21.530593,95.0,0.957565,45.956379,5607.0,51.374382,59,2018
7,1013.0,Alabama,Butler,12779.304852,26.502613,11.615871,22.197656,36.0,6.5,33.9,...,1.310131,10442.0,52.215222,166.0,0.870386,53.385339,14921.0,71.232157,57,2018
8,1015.0,Alabama,Calhoun,11527.781204,19.584701,8.766716,20.462232,36.1,6.8,32.5,...,3.577318,83235.0,72.623919,776.0,0.709622,51.891180,39955.0,33.696826,29,2018
9,1017.0,Alabama,Chambers,10350.047581,25.219836,11.821429,22.040370,38.0,6.1,32.6,...,2.260438,18967.0,56.044086,114.0,0.356595,52.004846,16816.0,49.148034,48,2018


In [20]:
list(appended_data2018.columns.values)

['FIPS',
 'State',
 'County/Borough/Parish',
 'Yrs_Potential_Life_Lost_per100K(YPLL)',
 '%_Fair/Poor_Health',
 '%_Low_Birthweight_Births',
 '%_Adult_Smokers',
 '%_Adult_Obesity',
 'Healthy_Food_Access(0=Worst,10=Best)',
 '%_Adults_Physical_Inactivity',
 '%_Access_to_Excercise_Opportunities',
 '%_Adults_Excessive_Drinking',
 '%_Driving_Deaths_with_Alcohol_Involvement',
 'Chlamydia_Cases_per_100K',
 'Teen_Birth_Rate',
 '%_Ppl_Under65_w/o_Insurance',
 'PCP_per_100K',
 'Dentists_per_100K',
 'MHP_per_100K',
 'Preventable_Hospitalization_Rate(per100K_Medicare_Enrollees)',
 'Graduation_Rate',
 '%_Adults(25-44)_with_Some_Post-secondary_Ed',
 '%_Ppl(16+)_Unemployed_&_Looking_for_Work',
 '%_Children(under18)_Living_in_Poverty',
 '80th_Percentile_MHI',
 '20th_Percentile_MHI',
 '%_Children_Living_in_Single_Parent_Household',
 'Associations_per_10K',
 'Violent_Crimes_per_100K',
 'Injury_Death_Rate_per_100K',
 'Avg_Daily_Particulate_Matter_in_Micrograms_per_Cubic_Meter',
 'County_Affected_by_Water_V

In [21]:
appended_data2018.count()

FIPS                                                            3193
State                                                           3193
County/Borough/Parish                                           3142
Yrs_Potential_Life_Lost_per100K(YPLL)                           3012
%_Fair/Poor_Health                                              3193
%_Low_Birthweight_Births                                        3088
%_Adult_Smokers                                                 3193
%_Adult_Obesity                                                 3193
Healthy_Food_Access(0=Worst,10=Best)                            3174
%_Adults_Physical_Inactivity                                    3193
%_Access_to_Excercise_Opportunities                             3187
%_Adults_Excessive_Drinking                                     3193
%_Driving_Deaths_with_Alcohol_Involvement                       3168
Chlamydia_Cases_per_100K                                        3026
Teen_Birth_Rate                   

In [22]:
appended_data2018.isnull().any()

FIPS                                                             True
State                                                            True
County/Borough/Parish                                            True
Yrs_Potential_Life_Lost_per100K(YPLL)                            True
%_Fair/Poor_Health                                               True
%_Low_Birthweight_Births                                         True
%_Adult_Smokers                                                  True
%_Adult_Obesity                                                  True
Healthy_Food_Access(0=Worst,10=Best)                             True
%_Adults_Physical_Inactivity                                     True
%_Access_to_Excercise_Opportunities                              True
%_Adults_Excessive_Drinking                                      True
%_Driving_Deaths_with_Alcohol_Involvement                        True
Chlamydia_Cases_per_100K                                         True
Teen_Birth_Rate     

In [23]:
appended_data2018.loc[appended_data2018.isnull().any(axis=1)]

,FIPS,State,County/Borough/Parish,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,%_Low_Birthweight_Births,%_Adult_Smokers,%_Adult_Obesity,"Healthy_Food_Access(0=Worst,10=Best)",%_Adults_Physical_Inactivity,...,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Health Outcomes_Rank,Year
0,1000.0,Alabama,NaN,9642.412033,21.402406,10.148697,21.538154,34.7,5.6,29.0,...,4.191495,3201937.0,65.838772,51759.0,1.137962,51.564041,1957932.0,40.963183,NaN,2018
3,1005.0,Alabama,Barbour,8929.474982,25.773416,11.741294,21.540878,44.2,5.5,30.6,...,4.213364,12061.0,46.450992,382.0,1.519370,46.932409,18613.0,67.789635,34,2018
6,1011.0,Alabama,Bullock,12343.052871,29.407714,14.727085,23.478291,40.2,4.5,29.8,...,7.788072,2231.0,21.530593,95.0,0.957565,45.956379,5607.0,51.374382,59,2018
7,1013.0,Alabama,Butler,12779.304852,26.502613,11.615871,22.197656,36.0,6.5,33.9,...,1.310131,10442.0,52.215222,166.0,0.870386,53.385339,14921.0,71.232157,57,2018
9,1017.0,Alabama,Chambers,10350.047581,25.219836,11.821429,22.040370,38.0,6.1,32.6,...,2.260438,18967.0,56.044086,114.0,0.356595,52.004846,16816.0,49.148034,48,2018
10,1019.0,Alabama,Cherokee,11683.171114,19.291593,8.698297,19.144853,35.5,8.2,35.9,...,1.605442,23588.0,91.692906,15.0,0.060901,50.363460,22282.0,85.736273,35,2018
12,1023.0,Alabama,Choctaw,11558.654472,24.111285,11.134235,19.062060,39.5,6.2,33.1,...,0.962056,7357.0,56.622797,0.0,0.000000,52.435927,13859.0,100.000000,50,2018
13,1025.0,Alabama,Clarke,9133.840426,27.700529,13.201663,20.933323,38.4,5.2,29.4,...,1.426697,12879.0,52.800098,56.0,0.238420,52.693506,19628.0,75.980335,45,2018
14,1027.0,Alabama,Clay,10235.922515,20.587635,9.979633,19.139404,38.1,7.8,31.6,...,3.046250,10851.0,80.425437,68.0,0.531084,50.963534,13932.0,100.000000,22,2018
15,1029.0,Alabama,Cleburne,11380.483302,18.300984,8.699254,19.710512,34.6,8.1,32.2,...,2.452426,13842.0,92.749933,115.0,0.816761,50.274725,14972.0,100.000000,27,2018


In [24]:
appended_data2018 = appended_data2018.fillna(value=0)
appended_data2018.head(20)

,FIPS,State,County/Borough/Parish,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,%_Low_Birthweight_Births,%_Adult_Smokers,%_Adult_Obesity,"Healthy_Food_Access(0=Worst,10=Best)",%_Adults_Physical_Inactivity,...,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Health Outcomes_Rank,Year
0,1000.0,Alabama,0,9642.412033,21.402406,10.148697,21.538154,34.7,5.6,29.0,...,4.191495,3201937.0,65.838772,51759.0,1.137962,51.564041,1957932.0,40.963183,0,2018
1,1001.0,Alabama,Autauga,9409.294766,18.411124,8.594631,19.124658,36.4,7.1,30.9,...,2.688754,41562.0,75.000000,432.0,0.832739,51.288437,22921.0,42.002162,11,2018
2,1003.0,Alabama,Baldwin,7467.596664,18.060458,8.480290,16.795485,29.3,7.9,24.5,...,4.416411,173529.0,83.202198,1540.0,0.818809,51.457833,77060.0,42.279099,3,2018
3,1005.0,Alabama,Barbour,8929.474982,25.773416,11.741294,21.540878,44.2,5.5,30.6,...,4.213364,12061.0,46.450992,382.0,1.519370,46.932409,18613.0,67.789635,34,2018
4,1007.0,Alabama,Bibb,11741.938889,19.996912,11.258278,19.916404,38.4,7.6,37.5,...,2.499669,16850.0,74.415934,96.0,0.448137,46.363114,15663.0,68.352607,41,2018
5,1009.0,Alabama,Blount,9359.122343,21.095323,7.791667,19.652158,35.8,8.5,29.0,...,9.068695,50472.0,87.467073,1018.0,1.878506,50.613476,51562.0,89.951502,14,2018
6,1011.0,Alabama,Bullock,12343.052871,29.407714,14.727085,23.478291,40.2,4.5,29.8,...,7.788072,2231.0,21.530593,95.0,0.957565,45.956379,5607.0,51.374382,59,2018
7,1013.0,Alabama,Butler,12779.304852,26.502613,11.615871,22.197656,36.0,6.5,33.9,...,1.310131,10442.0,52.215222,166.0,0.870386,53.385339,14921.0,71.232157,57,2018
8,1015.0,Alabama,Calhoun,11527.781204,19.584701,8.766716,20.462232,36.1,6.8,32.5,...,3.577318,83235.0,72.623919,776.0,0.709622,51.891180,39955.0,33.696826,29,2018
9,1017.0,Alabama,Chambers,10350.047581,25.219836,11.821429,22.040370,38.0,6.1,32.6,...,2.260438,18967.0,56.044086,114.0,0.356595,52.004846,16816.0,49.148034,48,2018


In [25]:
# appended_data2018.to_csv(os.path.join("Cleaned_Files","2018_County_Health_Ranking_Data.csv"),index=False)

### Loop to move through files within 2017 Data Folder
- join 3 worksheets within each state file together
- rename required columns
- drop unnecessary columns
- append all state data to single dataframe for 2017

In [26]:
appended_data2017 = []
path = os.path.join("Original_Files","CountyHealthRankingsData","2017")
for WorkingFile in os.listdir(path):
        #identifies the list of files to be grouped together from designated folder
        print(WorkingFile)
        
        # Import the excel file by joining the path directory and file name together and call Ranked Data and Additonal Measure tabs
        file = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Ranked Measure Data',header=(0,1))
        file2 = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Additional Measure Data',header=(0,1))
        file3 = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Outcomes & Factors Rankings',header=(0,1))
       
        # Load the excel files as two dataframes
        df = pd.DataFrame(file)
        df2 = pd.DataFrame(file2)
        df3 = pd.DataFrame(file3)
        
        # Code from Data_cleaning
       
        # Clean Ranked Data First by addressing multiindex column names
        df.columns = df.columns.map('_'.join)
        
        # Rename Columns
        df.rename(columns = {'Unnamed: 0_level_0_FIPS' : 'FIPS',
         'Unnamed: 1_level_0_State' :'State',
         'Unnamed: 2_level_0_County':'County/Borough/Parish',
         'Unnamed: 2_level_0_Borough':'County/Borough/Parish',
         'Unnamed: 2_level_0_Parish':'County/Borough/Parish',
         'Premature death_Years of Potential Life Lost Rate':'Yrs_Potential_Life_Lost_per100K(YPLL)',
         'Poor or fair health_% Fair/Poor':'%_Fair/Poor_Health',
         'Low birthweight_% LBW':'%_Low_Birthweight_Births',
         'Adult smoking_% Smokers':'%_Adult_Smokers',
         'Adult obesity_% Obese':'%_Adult_Obesity',
         'Food environment index_Food Environment Index':'Healthy_Food_Access(0=Worst,10=Best)',
         'Physical inactivity_% Physically Inactive':'%_Adults_Physical_Inactivity',
         'Access to exercise opportunities_% With Access':'%_Access_to_Excercise_Opportunities',
         'Excessive drinking_% Excessive Drinking':'%_Adults_Excessive_Drinking',
         'Alcohol-impaired driving deaths_% Alcohol-Impaired':'%_Driving_Deaths_with_Alcohol_Involvement',
         'Sexually transmitted infections_Chlamydia Rate':'Chlamydia_Cases_per_100K',
         'Teen births_Teen Birth Rate':'Teen_Birth_Rate',
         'Uninsured_% Uninsured':'%_Ppl_Under65_w/o_Insurance',                            
         'Primary care physicians_PCP Rate':'PCP_per_100K',
         'Dentists_Dentist Rate':'Dentists_per_100K',
         'Mental health providers_MHP Rate':'MHP_per_100K',
         'Preventable hospital stays_Preventable Hosp. Rate':'Preventable_Hospitalization_Rate(per100K_Medicare_Enrollees)',
         'High school graduation_Graduation Rate':'Graduation_Rate',
         'Some college_% Some College':'%_Adults(25-44)_with_Some_Post-secondary_Ed',
         'Unemployment_% Unemployed':'%_Ppl(16+)_Unemployed_&_Looking_for_Work',
         'Children in poverty_% Children in Poverty':'%_Children(under18)_Living_in_Poverty',
         'Income inequality_80th Percentile Income':'80th_Percentile_MHI',
         'Income inequality_20th Percentile Income':'20th_Percentile_MHI',
         'Children in single-parent households_% Single-Parent Households':'%_Children_Living_in_Single_Parent_Household',
         'Social associations_Association Rate':'Associations_per_10K',
         'Violent crime_Annual Average Violent Crimes':'Annual_Avg_Violent_Crimes',
         'Violent crime_Violent Crime Rate':'Violent_Crimes_per_100K',
         'Injury deaths_Injury Death Rate':'Injury_Death_Rate_per_100K',
         'Air pollution - particulate matter_Average Daily PM2.5':'Avg_Daily_Particulate_Matter_in_Micrograms_per_Cubic_Meter',
         'Drinking water violations_Presence of violation':'County_Affected_by_Water_Violation(1=Yes,0=No)',
         'Severe housing problems_% Severe Housing Problems':'%_Households_with_One_or_More Problems(overcrowding,high-housing-cost,lack-kitchen,lack-plumbing-facilities)',
         'Long commute - driving alone_% Long Commute - Drives Alone':'%_Lone_Commuters_who_Commute_More_Than_30Min'}, 
                  inplace = True)
        
        # Drop unnecessary columns
        df = df.drop(['Premature death_# Deaths',
         'Premature death_95% CI - Low',
         'Premature death_95% CI - High',
         'Premature death_Z-Score',
         'Poor or fair health_95% CI - Low',
         'Poor or fair health_95% CI - High',
         'Poor or fair health_Z-Score',
         'Poor physical health days_Physically Unhealthy Days',
         'Poor physical health days_95% CI - Low',
         'Poor physical health days_95% CI - High',
         'Poor physical health days_Z-Score',
         'Poor mental health days_Mentally Unhealthy Days',
         'Poor mental health days_95% CI - Low',
         'Poor mental health days_95% CI - High',
         'Poor mental health days_Z-Score',
         'Low birthweight_Unreliable',
         'Low birthweight_# Low Birthweight Births',
         'Low birthweight_# Live Births',
         'Low birthweight_95% CI - Low',
         'Low birthweight_95% CI - High',
         'Low birthweight_Z-Score',
         'Adult smoking_95% CI - Low',
         'Adult smoking_95% CI - High',
         'Adult smoking_Z-Score',
         'Adult obesity_95% CI - Low',
         'Adult obesity_95% CI - High',
         'Adult obesity_Z-Score',
         'Food environment index_Z-Score',
         'Physical inactivity_95% CI - Low',
         'Physical inactivity_95% CI - High',
         'Physical inactivity_Z-Score',
         'Access to exercise opportunities_Z-Score',
         'Excessive drinking_95% CI - Low',
         'Excessive drinking_95% CI - High',
         'Excessive drinking_Z-Score',
         'Alcohol-impaired driving deaths_# Alcohol-Impaired Driving Deaths',
         'Alcohol-impaired driving deaths_# Driving Deaths',
         'Alcohol-impaired driving deaths_95% CI - Low',
         'Alcohol-impaired driving deaths_95% CI - High',
         'Alcohol-impaired driving deaths_Z-Score',
         'Sexually transmitted infections_# Chlamydia Cases',
         'Sexually transmitted infections_Z-Score',
         'Teen births_Teen Births',
         'Teen births_Teen Population',
         'Teen births_95% CI - Low',
         'Teen births_95% CI - High',
         'Teen births_Z-Score',
         'Uninsured_# Uninsured',
         'Uninsured_95% CI - Low',
         'Uninsured_95% CI - High',
         'Uninsured_Z-Score',
         'Primary care physicians_# Primary Care Physicians',
         'Primary care physicians_PCP Ratio',
         'Primary care physicians_Z-Score',
         'Dentists_# Dentists',
         'Dentists_Dentist Ratio',
         'Dentists_Z-Score',
         'Mental health providers_# Mental Health Providers',
         'Mental health providers_MHP Ratio',
         'Mental health providers_Z-Score',
         'Preventable hospital stays_# Medicare Enrollees',
         'Preventable hospital stays_95% CI - Low',
         'Preventable hospital stays_95% CI - High',
         'Preventable hospital stays_Z-Score',
         'Preventable hospital stays_Preventable Hosp. Rate (white)',
         'Preventable hospital stays_Preventable Hosp. Rate (black)',
         'Diabetes monitoring_# Diabetics',
         'Diabetes monitoring_% Receiving HbA1c',
         'Diabetes monitoring_95% CI - Low',
         'Diabetes monitoring_95% CI - High',
         'Diabetes monitoring_Z-Score',
         'Diabetes monitoring_% Receiving HbA1c (white)',
         'Diabetes monitoring_% Receiving HbA1c (black)',
         'Mammography screening_# Medicare Enrollees',
         'Mammography screening_% Mammography',
         'Mammography screening_95% CI - Low',
         'Mammography screening_95% CI - High',
         'Mammography screening_Z-Score',
         'Mammography screening_% Mammography (white)',
         'Mammography screening_% Mammography (black)',
         'High school graduation_Cohort Size',
         'High school graduation_Z-Score',
         'Some college_# Some College',
         'Some college_Population',
         'Some college_95% CI - Low',
         'Some college_95% CI - High',
         'Some college_Z-Score',
         'Unemployment_# Unemployed',
         'Unemployment_Labor Force',
         'Unemployment_Z-Score',
         'Children in poverty_95% CI - Low',
         'Children in poverty_95% CI - High',
         'Children in poverty_Z-Score',
         'Children in poverty_% Children in Poverty - Black',
         'Children in poverty_% Children in Poverty - Hispanic',
         'Children in poverty_% Children in Poverty - White',
         'Income inequality_Income Ratio',
         'Income inequality_Z-Score',
         'Children in single-parent households_# Single-Parent Households',
         'Children in single-parent households_# Households',
         'Children in single-parent households_95% CI - Low',
         'Children in single-parent households_95% CI - High',
         'Children in single-parent households_Z-Score',
         'Social associations_# Associations',
         'Social associations_Z-Score',
         'Violent crime_# Violent Crimes',
         'Violent crime_Z-Score',
         'Injury deaths_# Injury Deaths',
         'Injury deaths_95% CI - Low',
         'Injury deaths_95% CI - High',
         'Injury deaths_Z-Score',
         'Air pollution - particulate matter_Z-Score',
         'Drinking water violations_Z-Score',
         'Severe housing problems_# Households with Severe Problems',
         'Severe housing problems_95% CI - Low',
         'Severe housing problems_95% CI - High',
         'Severe housing problems_Z-Score',
         'Driving alone to work_% Drive Alone',
         'Driving alone to work_95% CI - Low',
         'Driving alone to work_95% CI - High',
         'Driving alone to work_Z-Score',
         'Driving alone to work_% Drive Alone - Black',
         'Driving alone to work_% Drive Alone - Hispanic',
         'Driving alone to work_% Drive Alone - White',
         'Long commute - driving alone_# Workers who Drive Alone',
         'Long commute - driving alone_95% CI - Low',
         'Long commute - driving alone_95% CI - High',
         'Long commute - driving alone_Z-Score']
                             ,axis=1)
        
        # Clean Additional Data Second by addressing multiindex column names
        df2.columns = df2.columns.map('_'.join)
        
        # Rename Columns
        df2.rename(columns = {'Unnamed: 0_level_0_FIPS':'FIPS',
         'Unnamed: 1_level_0_State':'State',
         'Unnamed: 2_level_0_County':'County/Borough/Parish',
         'Unnamed: 2_level_0_Borough':'County/Borough/Parish',
         'Unnamed: 2_level_0_Parish':'County/Borough/Parish',
         'Premature age-adjusted mortality_# Deaths':'Premature_Deaths',
         'Premature age-adjusted mortality_Age-Adjusted Mortality':'Premature(under75)_Age_Adjusted_Mortality_per_100K',
         'Child mortality_Child Mortality Rate':'Child(under18)_Mortality_per_100K',
         'Infant mortality_Infant Mortality Rate':'Infant(within1yr)_Mortality_per_1K_live_births',
         'Diabetes prevalence_% Diabetic':'%_Adults(20+)_Diagnosed_Diabetes',
         'HIV prevalence_HIV Prevalence Rate':'#_Ppl(Age13+)_diagnosed_HIV_per_100K',
         'Food insecurity_% Food Insecure':'%_Pop_w/o_Adequate_Access_to_Food',
         'Drug overdose deaths_Drug Overdose Mortality Rate':'Drug_Overdose_Deaths_per_100K',
         'Motor vehicle crash deaths_MV Mortality Rate':'Motor_Vehicle_Deaths_per_100K',
         'Uninsured children_% Uninsured':'%_Children(under19)_w/o_Health_Insurance',
         'Other primary care providers_Other PCP Rate':'Other_Primary_Care_Providers(non_physician)_per_100K',
         'Median household income_Household Income':'Median_Household_Income',
         'Children eligible for free or reduced price lunch_% Free or Reduced Lunch':'%_Children_Enrolled_in_Free/Reduced_Lunch',
         'Homicides_Homicide Rate':'#_Deaths_due_to_Homicide_per_100K',
         'Demographics_Population':'Resident_Population',
         'Demographics_% < 18':'%_under_18_years_of_age',
         'Demographics_% 65 and over':'%_Age_65_and_over',
         'Demographics_# African American':'#_Non-Hispanic_Black_or_African_American',
         'Demographics_% African American':'%_Non-Hispanic_Black_or_African_American',
         'Demographics_# American Indian/Alaskan Native':'#_American_Indian_or_Alaskan_Native',
         'Demographics_% American Indian/Alaskan Native':'%_American_Indian_or_Alaskan_Native',
         'Demographics_# Asian':'#_Asian',
         'Demographics_% Asian':'%_Asian',
         'Demographics_# Native Hawaiian/Other Pacific Islander':'#_Native_Hawaiian/Other_Pacific_Islander',
         'Demographics_% Native Hawaiian/Other Pacific Islander':'%_Native_Hawaiian/Other_Pacific_Islander',
         'Demographics_# Hispanic':'#_Hispanic',
         'Demographics_% Hispanic':'%_Hispanic',
         'Demographics_# Non-Hispanic White':'#_Non-Hispanic_White',
         'Demographics_% Non-Hispanic White':'%_Non-Hispanic_White',
         'Demographics_# Not Proficient in English':'#_Not_Proficient_in_English',
         'Demographics_% Not Proficient in English':'%_Not_Proficient_in_English',
         'Demographics_% Female':'%_Female',
         'Demographics_# Rural':'#_Pop_Living_in_Rural_Area',
         'Demographics_% Rural':'%_Rural'}, 
           inplace = True)
        
        # Drop unnecessary columns
        df2 = df2.drop(['Premature age-adjusted mortality_95% CI - Low',
         'Premature age-adjusted mortality_95% CI - High',
         'Child mortality_# Deaths',
         'Child mortality_95% CI - Low',
         'Child mortality_95% CI - High',
         'Infant mortality_# Deaths',
         'Infant mortality_95% CI - Low',
         'Infant mortality_95% CI - High',
         'Frequent physical distress_% Frequent Physical Distress',
         'Frequent physical distress_95% CI - Low',
         'Frequent physical distress_95% CI - High',
         'Frequent mental distress_% Frequent Mental Distress',
         'Frequent mental distress_95% CI - Low',
         'Frequent mental distress_95% CI - High',
         'Diabetes prevalence_95% CI - Low',
         'Diabetes prevalence_95% CI - High',
         'HIV prevalence_# HIV Cases',
         'Food insecurity_# Food Insecure',
         'Limited access to healthy foods_# Limited Access',
         'Limited access to healthy foods_% Limited Access',
         'Drug overdose deaths_# Drug Overdose Deaths',
         'Motor vehicle crash deaths_# Motor Vehicle Deaths',
         'Motor vehicle crash deaths_95% CI - Low',
         'Motor vehicle crash deaths_95% CI - High',
         'Insufficient sleep_% Insufficient Sleep',
         'Insufficient sleep_95% CI - Low',
         'Insufficient sleep_95% CI - High',
         'Uninsured adults_# Uninsured',
         'Uninsured adults_% Uninsured',
         'Uninsured adults_95% CI - Low',
         'Uninsured adults_95% CI - High',
         'Uninsured children_# Uninsured',
         'Uninsured children_95% CI - Low',
         'Uninsured children_95% CI - High',
         'Health care costs_Costs',
         'Other primary care providers_Other PCP Ratio',
         'Disconnected youth_% Disconnected Youth',
         'Median household income_95% CI - Low',
         'Median household income_95% CI - High',
         'Median household income_Household Income (white alone)',
         'Median household income_Household Income (black alone)',
         'Median household income_Household income (Hispanic)',
         'Residential segregation - black/white_Segregation index',
         'Residential segregation - non-white/white_Segregation Index',
         'Homicides_95% CI - Low',
         'Homicides_95% CI - High',
         'Firearm fatalities_# Firearm Fatalities',
         'Firearm fatalities_Firearm Fatalities Rate',
         'Firearm fatalities_95% CI - Low',
         'Firearm fatalities_95% CI - High',
         'Demographics_95% CI - Low',
         'Demographics_95% CI - High']
                     ,axis=1)

        
        df3.columns = df3.columns.map('_'.join)
        df3.rename(columns = {
             'Unnamed: 0_level_0_FIPS':'FIPS',
             'Unnamed: 1_level_0_State':'State',
             'Unnamed: 2_level_0_County':'County/Borough/Parish',
             'Unnamed: 2_level_0_Borough':'County/Borough/Parish',
             'Unnamed: 2_level_0_Parish':'County/Borough/Parish'}, 
                   inplace = True)
        df3 = df3.drop(['Health Outcomes_Z-Score','Health Factors_Z-Score','Health Factors_Rank'], axis=1)
        
        # Join two dataframes
        joined_df = pd.merge(df, df2, on='FIPS', how='outer')
        joined_df = joined_df.drop(['State_y','County/Borough/Parish_y'],axis=1)
        joined_df.rename(columns = {'State_x':'State','County/Borough/Parish_x':'County/Borough/Parish'},inplace = True)
        
        joined_df = pd.merge(joined_df, df3, on='FIPS', how='outer')
        joined_df = joined_df.drop(['State_y','County/Borough/Parish_y'],axis=1)
        joined_df.rename(columns = {'State_x':'State','County/Borough/Parish_x':'County/Borough/Parish'},inplace = True)
        
        joined_df['Year'] = '2017'
        
        # append to list
        appended_data2017.append(joined_df)
        
#convert to dataframe
appended_data2017 = pd.concat(appended_data2017)

2017 County Health Rankings Alabama Data - v2.xls
2017 County Health Rankings Alaska Data - v2.xls
2017 County Health Rankings Arizona Data - v2.xls
2017 County Health Rankings Arkansas Data - v2.xls
2017 County Health Rankings California Data - v2.xls
2017 County Health Rankings Colorado Data - v2.xls
2017 County Health Rankings Connecticut Data - v2.xls
2017 County Health Rankings Delaware Data - v2.xls
2017 County Health Rankings District of Columbia Data - v2.xls
2017 County Health Rankings Florida Data - v2.xls
2017 County Health Rankings Georgia Data - v2.xls
2017 County Health Rankings Hawaii Data - v2.xls
2017 County Health Rankings Idaho Data - v2.xls
2017 County Health Rankings Illinois Data - v2.xls
2017 County Health Rankings Indiana Data - v2.xls
2017 County Health Rankings Iowa Data - v2.xls
2017 County Health Rankings Kansas Data - v2.xls
2017 County Health Rankings Kentucky Data - v2.xls
2017 County Health Rankings Louisiana Data - v2.xls
2017 County Health Rankings Mai

In [27]:
appended_data2017.head(20)

,FIPS,State,County/Borough/Parish,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,%_Low_Birthweight_Births,%_Adult_Smokers,%_Adult_Obesity,"Healthy_Food_Access(0=Worst,10=Best)",%_Adults_Physical_Inactivity,...,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Health Outcomes_Rank,Year
0,1000,Alabama,NaN,9573.2,21.225378,10.179326,21.404947,34.040806,6.5,27.684505,...,4.184521,3204583,65.951777,55798,1.230232,51.570402,1957932,40.963183,NaN,2017
1,1001,Alabama,Autauga,9158.2,17.878814,8.789277,17.380239,34.100000,6.9,28.600000,...,2.842069,41459,74.907402,338,0.650263,51.475238,22921,42.002162,8,2017
2,1003,Alabama,Baldwin,7393.9,16.361973,8.630640,17.937120,27.400000,7.5,22.300000,...,4.549627,168996,82.959516,2159,1.169385,51.260867,77060,42.279099,2,2017
3,1005,Alabama,Barbour,8573.1,25.020401,11.669058,22.776689,44.400000,4.9,31.800000,...,4.424478,12340,46.585375,447,1.755971,46.589150,18613,67.789635,27,2017
4,1007,Alabama,Bibb,11766.1,19.540231,11.968236,19.850766,40.300000,7.3,33.900000,...,2.408892,16724,74.055706,76,0.353966,46.109906,15663,68.352607,48,2017
5,1009,Alabama,Blount,9332.8,18.947608,7.348972,21.037382,34.600000,8.3,28.000000,...,8.953930,50403,87.394448,918,1.694821,50.592132,51562,89.951502,11,2017
6,1011,Alabama,Bullock,10857.7,28.257945,13.429952,22.774925,43.000000,3.8,31.700000,...,7.526178,2386,22.307405,306,3.059082,45.241212,5607,51.374382,54,2017
7,1013,Alabama,Butler,12427.4,26.621290,11.141603,21.832347,36.300000,6.0,34.400000,...,1.170983,10639,52.788528,93,0.486631,53.503027,14921,71.232157,57,2017
8,1015,Alabama,Calhoun,10716.9,22.003205,8.865321,20.320382,32.200000,6.1,31.300000,...,3.614427,83958,72.615464,884,0.804866,51.777374,39955,33.696826,37,2017
9,1017,Alabama,Chambers,11410.4,24.489617,11.429582,20.905875,39.700000,5.6,31.600000,...,2.025027,19239,56.381326,77,0.240332,52.114410,16816,49.148034,51,2017


In [28]:
list(appended_data2017.columns.values)

['FIPS',
 'State',
 'County/Borough/Parish',
 'Yrs_Potential_Life_Lost_per100K(YPLL)',
 '%_Fair/Poor_Health',
 '%_Low_Birthweight_Births',
 '%_Adult_Smokers',
 '%_Adult_Obesity',
 'Healthy_Food_Access(0=Worst,10=Best)',
 '%_Adults_Physical_Inactivity',
 '%_Access_to_Excercise_Opportunities',
 '%_Adults_Excessive_Drinking',
 '%_Driving_Deaths_with_Alcohol_Involvement',
 'Chlamydia_Cases_per_100K',
 'Teen_Birth_Rate',
 '%_Ppl_Under65_w/o_Insurance',
 'PCP_per_100K',
 'Dentists_per_100K',
 'MHP_per_100K',
 'Preventable_Hospitalization_Rate(per100K_Medicare_Enrollees)',
 'Graduation_Rate',
 '%_Adults(25-44)_with_Some_Post-secondary_Ed',
 '%_Ppl(16+)_Unemployed_&_Looking_for_Work',
 '%_Children(under18)_Living_in_Poverty',
 '80th_Percentile_MHI',
 '20th_Percentile_MHI',
 '%_Children_Living_in_Single_Parent_Household',
 'Associations_per_10K',
 'Violent_Crimes_per_100K',
 'Injury_Death_Rate_per_100K',
 'Avg_Daily_Particulate_Matter_in_Micrograms_per_Cubic_Meter',
 'County_Affected_by_Water_V

In [29]:
appended_data2017.count()

FIPS                                                            3187
State                                                           3187
County/Borough/Parish                                           3136
Yrs_Potential_Life_Lost_per100K(YPLL)                           3008
%_Fair/Poor_Health                                              3187
%_Low_Birthweight_Births                                        3091
%_Adult_Smokers                                                 3187
%_Adult_Obesity                                                 3187
Healthy_Food_Access(0=Worst,10=Best)                            3187
%_Adults_Physical_Inactivity                                    3187
%_Access_to_Excercise_Opportunities                             3114
%_Adults_Excessive_Drinking                                     3187
%_Driving_Deaths_with_Alcohol_Involvement                       3169
Chlamydia_Cases_per_100K                                        3037
Teen_Birth_Rate                   

In [30]:
appended_data2017.isnull().any()

FIPS                                                            False
State                                                           False
County/Borough/Parish                                            True
Yrs_Potential_Life_Lost_per100K(YPLL)                            True
%_Fair/Poor_Health                                              False
%_Low_Birthweight_Births                                         True
%_Adult_Smokers                                                 False
%_Adult_Obesity                                                 False
Healthy_Food_Access(0=Worst,10=Best)                            False
%_Adults_Physical_Inactivity                                    False
%_Access_to_Excercise_Opportunities                              True
%_Adults_Excessive_Drinking                                     False
%_Driving_Deaths_with_Alcohol_Involvement                        True
Chlamydia_Cases_per_100K                                         True
Teen_Birth_Rate     

In [31]:
appended_data2017.loc[appended_data2017.isnull().any(axis=1)]

,FIPS,State,County/Borough/Parish,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,%_Low_Birthweight_Births,%_Adult_Smokers,%_Adult_Obesity,"Healthy_Food_Access(0=Worst,10=Best)",%_Adults_Physical_Inactivity,...,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Health Outcomes_Rank,Year
0,1000,Alabama,NaN,9573.2,21.225378,10.179326,21.404947,34.040806,6.5,27.684505,...,4.184521,3204583,65.951777,55798,1.230232,51.570402,1957932,40.963183,NaN,2017
3,1005,Alabama,Barbour,8573.1,25.020401,11.669058,22.776689,44.400000,4.9,31.800000,...,4.424478,12340,46.585375,447,1.755971,46.589150,18613,67.789635,27,2017
4,1007,Alabama,Bibb,11766.1,19.540231,11.968236,19.850766,40.300000,7.3,33.900000,...,2.408892,16724,74.055706,76,0.353966,46.109906,15663,68.352607,48,2017
6,1011,Alabama,Bullock,10857.7,28.257945,13.429952,22.774925,43.000000,3.8,31.700000,...,7.526178,2386,22.307405,306,3.059082,45.241212,5607,51.374382,54,2017
7,1013,Alabama,Butler,12427.4,26.621290,11.141603,21.832347,36.300000,6.0,34.400000,...,1.170983,10639,52.788528,93,0.486631,53.503027,14921,71.232157,57,2017
9,1017,Alabama,Chambers,11410.4,24.489617,11.429582,20.905875,39.700000,5.6,31.600000,...,2.025027,19239,56.381326,77,0.240332,52.114410,16816,49.148034,51,2017
10,1019,Alabama,Cherokee,12533.9,19.294713,8.411215,18.420824,33.600000,7.9,33.500000,...,1.539116,23613,91.314436,13,0.052583,50.349975,22282,85.736273,45,2017
11,1021,Alabama,Chilton,10678.3,20.852634,9.179590,20.112230,35.900000,7.8,30.400000,...,7.703161,35270,80.263068,1006,2.453718,50.854516,37858,86.744724,32,2017
12,1023,Alabama,Choctaw,12937.3,24.951272,10.031024,20.478585,40.800000,5.4,30.000000,...,0.759301,7422,56.355353,0,0.000000,52.528474,13859,100.000000,56,2017
13,1025,Alabama,Clarke,9468.9,22.904340,12.657590,20.058759,38.200000,4.6,27.500000,...,1.410334,13101,53.094225,39,0.164446,52.725431,19628,75.980335,40,2017


In [32]:
appended_data2017 = appended_data2017.fillna(value=0)
appended_data2017.head(20)

,FIPS,State,County/Borough/Parish,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,%_Low_Birthweight_Births,%_Adult_Smokers,%_Adult_Obesity,"Healthy_Food_Access(0=Worst,10=Best)",%_Adults_Physical_Inactivity,...,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Health Outcomes_Rank,Year
0,1000,Alabama,0,9573.2,21.225378,10.179326,21.404947,34.040806,6.5,27.684505,...,4.184521,3204583,65.951777,55798,1.230232,51.570402,1957932,40.963183,0,2017
1,1001,Alabama,Autauga,9158.2,17.878814,8.789277,17.380239,34.100000,6.9,28.600000,...,2.842069,41459,74.907402,338,0.650263,51.475238,22921,42.002162,8,2017
2,1003,Alabama,Baldwin,7393.9,16.361973,8.630640,17.937120,27.400000,7.5,22.300000,...,4.549627,168996,82.959516,2159,1.169385,51.260867,77060,42.279099,2,2017
3,1005,Alabama,Barbour,8573.1,25.020401,11.669058,22.776689,44.400000,4.9,31.800000,...,4.424478,12340,46.585375,447,1.755971,46.589150,18613,67.789635,27,2017
4,1007,Alabama,Bibb,11766.1,19.540231,11.968236,19.850766,40.300000,7.3,33.900000,...,2.408892,16724,74.055706,76,0.353966,46.109906,15663,68.352607,48,2017
5,1009,Alabama,Blount,9332.8,18.947608,7.348972,21.037382,34.600000,8.3,28.000000,...,8.953930,50403,87.394448,918,1.694821,50.592132,51562,89.951502,11,2017
6,1011,Alabama,Bullock,10857.7,28.257945,13.429952,22.774925,43.000000,3.8,31.700000,...,7.526178,2386,22.307405,306,3.059082,45.241212,5607,51.374382,54,2017
7,1013,Alabama,Butler,12427.4,26.621290,11.141603,21.832347,36.300000,6.0,34.400000,...,1.170983,10639,52.788528,93,0.486631,53.503027,14921,71.232157,57,2017
8,1015,Alabama,Calhoun,10716.9,22.003205,8.865321,20.320382,32.200000,6.1,31.300000,...,3.614427,83958,72.615464,884,0.804866,51.777374,39955,33.696826,37,2017
9,1017,Alabama,Chambers,11410.4,24.489617,11.429582,20.905875,39.700000,5.6,31.600000,...,2.025027,19239,56.381326,77,0.240332,52.114410,16816,49.148034,51,2017


In [33]:
# appended_data2017.to_csv(os.path.join("Cleaned_Files","2017_County_Health_Ranking_Data.csv"),index=False)

### Loop to move through files within 2016 Data Folder
- join 3 worksheets within each state file together
- rename required columns
- drop unnecessary columns
- append all state data to single dataframe for 2016

In [34]:
appended_data2016 = []
path = os.path.join("Original_Files","CountyHealthRankingsData","2016")
for WorkingFile in os.listdir(path):
        #identifies the list of files to be grouped together from designated folder
        print(WorkingFile)
        
        # Import the excel file by joining the path directory and file name together and call Ranked Data and Additonal Measure tabs
        file = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Ranked Measure Data',header=(0,1))
        file2 = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Additional Measure Data',header=(0,1))
        file3 = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Outcomes & Factors Rankings',header=(0,1))
       
        # Load the excel files as two dataframes
        df = pd.DataFrame(file)
        df2 = pd.DataFrame(file2)
        df3 = pd.DataFrame(file3)
        
        # Code from Data_cleaning
       
        # Clean Ranked Data First by addressing multiindex column names
        df.columns = df.columns.map('_'.join)
        
        # Rename Columns
        df.rename(columns = {'Unnamed: 0_level_0_FIPS' : 'FIPS',
 'Unnamed: 1_level_0_State' :'State',
 'Unnamed: 2_level_0_County':'County/Borough/Parish',
 'Unnamed: 2_level_0_Borough':'County/Borough/Parish',
 'Unnamed: 2_level_0_Parish':'County/Borough/Parish',
 'Premature death_Years of Potential Life Lost Rate':'Yrs_Potential_Life_Lost_per100K(YPLL)',
 'Poor or fair health_% Fair/Poor':'%_Fair/Poor_Health',
 'Low birthweight_% LBW':'%_Low_Birthweight_Births',
 'Adult smoking_% Smokers':'%_Adult_Smokers',
 'Adult obesity_% Obese':'%_Adult_Obesity',
 'Food environment index_Food Environment Index':'Healthy_Food_Access(0=Worst,10=Best)',
 'Physical inactivity_% Physically Inactive':'%_Adults_Physical_Inactivity',
 'Access to exercise opportunities_% With Access':'%_Access_to_Excercise_Opportunities',
 'Excessive drinking_% Excessive Drinking':'%_Adults_Excessive_Drinking',
 'Alcohol-impaired driving deaths_% Alcohol-Impaired':'%_Driving_Deaths_with_Alcohol_Involvement',
 'Sexually transmitted infections_Chlamydia Rate':'Chlamydia_Cases_per_100K',
 'Teen births_Teen Birth Rate':'Teen_Birth_Rate',
 'Uninsured_% Uninsured':'%_Ppl_Under65_w/o_Insurance',                            
 'Primary care physicians_PCP Rate':'PCP_per_100K',
 'Dentists_Dentist Rate':'Dentists_per_100K',
 'Mental health providers^_MHP Rate':'MHP_per_100K',
 'Preventable hospital stays_Preventable Hosp. Rate':'Preventable_Hospitalization_Rate(per100K_Medicare_Enrollees)',
 'High school graduation_Graduation Rate':'Graduation_Rate',
 'Some college_% Some College':'%_Adults(25-44)_with_Some_Post-secondary_Ed',
 'Unemployment_% Unemployed':'%_Ppl(16+)_Unemployed_&_Looking_for_Work',
 'Children in poverty_% Children in Poverty':'%_Children(under18)_Living_in_Poverty',
 'Income inequality_80th Percentile Income':'80th_Percentile_MHI',
 'Income inequality_20th Percentile Income':'20th_Percentile_MHI',
 'Children in single-parent households_% Single-Parent Households':'%_Children_Living_in_Single_Parent_Household',
 'Social associations_Association Rate':'Associations_per_10K',
 'Violent crime_Annual Average Violent Crimes':'Annual_Avg_Violent_Crimes',
 'Violent crime_Violent Crime Rate':'Violent_Crimes_per_100K',
 'Injury deaths_Injury Death Rate':'Injury_Death_Rate_per_100K',
 'Air pollution - particulate matter_Average Daily PM2.5':'Avg_Daily_Particulate_Matter_in_Micrograms_per_Cubic_Meter',
 'Drinking water violations_Presence of violation':'County_Affected_by_Water_Violation(1=Yes,0=No)',
 'Severe housing problems_% Severe Housing Problems':'%_Households_with_One_or_More Problems(overcrowding,high-housing-cost,lack-kitchen,lack-plumbing-facilities)',
 'Long commute - driving alone_% Long Commute - Drives Alone':'%_Lone_Commuters_who_Commute_More_Than_30Min'}, 
                  inplace = True)
        
        # Drop unnecessary columns
        df = df.drop(['Premature death_# Deaths',
 'Premature death_95% CI - Low',
 'Premature death_95% CI - High',
 'Premature death_Z-Score',
 'Poor or fair health_95% CI - Low',
 'Poor or fair health_95% CI - High',
 'Poor or fair health_Z-Score',
 'Poor physical health days_Physically Unhealthy Days',
 'Poor physical health days_95% CI - Low',
 'Poor physical health days_95% CI - High',
 'Poor physical health days_Z-Score',
 'Poor mental health days_Mentally Unhealthy Days',
 'Poor mental health days_95% CI - Low',
 'Poor mental health days_95% CI - High',
 'Poor mental health days_Z-Score',
 'Low birthweight_Unreliable',
 'Low birthweight_# Low Birthweight Births',
 'Low birthweight_# Live Births',
 'Low birthweight_95% CI - Low',
 'Low birthweight_95% CI - High',
 'Low birthweight_Z-Score',
 'Adult smoking_95% CI - Low',
 'Adult smoking_95% CI - High',
 'Adult smoking_Z-Score',
 'Adult obesity_95% CI - Low',
 'Adult obesity_95% CI - High',
 'Adult obesity_Z-Score',
 'Food environment index_Z-Score',
 'Physical inactivity_95% CI - Low',
 'Physical inactivity_95% CI - High',
 'Physical inactivity_Z-Score',
 'Access to exercise opportunities_Z-Score',
 'Excessive drinking_95% CI - Low',
 'Excessive drinking_95% CI - High',
 'Excessive drinking_Z-Score',
 'Alcohol-impaired driving deaths_# Alcohol-Impaired Driving Deaths',
 'Alcohol-impaired driving deaths_# Driving Deaths',
 'Alcohol-impaired driving deaths_95% CI - Low',
 'Alcohol-impaired driving deaths_95% CI - High',
 'Alcohol-impaired driving deaths_Z-Score',
 'Sexually transmitted infections_# Chlamydia Cases',
 'Sexually transmitted infections_Z-Score',
 'Teen births_Teen Births',
 'Teen births_Teen Population',
 'Teen births_95% CI - Low',
 'Teen births_95% CI - High',
 'Teen births_Z-Score',
 'Uninsured_# Uninsured',
 'Uninsured_95% CI - Low',
 'Uninsured_95% CI - High',
 'Uninsured_Z-Score',
 'Primary care physicians_# Primary Care Physicians',
 'Primary care physicians_PCP Ratio',
 'Primary care physicians_Z-Score',
 'Dentists_# Dentists',
 'Dentists_Dentist Ratio',
 'Dentists_Z-Score',
 'Mental health providers^_# Mental Health Providers',
 'Mental health providers^_MHP Ratio',
 'Previous Mental health providers data -- used to calculate rankings^_# Mental Health Providers',
 'Previous Mental health providers data -- used to calculate rankings^_MHP Rate',
 'Previous Mental health providers data -- used to calculate rankings^_MHP Ratio',
 'Previous Mental health providers data -- used to calculate rankings^_Z-Score',
 'Preventable hospital stays_# Medicare Enrollees',
 'Preventable hospital stays_95% CI - Low',
 'Preventable hospital stays_95% CI - High',
 'Preventable hospital stays_Z-Score',
 'Diabetic monitoring_# Diabetics',
 'Diabetic monitoring_% Receiving HbA1c',
 'Diabetic monitoring_95% CI - Low',
 'Diabetic monitoring_95% CI - High',
 'Diabetic monitoring_Z-Score',
 'Mammography screening_# Medicare Enrollees',
 'Mammography screening_% Mammography',
 'Mammography screening_95% CI - Low',
 'Mammography screening_95% CI - High',
 'Mammography screening_Z-Score',
 'High school graduation_Cohort Size',
 'High school graduation_Z-Score',
 'Some college_# Some College',
 'Some college_Population',
 'Some college_95% CI - Low',
 'Some college_95% CI - High',
 'Some college_Z-Score',
 'Unemployment_# Unemployed',
 'Unemployment_Labor Force',
 'Unemployment_Z-Score',
 'Children in poverty_# Children in Poverty',
 'Children in poverty_95% CI - Low',
 'Children in poverty_95% CI - High',
 'Children in poverty_Z-Score',
 'Income inequality_Income Ratio',
 'Income inequality_Z-Score',
 'Children in single-parent households_# Single-Parent Households',
 'Children in single-parent households_# Households',
 'Children in single-parent households_95% CI - Low',
 'Children in single-parent households_95% CI - High',
 'Children in single-parent households_Z-Score',
 'Social associations_# Associations',
 'Social associations_Z-Score',
 'Violent crime_# Violent Crimes',
 'Violent crime_Z-Score',
 'Injury deaths_# Injury Deaths',
 'Injury deaths_95% CI - Low',
 'Injury deaths_95% CI - High',
 'Injury deaths_Z-Score',
 'Air pollution - particulate matter_Z-Score',
 'Drinking water violations_Z-Score',
 'Severe housing problems_# Households with Severe Problems',
 'Severe housing problems_95% CI - Low',
 'Severe housing problems_95% CI - High',
 'Severe housing problems_Z-Score',
 'Driving alone to work_# Drive Alone',
 'Driving alone to work_# Workers',
 'Driving alone to work_% Drive Alone',
 'Driving alone to work_95% CI - Low',
 'Driving alone to work_95% CI - High',
 'Driving alone to work_Z-Score',
 'Long commute - driving alone_# Workers who Drive Alone',
 'Long commute - driving alone_95% CI - Low',
 'Long commute - driving alone_95% CI - High',
 'Long commute - driving alone_Z-Score']
                             ,axis=1)
        
        # Clean Additional Data Second by addressing multiindex column names
        df2.columns = df2.columns.map('_'.join)
        
        # Rename Columns
        df2.rename(columns = {'Unnamed: 0_level_0_FIPS':'FIPS',
 'Unnamed: 1_level_0_State':'State',
 'Unnamed: 2_level_0_County':'County/Borough/Parish',
 'Unnamed: 2_level_0_Borough':'County/Borough/Parish',
 'Unnamed: 2_level_0_Parish':'County/Borough/Parish',
 'Premature age-adjusted mortality_# Deaths':'Premature_Deaths',
 'Premature age-adjusted mortality_Age-Adjusted Mortality':'Premature(under75)_Age_Adjusted_Mortality_per_100K',
 'Child mortality_Child Mortality Rate':'Child(under18)_Mortality_per_100K',
 'Infant mortality_Infant Mortality Rate':'Infant(within1yr)_Mortality_per_1K_live_births',
 'Diabetes prevalence_% Diabetic':'%_Adults(20+)_Diagnosed_Diabetes',
 'HIV prevalence_HIV Prevalence Rate':'#_Ppl(Age13+)_diagnosed_HIV_per_100K',
 'Food insecurity_% Food Insecure':'%_Pop_w/o_Adequate_Access_to_Food',
 'Drug overdose deaths_Drug Overdose Mortality Rate':'Drug_Overdose_Deaths_per_100K',
 'Motor vehicle crash deaths_MV Mortality Rate':'Motor_Vehicle_Deaths_per_100K',
 'Uninsured children_% Uninsured':'%_Children(under19)_w/o_Health_Insurance',
 'Other primary care providers^_Other PCP Rate':'Other_Primary_Care_Providers(non_physician)_per_100K',
 'Median household income_Household Income':'Median_Household_Income',
 'Children eligible for free lunch_% Free Lunch':'%_Children_Enrolled_in_Free/Reduced_Lunch',
 'Homicides_Homicide Rate':'#_Deaths_due_to_Homicide_per_100K',
 'Demographics_Population':'Resident_Population',
 'Demographics_% < 18':'%_under_18_years_of_age',
 'Demographics_% 65 and over':'%_Age_65_and_over',
 'Demographics_# African American':'#_Non-Hispanic_Black_or_African_American',
 'Demographics_% African American':'%_Non-Hispanic_Black_or_African_American',
 'Demographics_# American Indian/Alaskan Native':'#_American_Indian_or_Alaskan_Native',
 'Demographics_% American Indian/Alaskan Native':'%_American_Indian_or_Alaskan_Native',
 'Demographics_# Asian':'#_Asian',
 'Demographics_% Asian':'%_Asian',
 'Demographics_# Native Hawaiian/Other Pacific Islander':'#_Native_Hawaiian/Other_Pacific_Islander',
 'Demographics_% Native Hawaiian/Other Pacific Islander':'%_Native_Hawaiian/Other_Pacific_Islander',
 'Demographics_# Hispanic':'#_Hispanic',
 'Demographics_% Hispanic':'%_Hispanic',
 'Demographics_# Non-Hispanic White':'#_Non-Hispanic_White',
 'Demographics_% Non-Hispanic White':'%_Non-Hispanic_White',
 'Demographics_# Not Proficient in English':'#_Not_Proficient_in_English',
 'Demographics_% Not Proficient in English':'%_Not_Proficient_in_English',
 'Demographics_% Female':'%_Female',
 'Demographics_# Rural':'#_Pop_Living_in_Rural_Area',
 'Demographics_% Rural':'%_Rural'}, 
           inplace = True)
        
        # Drop unnecessary columns
        df2 = df2.drop([
 'Premature age-adjusted mortality_95% CI - Low',
 'Premature age-adjusted mortality_95% CI - High',
 'Child mortality_# Deaths',
 'Child mortality_95% CI - Low',
 'Child mortality_95% CI - High',
 'Infant mortality_# Deaths',
 'Infant mortality_95% CI - Low',
 'Infant mortality_95% CI - High',
 'Frequent physical distress_% Frequent Physical Distress',
 'Frequent physical distress_95% CI - Low',
 'Frequent physical distress_95% CI - High',
 'Frequent mental distress_% Frequent Mental Distress',
 'Frequent mental distress_95% CI - Low',
 'Frequent mental distress_95% CI - High',
 'Diabetes prevalence_95% CI - Low',
 'Diabetes prevalence_95% CI - High',
 'HIV prevalence_# HIV Cases',
 'Food insecurity_# Food Insecure',
 'Limited access to healthy foods_# Limited Access',
 'Limited access to healthy foods_% Limited Access',
 'Drug overdose deaths_# Drug Overdose Deaths',
 'Drug overdose deaths - modeled_Range Drug Overdose Mortality Rate',
 'Motor vehicle crash deaths_# Motor Vehicle Deaths',
 'Motor vehicle crash deaths_95% CI - Low',
 'Motor vehicle crash deaths_95% CI - High',
 'Insufficient sleep_% Insufficient Sleep',
 'Insufficient sleep_95% CI - Low',
 'Insufficient sleep_95% CI - High',
 'Uninsured adults_# Uninsured',
 'Uninsured adults_% Uninsured',
 'Uninsured adults_95% CI - Low',
 'Uninsured adults_95% CI - High',
 'Uninsured children_# Uninsured',
 'Uninsured children_95% CI - Low',
 'Uninsured children_95% CI - High',
 'Health care costs_Costs',
 'Other primary care providers^_Other PCP Ratio',
 'Previous Other primary care providers data^_Other PCP Rate',
 'Previous Other primary care providers data^_Other PCP Ratio',
 'Median household income_95% CI - Low',
 'Median household income_95% CI - High',
 'Residential segregation - black/white_Segregation index',
 'Residential segregation - \nnon-white/white_Segregation Index',
 'Homicides_95% CI - Low',
 'Homicides_95% CI - High',
 'Demographics_95% CI - Low',
 'Demographics_95% CI - High']
                     ,axis=1)
        
        df3.columns = df3.columns.map('_'.join)
        df3.rename(columns = {
             'Unnamed: 0_level_0_FIPS':'FIPS',
             'Unnamed: 1_level_0_State':'State',
             'Unnamed: 2_level_0_County':'County/Borough/Parish',
             'Unnamed: 2_level_0_Borough':'County/Borough/Parish',
             'Unnamed: 2_level_0_Parish':'County/Borough/Parish'}, 
                   inplace = True)
        df3 = df3.drop(['Health Outcomes_Z-Score','Health Factors_Z-Score','Health Factors_Rank'], axis=1)
        
        # Join two dataframes
        joined_df = pd.merge(df, df2, on='FIPS', how='outer')
        joined_df = joined_df.drop(['State_y','County/Borough/Parish_y'],axis=1)
        joined_df.rename(columns = {'State_x':'State','County/Borough/Parish_x':'County/Borough/Parish'},inplace = True)
        
        joined_df = pd.merge(joined_df, df3, on='FIPS', how='outer')
        joined_df = joined_df.drop(['State_y','County/Borough/Parish_y'],axis=1)
        joined_df.rename(columns = {'State_x':'State','County/Borough/Parish_x':'County/Borough/Parish'},inplace = True)
        
        joined_df['Year'] = '2016'
        
        # append to list
        appended_data2016.append(joined_df)
        
#convert to dataframe
appended_data2016 = pd.concat(appended_data2016)

2016 County Health Rankings Alabama Data - v3.xls
2016 County Health Rankings Alaska Data - v3.xls
2016 County Health Rankings Arizona Data - v3.xls
2016 County Health Rankings Arkansas Data - v3.xls
2016 County Health Rankings California Data - v3.xls
2016 County Health Rankings Colorado Data - v3.xls
2016 County Health Rankings Connecticut Data - v3.xls
2016 County Health Rankings Delaware Data - v3.xls
2016 County Health Rankings District of Columbia Data - v3.xls
2016 County Health Rankings Florida Data - v3.xls
2016 County Health Rankings Georgia Data - v3.xls
2016 County Health Rankings Hawaii Data - v3.xls
2016 County Health Rankings Idaho Data - v3.xls
2016 County Health Rankings Illinois Data - v3.xls
2016 County Health Rankings Indiana Data - v3.xls
2016 County Health Rankings Iowa Data - v3.xls
2016 County Health Rankings Kansas Data - v3.xls
2016 County Health Rankings Kentucky Data - v3.xls
2016 County Health Rankings Louisiana Data - v3.xls
2016 County Health Rankings Mai

In [35]:
appended_data2016.head(20)

,FIPS,State,County/Borough/Parish,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,%_Low_Birthweight_Births,%_Adult_Smokers,%_Adult_Obesity,"Healthy_Food_Access(0=Worst,10=Best)",%_Adults_Physical_Inactivity,...,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Health Outcomes_Rank,Year
0,1000.0,Alabama,NaN,9544.7,21.9,10.225973,21.1,33.591186,6.6,29.318673,...,4.137934,3209637.0,66.186584,59335.0,1.313271,51.540847,1957932.0,40.963183,NaN,2016
1,1001.0,Alabama,Autauga,9215.1,19.4,8.786701,18.7,30.900000,7.1,28.700000,...,2.733099,41877.0,75.597076,249.0,0.481429,51.362036,22921.0,42.002162,10,2016
2,1003.0,Alabama,Baldwin,7454.6,16.0,8.507297,18.6,26.700000,7.6,25.200000,...,4.562468,166093.0,83.000435,2571.0,1.429898,51.228568,77060.0,42.279099,2,2016
3,1005.0,Alabama,Barbour,8499.9,25.7,12.148280,21.4,40.800000,5.1,32.900000,...,4.533790,12525.0,46.583851,549.0,2.150664,46.613605,18613.0,67.789635,24,2016
4,1007.0,Alabama,Bibb,11007.7,22.0,12.415730,21.0,40.100000,7.4,33.200000,...,2.106105,16772.0,74.522350,112.0,0.521100,45.934417,15663.0,68.352607,43,2016
5,1009.0,Alabama,Blount,8655.9,20.7,7.472798,19.4,32.400000,8.3,28.500000,...,8.735425,50652.0,87.756198,954.0,1.765032,50.531021,51562.0,89.951502,8,2016
6,1011.0,Alabama,Bullock,10262.1,29.3,13.702359,22.3,44.500000,3.9,33.100000,...,7.460052,2381.0,22.120030,257.0,2.573603,45.252694,5607.0,51.374382,51,2016
7,1013.0,Alabama,Butler,11155.1,27.4,11.004274,22.8,36.000000,6.0,34.400000,...,1.197280,10777.0,53.099133,82.0,0.425974,53.581987,14921.0,71.232157,52,2016
8,1015.0,Alabama,Calhoun,10352.0,22.3,8.873310,23.8,32.000000,6.2,33.600000,...,3.528417,84450.0,72.854481,915.0,0.829721,51.792677,39955.0,33.696826,31,2016
9,1017.0,Alabama,Chambers,11923.0,25.8,12.128713,20.8,39.800000,5.7,34.800000,...,1.974997,19370.0,56.843526,67.0,0.208833,52.318347,16816.0,49.148034,57,2016


In [36]:
list(appended_data2016.columns.values)

['FIPS',
 'State',
 'County/Borough/Parish',
 'Yrs_Potential_Life_Lost_per100K(YPLL)',
 '%_Fair/Poor_Health',
 '%_Low_Birthweight_Births',
 '%_Adult_Smokers',
 '%_Adult_Obesity',
 'Healthy_Food_Access(0=Worst,10=Best)',
 '%_Adults_Physical_Inactivity',
 '%_Access_to_Excercise_Opportunities',
 '%_Adults_Excessive_Drinking',
 '%_Driving_Deaths_with_Alcohol_Involvement',
 'Chlamydia_Cases_per_100K',
 'Teen_Birth_Rate',
 '%_Ppl_Under65_w/o_Insurance',
 'PCP_per_100K',
 'Dentists_per_100K',
 'MHP_per_100K',
 'Preventable_Hospitalization_Rate(per100K_Medicare_Enrollees)',
 'Graduation_Rate',
 '%_Adults(25-44)_with_Some_Post-secondary_Ed',
 '%_Ppl(16+)_Unemployed_&_Looking_for_Work',
 '%_Children(under18)_Living_in_Poverty',
 '80th_Percentile_MHI',
 '20th_Percentile_MHI',
 '%_Children_Living_in_Single_Parent_Household',
 'Associations_per_10K',
 'Violent_Crimes_per_100K',
 'Injury_Death_Rate_per_100K',
 'Avg_Daily_Particulate_Matter_in_Micrograms_per_Cubic_Meter',
 'County_Affected_by_Water_V

In [37]:
appended_data2016.count()

FIPS                                                            3192
State                                                           3192
County/Borough/Parish                                           3141
Yrs_Potential_Life_Lost_per100K(YPLL)                           3040
%_Fair/Poor_Health                                              3191
%_Low_Birthweight_Births                                        3093
%_Adult_Smokers                                                 3191
%_Adult_Obesity                                                 3192
Healthy_Food_Access(0=Worst,10=Best)                            3192
%_Adults_Physical_Inactivity                                    3192
%_Access_to_Excercise_Opportunities                             3118
%_Adults_Excessive_Drinking                                     3191
%_Driving_Deaths_with_Alcohol_Involvement                       3169
Chlamydia_Cases_per_100K                                        3012
Teen_Birth_Rate                   

In [38]:
appended_data2016.isnull().any()

FIPS                                                             True
State                                                            True
County/Borough/Parish                                            True
Yrs_Potential_Life_Lost_per100K(YPLL)                            True
%_Fair/Poor_Health                                               True
%_Low_Birthweight_Births                                         True
%_Adult_Smokers                                                  True
%_Adult_Obesity                                                  True
Healthy_Food_Access(0=Worst,10=Best)                             True
%_Adults_Physical_Inactivity                                     True
%_Access_to_Excercise_Opportunities                              True
%_Adults_Excessive_Drinking                                      True
%_Driving_Deaths_with_Alcohol_Involvement                        True
Chlamydia_Cases_per_100K                                         True
Teen_Birth_Rate     

In [39]:
appended_data2016.loc[appended_data2016.isnull().any(axis=1)]

,FIPS,State,County/Borough/Parish,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,%_Low_Birthweight_Births,%_Adult_Smokers,%_Adult_Obesity,"Healthy_Food_Access(0=Worst,10=Best)",%_Adults_Physical_Inactivity,...,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Health Outcomes_Rank,Year
0,1000.0,Alabama,NaN,9544.7,21.9,10.225973,21.1,33.591186,6.6,29.318673,...,4.137934,3209637.0,66.186584,59335.0,1.313271,51.540847,1957932.0,40.963183,NaN,2016
3,1005.0,Alabama,Barbour,8499.9,25.7,12.148280,21.4,40.800000,5.1,32.900000,...,4.533790,12525.0,46.583851,549.0,2.150664,46.613605,18613.0,67.789635,24,2016
4,1007.0,Alabama,Bibb,11007.7,22.0,12.415730,21.0,40.100000,7.4,33.200000,...,2.106105,16772.0,74.522350,112.0,0.521100,45.934417,15663.0,68.352607,43,2016
6,1011.0,Alabama,Bullock,10262.1,29.3,13.702359,22.3,44.500000,3.9,33.100000,...,7.460052,2381.0,22.120030,257.0,2.573603,45.252694,5607.0,51.374382,51,2016
7,1013.0,Alabama,Butler,11155.1,27.4,11.004274,22.8,36.000000,6.0,34.400000,...,1.197280,10777.0,53.099133,82.0,0.425974,53.581987,14921.0,71.232157,52,2016
9,1017.0,Alabama,Chambers,11923.0,25.8,12.128713,20.8,39.800000,5.7,34.800000,...,1.974997,19370.0,56.843526,67.0,0.208833,52.318347,16816.0,49.148034,57,2016
10,1019.0,Alabama,Cherokee,12007.3,19.8,8.968883,19.8,32.700000,7.8,31.000000,...,1.509390,23854.0,91.615778,25.0,0.100904,50.228521,22282.0,85.736273,38,2016
11,1021.0,Alabama,Chilton,10790.2,20.8,9.339576,21.2,37.100000,7.8,32.000000,...,7.723475,35287.0,80.323689,1138.0,2.784370,50.788737,37858.0,86.744724,30,2016
12,1023.0,Alabama,Choctaw,12530.3,25.4,10.202020,19.0,38.700000,5.8,33.800000,...,0.773099,7468.0,56.053441,2.0,0.015555,52.525707,13859.0,100.000000,53,2016
13,1025.0,Alabama,Clarke,10306.3,25.8,12.719079,21.2,36.100000,4.9,30.200000,...,1.334937,13248.0,53.108839,41.0,0.171405,52.836240,19628.0,75.980335,46,2016


In [40]:
appended_data2016 = appended_data2016.fillna(value=0)
appended_data2016.head(20)

,FIPS,State,County/Borough/Parish,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,%_Low_Birthweight_Births,%_Adult_Smokers,%_Adult_Obesity,"Healthy_Food_Access(0=Worst,10=Best)",%_Adults_Physical_Inactivity,...,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Health Outcomes_Rank,Year
0,1000.0,Alabama,0,9544.7,21.9,10.225973,21.1,33.591186,6.6,29.318673,...,4.137934,3209637.0,66.186584,59335.0,1.313271,51.540847,1957932.0,40.963183,0,2016
1,1001.0,Alabama,Autauga,9215.1,19.4,8.786701,18.7,30.900000,7.1,28.700000,...,2.733099,41877.0,75.597076,249.0,0.481429,51.362036,22921.0,42.002162,10,2016
2,1003.0,Alabama,Baldwin,7454.6,16.0,8.507297,18.6,26.700000,7.6,25.200000,...,4.562468,166093.0,83.000435,2571.0,1.429898,51.228568,77060.0,42.279099,2,2016
3,1005.0,Alabama,Barbour,8499.9,25.7,12.148280,21.4,40.800000,5.1,32.900000,...,4.533790,12525.0,46.583851,549.0,2.150664,46.613605,18613.0,67.789635,24,2016
4,1007.0,Alabama,Bibb,11007.7,22.0,12.415730,21.0,40.100000,7.4,33.200000,...,2.106105,16772.0,74.522350,112.0,0.521100,45.934417,15663.0,68.352607,43,2016
5,1009.0,Alabama,Blount,8655.9,20.7,7.472798,19.4,32.400000,8.3,28.500000,...,8.735425,50652.0,87.756198,954.0,1.765032,50.531021,51562.0,89.951502,8,2016
6,1011.0,Alabama,Bullock,10262.1,29.3,13.702359,22.3,44.500000,3.9,33.100000,...,7.460052,2381.0,22.120030,257.0,2.573603,45.252694,5607.0,51.374382,51,2016
7,1013.0,Alabama,Butler,11155.1,27.4,11.004274,22.8,36.000000,6.0,34.400000,...,1.197280,10777.0,53.099133,82.0,0.425974,53.581987,14921.0,71.232157,52,2016
8,1015.0,Alabama,Calhoun,10352.0,22.3,8.873310,23.8,32.000000,6.2,33.600000,...,3.528417,84450.0,72.854481,915.0,0.829721,51.792677,39955.0,33.696826,31,2016
9,1017.0,Alabama,Chambers,11923.0,25.8,12.128713,20.8,39.800000,5.7,34.800000,...,1.974997,19370.0,56.843526,67.0,0.208833,52.318347,16816.0,49.148034,57,2016


In [41]:
# appended_data2016.to_csv(os.path.join("Cleaned_Files","2016_County_Health_Ranking_Data.csv"),index=False)

### Loop to move through files within 2015 Data Folder
- join 3 worksheets within each state file together
- rename required columns
- drop unnecessary columns
- append all state data to single dataframe for 2015

In [42]:
appended_data2015 = []
path = os.path.join("Original_Files","CountyHealthRankingsData","2015")
for WorkingFile in os.listdir(path):
        #identifies the list of files to be grouped together from designated folder
        print(WorkingFile)
        
        # Import the excel file by joining the path directory and file name together and call Ranked Data and Additonal Measure tabs
        file = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Ranked Measure Data',header=(0,1))
        file2 = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Additional Measure Data',header=(0,1))
        file3 = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Outcomes & Factors Rankings',header=(0,1))
       
        # Load the excel files as two dataframes
        df = pd.DataFrame(file)
        df2 = pd.DataFrame(file2)
        df3 = pd.DataFrame(file3)
        
        # Code from Data_cleaning
       
        # Clean Ranked Data First by addressing multiindex column names
        df.columns = df.columns.map('_'.join)
        
        # Rename Columns
        df.rename(columns = {'Unnamed: 0_level_0_FIPS' : 'FIPS',
 'Unnamed: 1_level_0_State' :'State',
 'Unnamed: 2_level_0_County':'County/Borough/Parish',
 'Unnamed: 2_level_0_Borough':'County/Borough/Parish',
 'Unnamed: 2_level_0_Parish':'County/Borough/Parish',
 'Premature death_Years of Potential Life Lost Rate':'Yrs_Potential_Life_Lost_per100K(YPLL)',
 'Poor or fair health_% Fair/Poor':'%_Fair/Poor_Health',
 'Low birthweight_% LBW':'%_Low_Birthweight_Births',
 'Adult smoking_% Smokers':'%_Adult_Smokers',
 'Adult obesity_% Obese':'%_Adult_Obesity',
 'Food environment index_Food Environment Index':'Healthy_Food_Access(0=Worst,10=Best)',
 'Physical inactivity_% Physically Inactive':'%_Adults_Physical_Inactivity',
 'Access to exercise opportunities_% With Access':'%_Access_to_Excercise_Opportunities',
 'Excessive drinking_% Excessive Drinking':'%_Adults_Excessive_Drinking',
 'Alcohol-impaired driving deaths_% Alcohol-Impaired':'%_Driving_Deaths_with_Alcohol_Involvement',
 'Sexually transmitted infections_Chlamydia Rate':'Chlamydia_Cases_per_100K',
 'Teen births_Teen Birth Rate':'Teen_Birth_Rate',
 'Uninsured_% Uninsured':'%_Ppl_Under65_w/o_Insurance',                            
 'Primary care physicians_PCP Rate':'PCP_per_100K',
 'Dentists_Dentist Rate':'Dentists_per_100K',
 'Mental health providers^_MHP Rate':'MHP_per_100K',
 'Preventable hospital stays_Preventable Hosp. Rate':'Preventable_Hospitalization_Rate(per100K_Medicare_Enrollees)',
 'High school graduation_Graduation Rate':'Graduation_Rate',
 'Some college_% Some College':'%_Adults(25-44)_with_Some_Post-secondary_Ed',
 'Unemployment_% Unemployed':'%_Ppl(16+)_Unemployed_&_Looking_for_Work',
 'Children in poverty_% Children in Poverty':'%_Children(under18)_Living_in_Poverty',
 'Income inequality_80th Percentile Income':'80th_Percentile_MHI',
 'Income inequality_20th Percentile Income':'20th_Percentile_MHI',
 'Children in single-parent households_% Single-Parent Households':'%_Children_Living_in_Single_Parent_Household',
 'Social associations_Association Rate':'Associations_per_10K',
 'Violent crime_Annual Average Violent Crimes':'Annual_Avg_Violent_Crimes',
 'Violent crime_Violent Crime Rate':'Violent_Crimes_per_100K',
 'Injury deaths_Injury Death Rate':'Injury_Death_Rate_per_100K',
 'Air pollution - particulate matter_Average Daily PM2.5':'Avg_Daily_Particulate_Matter_in_Micrograms_per_Cubic_Meter',
 'Drinking water violations_% Pop in Viol':'County_Affected_by_Water_Violation(1=Yes,0=No)',
 'Severe housing problems_% Severe Housing Problems':'%_Households_with_One_or_More Problems(overcrowding,high-housing-cost,lack-kitchen,lack-plumbing-facilities)',
 'Long commute - driving alone_% Long Commute - Drives Alone':'%_Lone_Commuters_who_Commute_More_Than_30Min'}, 
                  inplace = True)
        
        # Drop unnecessary columns
        df = df.drop(['Premature death_# Deaths',
 'Premature death_95% CI - Low',
 'Premature death_95% CI - High',
 'Premature death_Z-Score',
 'Poor or fair health_Sample Size',
 'Poor or fair health_95% CI - Low',
 'Poor or fair health_95% CI - High',
 'Poor or fair health_Z-Score',
 'Poor physical health days_Sample Size',
 'Poor physical health days_Physically Unhealthy Days',
 'Poor physical health days_95% CI - Low',
 'Poor physical health days_95% CI - High',
 'Poor physical health days_Z-Score',
 'Poor mental health days_Sample Size',
 'Poor mental health days_Mentally Unhealthy Days',
 'Poor mental health days_95% CI - Low',
 'Poor mental health days_95% CI - High',
 'Poor mental health days_Z-Score',
 'Low birthweight_Unreliable',
 'Low birthweight_# Low Birthweight Births',
 'Low birthweight_# Live births',
 'Low birthweight_95% CI - Low',
 'Low birthweight_95% CI - High',
 'Low birthweight_Z-Score',
 'Adult smoking_Sample Size',
 'Adult smoking_95% CI - Low',
 'Adult smoking_95% CI - High',
 'Adult smoking_Z-Score',
 'Adult obesity_95% CI - Low',
 'Adult obesity_95% CI - High',
 'Adult obesity_Z-Score',
 'Food environment index_Z-Score',
 'Physical inactivity_95% CI - Low',
 'Physical inactivity_95% CI - High',
 'Physical inactivity_Z-Score',
 'Access to exercise opportunities_# With Access',
 'Access to exercise opportunities_Z-Score',
 'Excessive drinking_Sample Size',
 'Excessive drinking_95% CI - Low',
 'Excessive drinking_95% CI - High',
 'Excessive drinking_Z-Score',
 'Alcohol-impaired driving deaths_# Alcohol-Impaired Driving Deaths',
 'Alcohol-impaired driving deaths_# Driving Deaths',
 'Alcohol-impaired driving deaths_Z-Score',
 'Sexually transmitted infections_# Chlamydia Cases',
 'Sexually transmitted infections_Z-Score',
 'Teen births_Teen Births ',
 'Teen births_Teen Population',
 'Teen births_95% CI - Low',
 'Teen births_95% CI - High',
 'Teen births_Z-Score',
 'Uninsured_# Uninsured',
 'Uninsured_95% CI - Low',
 'Uninsured_95% CI - High',
 'Uninsured_Z-Score',
 'Primary care physicians_# Primary Care Physicians',
 'Primary care physicians_PCP Ratio',
 'Primary care physicians_Z-Score',
 'Dentists_# Dentists',
 'Dentists_Dentist Ratio',
 'Dentists_Z-Score',
 'Mental health providers^_# Mental Health Providers',
 'Mental health providers^_MHP Ratio',
 'Previous Mental health providers data -- used to calculate rankings^_# Mental Health Providers',
 'Previous Mental health providers data -- used to calculate rankings^_MHP Rate',
 'Previous Mental health providers data -- used to calculate rankings^_MHP Ratio',
 'Previous Mental health providers data -- used to calculate rankings^_Z-Score',
 'Preventable hospital stays_# Medicare Enrollees',
 'Preventable hospital stays_95% CI - Low',
 'Preventable hospital stays_95% CI - High',
 'Preventable hospital stays_Z-Score',
 'Diabetic monitoring_# Diabetics',
 'Diabetic monitoring_% Receiving HbA1c',
 'Diabetic monitoring_95% CI - Low',
 'Diabetic monitoring_95% CI - High',
 'Diabetic monitoring_Z-Score',
 'Mammography screening_# Medicare Enrollees',
 'Mammography screening_% Mammography',
 'Mammography screening_95% CI - Low',
 'Mammography screening_95% CI - High',
 'Mammography screening_Z-Score',
 'High school graduation_Cohort Size',
 'High school graduation_Z-Score',
 'Some college_# Some College',
 'Some college_Population',
 'Some college_95% CI - Low',
 'Some college_95% CI - High',
 'Some college_Z-Score',
 'Unemployment_# Unemployed',
 'Unemployment_Labor Force',
 'Unemployment_Z-Score',
 'Children in poverty_# Children in Poverty',
 'Children in poverty_95% CI - Low',
 'Children in poverty_95% CI - High',
 'Children in poverty_Z-Score',
 'Income inequality_Income Ratio',
 'Income inequality_Z-Score',
 'Children in single-parent households_# Single-Parent Households',
 'Children in single-parent households_# Households',
 'Children in single-parent households_95% CI - Low',
 'Children in single-parent households_95% CI - High',
 'Children in single-parent households_Z-Score',
 'Social associations_# Associations',
 'Social associations_Z-Score',
 'Violent crime_# Violent Crimes',
 'Violent crime_Z-Score',
 'Injury deaths_# Injury Deaths',
 'Injury deaths_95% CI - Low',
 'Injury deaths_95% CI - High',
 'Injury deaths_Z-Score',
 'Air pollution - particulate matter_Z-Score',
 'Drinking water violations_Pop. In Viol',
 'Drinking water violations_Z-Score',
 'Severe housing problems_# Households with Severe Problems',
 'Severe housing problems_95% CI - Low',
 'Severe housing problems_95% CI - High',
 'Severe housing problems_Z-Score',
 'Driving alone to work_# Drive Alone',
 'Driving alone to work_% Drive Alone',
 'Driving alone to work_# Workers',
 'Driving alone to work_95% CI - Low',
 'Driving alone to work_95% CI - High',
 'Driving alone to work_Z-Score',
 'Long commute - driving alone_# Workers who Drive Alone',
 'Long commute - driving alone_95% CI - Low',
 'Long commute - driving alone_95% CI - High',
 'Long commute - driving alone_Z-Score']
                             ,axis=1)
        
        # Clean Additional Data Second by addressing multiindex column names
        df2.columns = df2.columns.map('_'.join)
        
        # Rename Columns
        df2.rename(columns = {'Unnamed: 0_level_0_FIPS':'FIPS',
 'Unnamed: 1_level_0_State':'State',
 'Unnamed: 2_level_0_County':'County/Borough/Parish',
 'Unnamed: 2_level_0_Borough':'County/Borough/Parish',
 'Unnamed: 2_level_0_Parish':'County/Borough/Parish',
 'Premature age-adjusted mortality_# Deaths':'Premature_Deaths',
 'Premature age-adjusted mortality_Age-Adjusted Mortality':'Premature(under75)_Age_Adjusted_Mortality_per_100K',
 'Child mortality_Child Mortality Rate':'Child(under18)_Mortality_per_100K',
 'Infant mortality_Infant Mortality Rate':'Infant(within1yr)_Mortality_per_1K_live_births',
 'Diabetes_% Diabetic':'%_Adults(20+)_Diagnosed_Diabetes',
 'HIV prevalence_HIV Prevalence Rate':'#_Ppl(Age13+)_diagnosed_HIV_per_100K',
 'Food insecurity_% Food Insecure':'%_Pop_w/o_Adequate_Access_to_Food',
 'Drug poisoning deaths_Drug Poisoning Mortality Rate':'Drug_Overdose_Deaths_per_100K',
 'Motor vehicle crash deaths_MV Mortality Rate':'Motor_Vehicle_Deaths_per_100K',
 'Uninsured children_% Uninsured':'%_Children(under19)_w/o_Health_Insurance',
 'Other primary care providers^_Other PCP Rate':'Other_Primary_Care_Providers(non_physician)_per_100K',
 'Median household income_Household Income':'Median_Household_Income',
 'Children eligible for free lunch_% Free Lunch':'%_Children_Enrolled_in_Free/Reduced_Lunch',
 'Homicides_Homicide Rate':'#_Deaths_due_to_Homicide_per_100K',
 'Demographics_Population':'Resident_Population',
 'Demographics_% < 18':'%_under_18_years_of_age',
 'Demographics_% 65 and over':'%_Age_65_and_over',
 'Demographics_# African American':'#_Non-Hispanic_Black_or_African_American',
 'Demographics_% African American':'%_Non-Hispanic_Black_or_African_American',
 'Demographics_# American Indian/ Alaskan Native':'#_American_Indian_or_Alaskan_Native',
 'Demographics_% American Indian/ Alaskan Native':'%_American_Indian_or_Alaskan_Native',
 'Demographics_# Asian':'#_Asian',
 'Demographics_% Asian':'%_Asian',
 'Demographics_# Native Hawaiian/ Other Pacific Islander':'#_Native_Hawaiian/Other_Pacific_Islander',
 'Demographics_% Native Hawaiian/ Other Pacific Islander':'%_Native_Hawaiian/Other_Pacific_Islander',
 'Demographics_# Hispanic':'#_Hispanic',
 'Demographics_% Hispanic':'%_Hispanic',
 'Demographics_# Non-Hispanic white':'#_Non-Hispanic_White',
 'Demographics_% Non-Hispanic white':'%_Non-Hispanic_White',
 'Demographics_# Not Proficient in English':'#_Not_Proficient_in_English',
 'Demographics_% Not Proficient in English':'%_Not_Proficient_in_English',
 'Demographics_% Female':'%_Female',
 'Demographics_# Rural':'#_Pop_Living_in_Rural_Area',
 'Demographics_% Rural':'%_Rural'}, 
           inplace = True)
        
        # Drop unnecessary columns
        df2 = df2.drop(['Demographics_# < 18',
 'Demographics_# 65 and over',
 'Demographics_95% CI - Low',
 'Demographics_95% CI - High',
 'Demographics_# Female',
 'Diabetes_# Diabetic',
 'Diabetes_95% CI - Low',
 'Diabetes_95% CI - High',
 'HIV prevalence_# HIV Cases',
 'Premature age-adjusted mortality_95% CI - Low',
 'Premature age-adjusted mortality_95% CI - High',
 'Infant mortality_# Deaths',
 'Infant mortality_95% CI - Low',
 'Infant mortality_95% CI - High',
 'Child mortality_# Deaths',
 'Child mortality_95% CI - Low',
 'Child mortality_95% CI - High',
 'Food insecurity_# Food Insecure',
 'Limited access to healthy foods_# Limited Access',
 'Limited access to healthy foods_% Limited Access',
 'Motor vehicle crash deaths_# Motor Vehicle Deaths',
 'Motor vehicle crash deaths_95% CI - Low',
 'Motor vehicle crash deaths_95% CI - High',
 'Drug poisoning deaths_# Drug Poisoning Deaths',
 'Uninsured adults_# Uninsured',
 'Uninsured adults_% Uninsured',
 'Uninsured adults_95% CI - Low',
 'Uninsured adults_95% CI - High',
 'Uninsured children_# Uninsured',
 'Uninsured children_95% CI - Low',
 'Uninsured children_95% CI - High',
 'Health care costs_Costs',
 'Could not see doctor due to cost_Sample Size',
 "Could not see doctor due to cost_% Couldn't Access",
 'Could not see doctor due to cost_95% CI - Low',
 'Could not see doctor due to cost_95% CI - High',
 'Other primary care providers^_Other PCP Ratio',
 'Previous Other primary care providers data^_Other PCP Rate',
 'Previous Other primary care providers data^_Other PCP Ratio',
 'Median household income_95% CI - Low',
 'Median household income_95% CI - High',
 'Homicides_95% CI - Low',
 'Homicides_95% CI - High']
                     ,axis=1)
        
        df3.columns = df3.columns.map('_'.join)
        df3.rename(columns = {
             'Unnamed: 0_level_0_FIPS':'FIPS',
             'Unnamed: 1_level_0_State':'State',
             'Unnamed: 2_level_0_County':'County/Borough/Parish',
             'Unnamed: 2_level_0_Borough':'County/Borough/Parish',
             'Unnamed: 2_level_0_Parish':'County/Borough/Parish'}, 
                   inplace = True)
        df3 = df3.drop(['Health Outcomes_Z-Score','Health Factors_Z-Score','Health Factors_Rank'], axis=1)
        
        # Join two dataframes
        joined_df = pd.merge(df, df2, on='FIPS', how='outer')
        joined_df = joined_df.drop(['State_y','County/Borough/Parish_y'],axis=1)
        joined_df.rename(columns = {'State_x':'State','County/Borough/Parish_x':'County/Borough/Parish'},inplace = True)
        
        joined_df = pd.merge(joined_df, df3, on='FIPS', how='outer')
        joined_df = joined_df.drop(['State_y','County/Borough/Parish_y'],axis=1)
        joined_df.rename(columns = {'State_x':'State','County/Borough/Parish_x':'County/Borough/Parish'},inplace = True)
        
        joined_df['Year'] = '2015'
        
        # append to list
        appended_data2015.append(joined_df)
        
#convert to dataframe
appended_data2015 = pd.concat(appended_data2015)

2015 County Health Rankings Alabama Data - v3.xls
2015 County Health Rankings Alaska Data - v3.xls
2015 County Health Rankings Arizona Data - v3.xls
2015 County Health Rankings Arkansas Data - v3.xls
2015 County Health Rankings California Data - v3.xls
2015 County Health Rankings Colorado Data - v3.xls
2015 County Health Rankings Connecticut Data - v3.xls
2015 County Health Rankings Delaware Data - v3.xls
2015 County Health Rankings District of Columbia Data - v3.xls
2015 County Health Rankings Florida Data - v3.xls
2015 County Health Rankings Georgia Data - v3.xls
2015 County Health Rankings Hawaii Data - v3.xls
2015 County Health Rankings Idaho Data - v3.xls
2015 County Health Rankings Illinois Data - v3.xls
2015 County Health Rankings Indiana Data - v3.xls
2015 County Health Rankings Iowa Data - v3.xls
2015 County Health Rankings Kansas Data - v3.xls
2015 County Health Rankings Kentucky Data - v3.xls
2015 County Health Rankings Louisiana Data - v3.xls
2015 County Health Rankings Mai

In [43]:
appended_data2015.head(20)

,FIPS,State,County/Borough/Parish,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,%_Low_Birthweight_Births,%_Adult_Smokers,%_Adult_Obesity,"Healthy_Food_Access(0=Worst,10=Best)",%_Adults_Physical_Inactivity,...,%_Pop_w/o_Adequate_Access_to_Food,Motor_Vehicle_Deaths_per_100K,Drug_Overdose_Deaths_per_100K,%_Children(under19)_w/o_Health_Insurance,Other_Primary_Care_Providers(non_physician)_per_100K,Median_Household_Income,%_Children_Enrolled_in_Free/Reduced_Lunch,#_Deaths_due_to_Homicide_per_100K,Health Outcomes_Rank,Year
0,1000.0,Alabama,NaN,9507.90,20.5,10.296385,21.9,33.22678,6.7,29.371208,...,18.569592,20.890552,11.472927,4.337595,46.1756,42882.0,50.026658,9.1,NaN,2015
1,1001.0,Alabama,Autauga,8405.21,22.8,9.085079,21.7,31.30000,7.2,27.900000,...,13.500000,21.248509,8.233797,3.710341,16.2908,51868.0,38.340967,4.6,10,2015
2,1003.0,Alabama,Baldwin,7456.89,12.7,8.670182,20.6,25.00000,7.6,25.200000,...,13.500000,16.566709,15.292347,5.356782,29.1501,47539.0,34.407666,4.3,2,2015
3,1005.0,Alabama,Barbour,8900.51,23.4,11.877875,25.1,38.40000,5.0,32.300000,...,22.700000,18.666390,NaN,4.546211,11.0799,30981.0,69.677926,NaN,22,2015
4,1007.0,Alabama,Bibb,10729.22,17.9,12.787437,25.9,37.30000,7.6,33.200000,...,15.400000,25.873874,18.932103,4.205701,8.88415,39781.0,54.648206,7.8,45,2015
5,1009.0,Alabama,Blount,8668.52,24.2,7.690755,21.3,30.90000,8.5,29.800000,...,11.800000,25.567370,18.298216,5.877066,6.91181,44392.0,44.229797,4.3,14,2015
6,1011.0,Alabama,Bullock,10743.52,23.0,13.632515,37.8,43.20000,3.8,32.800000,...,24.100000,54.191945,NaN,4.114335,56.3963,25937.0,85.998715,NaN,47,2015
7,1013.0,Alabama,Butler,10357.46,19.9,10.803758,30.5,38.40000,6.0,37.300000,...,22.100000,21.316244,8.939070,3.694479,34.5423,31041.0,69.352432,8.2,23,2015
8,1015.0,Alabama,Calhoun,10538.61,22.7,8.688799,24.4,31.50000,6.4,33.200000,...,17.200000,17.541701,12.059920,3.687423,34.2653,37898.0,52.990619,12.0,31,2015
9,1017.0,Alabama,Chambers,11863.82,38.5,11.880833,24.1,39.70000,5.7,35.300000,...,20.500000,28.220803,9.960283,3.858025,8.78169,33883.0,66.121936,9.7,60,2015


In [44]:
list(appended_data2015.columns.values)

['FIPS',
 'State',
 'County/Borough/Parish',
 'Yrs_Potential_Life_Lost_per100K(YPLL)',
 '%_Fair/Poor_Health',
 '%_Low_Birthweight_Births',
 '%_Adult_Smokers',
 '%_Adult_Obesity',
 'Healthy_Food_Access(0=Worst,10=Best)',
 '%_Adults_Physical_Inactivity',
 '%_Access_to_Excercise_Opportunities',
 '%_Adults_Excessive_Drinking',
 '%_Driving_Deaths_with_Alcohol_Involvement',
 'Chlamydia_Cases_per_100K',
 'Teen_Birth_Rate',
 '%_Ppl_Under65_w/o_Insurance',
 'PCP_per_100K',
 'Dentists_per_100K',
 'MHP_per_100K',
 'Preventable_Hospitalization_Rate(per100K_Medicare_Enrollees)',
 'Graduation_Rate',
 '%_Adults(25-44)_with_Some_Post-secondary_Ed',
 '%_Ppl(16+)_Unemployed_&_Looking_for_Work',
 '%_Children(under18)_Living_in_Poverty',
 '80th_Percentile_MHI',
 '20th_Percentile_MHI',
 '%_Children_Living_in_Single_Parent_Household',
 'Associations_per_10K',
 'Violent_Crimes_per_100K',
 'Injury_Death_Rate_per_100K',
 'Avg_Daily_Particulate_Matter_in_Micrograms_per_Cubic_Meter',
 'County_Affected_by_Water_V

In [45]:
appended_data2015.count()

FIPS                                                            3192
State                                                           3192
County/Borough/Parish                                           3141
Yrs_Potential_Life_Lost_per100K(YPLL)                           3050
%_Fair/Poor_Health                                              2792
%_Low_Birthweight_Births                                        3093
%_Adult_Smokers                                                 2762
%_Adult_Obesity                                                 3192
Healthy_Food_Access(0=Worst,10=Best)                            3192
%_Adults_Physical_Inactivity                                    3192
%_Access_to_Excercise_Opportunities                             3179
%_Adults_Excessive_Drinking                                     2276
%_Driving_Deaths_with_Alcohol_Involvement                       3168
Chlamydia_Cases_per_100K                                        3010
Teen_Birth_Rate                   

In [46]:
appended_data2015.isnull().any()

FIPS                                                             True
State                                                            True
County/Borough/Parish                                            True
Yrs_Potential_Life_Lost_per100K(YPLL)                            True
%_Fair/Poor_Health                                               True
%_Low_Birthweight_Births                                         True
%_Adult_Smokers                                                  True
%_Adult_Obesity                                                  True
Healthy_Food_Access(0=Worst,10=Best)                             True
%_Adults_Physical_Inactivity                                     True
%_Access_to_Excercise_Opportunities                              True
%_Adults_Excessive_Drinking                                      True
%_Driving_Deaths_with_Alcohol_Involvement                        True
Chlamydia_Cases_per_100K                                         True
Teen_Birth_Rate     

In [47]:
appended_data2015.loc[appended_data2015.isnull().any(axis=1)]

,FIPS,State,County/Borough/Parish,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,%_Low_Birthweight_Births,%_Adult_Smokers,%_Adult_Obesity,"Healthy_Food_Access(0=Worst,10=Best)",%_Adults_Physical_Inactivity,...,%_Pop_w/o_Adequate_Access_to_Food,Motor_Vehicle_Deaths_per_100K,Drug_Overdose_Deaths_per_100K,%_Children(under19)_w/o_Health_Insurance,Other_Primary_Care_Providers(non_physician)_per_100K,Median_Household_Income,%_Children_Enrolled_in_Free/Reduced_Lunch,#_Deaths_due_to_Homicide_per_100K,Health Outcomes_Rank,Year
0,1000.0,Alabama,NaN,9507.90,20.5,10.296385,21.9,33.226780,6.7,29.371208,...,18.569592,20.890552,11.472927,4.337595,46.1756,42882.0,50.026658,9.1,NaN,2015
3,1005.0,Alabama,Barbour,8900.51,23.4,11.877875,25.1,38.400000,5.0,32.300000,...,22.700000,18.666390,NaN,4.546211,11.0799,30981.0,69.677926,NaN,22,2015
4,1007.0,Alabama,Bibb,10729.22,17.9,12.787437,25.9,37.300000,7.6,33.200000,...,15.400000,25.873874,18.932103,4.205701,8.88415,39781.0,54.648206,7.8,45,2015
6,1011.0,Alabama,Bullock,10743.52,23.0,13.632515,37.8,43.200000,3.8,32.800000,...,24.100000,54.191945,NaN,4.114335,56.3963,25937.0,85.998715,NaN,47,2015
7,1013.0,Alabama,Butler,10357.46,19.9,10.803758,30.5,38.400000,6.0,37.300000,...,22.100000,21.316244,8.939070,3.694479,34.5423,31041.0,69.352432,8.2,23,2015
9,1017.0,Alabama,Chambers,11863.82,38.5,11.880833,24.1,39.700000,5.7,35.300000,...,20.500000,28.220803,9.960283,3.858025,8.78169,33883.0,66.121936,9.7,60,2015
10,1019.0,Alabama,Cherokee,10513.33,24.2,8.947048,24.4,30.300000,7.9,29.800000,...,14.800000,24.366472,16.613503,4.463809,11.4491,37478.0,53.388794,NaN,42,2015
11,1021.0,Alabama,Chilton,10911.40,20.5,9.819441,20.1,37.000000,7.9,31.800000,...,14.200000,30.335104,18.794575,5.989420,NaN,41348.0,53.262695,6.2,35,2015
12,1023.0,Alabama,Choctaw,13411.64,31.6,11.111111,19.9,40.200000,5.9,34.800000,...,18.500000,45.994399,NaN,4.771115,37.2412,33581.0,71.061453,NaN,63,2015
13,1025.0,Alabama,Clarke,10709.34,21.8,12.620424,13.7,35.200000,5.1,31.000000,...,22.300000,29.088754,NaN,4.524306,27.7701,34593.0,60.621327,7.5,33,2015


In [48]:
appended_data2015 = appended_data2015.fillna(value=0)
appended_data2015.head(20)

,FIPS,State,County/Borough/Parish,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,%_Low_Birthweight_Births,%_Adult_Smokers,%_Adult_Obesity,"Healthy_Food_Access(0=Worst,10=Best)",%_Adults_Physical_Inactivity,...,%_Pop_w/o_Adequate_Access_to_Food,Motor_Vehicle_Deaths_per_100K,Drug_Overdose_Deaths_per_100K,%_Children(under19)_w/o_Health_Insurance,Other_Primary_Care_Providers(non_physician)_per_100K,Median_Household_Income,%_Children_Enrolled_in_Free/Reduced_Lunch,#_Deaths_due_to_Homicide_per_100K,Health Outcomes_Rank,Year
0,1000.0,Alabama,0,9507.90,20.5,10.296385,21.9,33.22678,6.7,29.371208,...,18.569592,20.890552,11.472927,4.337595,46.1756,42882.0,50.026658,9.1,0,2015
1,1001.0,Alabama,Autauga,8405.21,22.8,9.085079,21.7,31.30000,7.2,27.900000,...,13.500000,21.248509,8.233797,3.710341,16.2908,51868.0,38.340967,4.6,10,2015
2,1003.0,Alabama,Baldwin,7456.89,12.7,8.670182,20.6,25.00000,7.6,25.200000,...,13.500000,16.566709,15.292347,5.356782,29.1501,47539.0,34.407666,4.3,2,2015
3,1005.0,Alabama,Barbour,8900.51,23.4,11.877875,25.1,38.40000,5.0,32.300000,...,22.700000,18.666390,0.000000,4.546211,11.0799,30981.0,69.677926,0.0,22,2015
4,1007.0,Alabama,Bibb,10729.22,17.9,12.787437,25.9,37.30000,7.6,33.200000,...,15.400000,25.873874,18.932103,4.205701,8.88415,39781.0,54.648206,7.8,45,2015
5,1009.0,Alabama,Blount,8668.52,24.2,7.690755,21.3,30.90000,8.5,29.800000,...,11.800000,25.567370,18.298216,5.877066,6.91181,44392.0,44.229797,4.3,14,2015
6,1011.0,Alabama,Bullock,10743.52,23.0,13.632515,37.8,43.20000,3.8,32.800000,...,24.100000,54.191945,0.000000,4.114335,56.3963,25937.0,85.998715,0.0,47,2015
7,1013.0,Alabama,Butler,10357.46,19.9,10.803758,30.5,38.40000,6.0,37.300000,...,22.100000,21.316244,8.939070,3.694479,34.5423,31041.0,69.352432,8.2,23,2015
8,1015.0,Alabama,Calhoun,10538.61,22.7,8.688799,24.4,31.50000,6.4,33.200000,...,17.200000,17.541701,12.059920,3.687423,34.2653,37898.0,52.990619,12.0,31,2015
9,1017.0,Alabama,Chambers,11863.82,38.5,11.880833,24.1,39.70000,5.7,35.300000,...,20.500000,28.220803,9.960283,3.858025,8.78169,33883.0,66.121936,9.7,60,2015


### Due to metrics for additional measure data being in a different order than all other years, this code just reorders columns for 2015 DF to align

In [49]:
appended_data2015 = appended_data2015[['FIPS',
 'State',
 'County/Borough/Parish',
 'Yrs_Potential_Life_Lost_per100K(YPLL)',
 '%_Fair/Poor_Health',
 '%_Low_Birthweight_Births',
 '%_Adult_Smokers',
 '%_Adult_Obesity',
 'Healthy_Food_Access(0=Worst,10=Best)',
 '%_Adults_Physical_Inactivity',
 '%_Access_to_Excercise_Opportunities',
 '%_Adults_Excessive_Drinking',
 '%_Driving_Deaths_with_Alcohol_Involvement',
 'Chlamydia_Cases_per_100K',
 'Teen_Birth_Rate',
 '%_Ppl_Under65_w/o_Insurance',
 'PCP_per_100K',
 'Dentists_per_100K',
 'MHP_per_100K',
 'Preventable_Hospitalization_Rate(per100K_Medicare_Enrollees)',
 'Graduation_Rate',
 '%_Adults(25-44)_with_Some_Post-secondary_Ed',
 '%_Ppl(16+)_Unemployed_&_Looking_for_Work',
 '%_Children(under18)_Living_in_Poverty',
 '80th_Percentile_MHI',
 '20th_Percentile_MHI',
 '%_Children_Living_in_Single_Parent_Household',
 'Associations_per_10K',
 'Violent_Crimes_per_100K',
 'Injury_Death_Rate_per_100K',
 'Avg_Daily_Particulate_Matter_in_Micrograms_per_Cubic_Meter',
 'County_Affected_by_Water_Violation(1=Yes,0=No)',
 '%_Households_with_One_or_More Problems(overcrowding,high-housing-cost,lack-kitchen,lack-plumbing-facilities)',
 '%_Lone_Commuters_who_Commute_More_Than_30Min',
 'Premature_Deaths',
 'Premature(under75)_Age_Adjusted_Mortality_per_100K',
 'Child(under18)_Mortality_per_100K',
 'Infant(within1yr)_Mortality_per_1K_live_births',
 '%_Adults(20+)_Diagnosed_Diabetes',
 '#_Ppl(Age13+)_diagnosed_HIV_per_100K',
 '%_Pop_w/o_Adequate_Access_to_Food',
 'Drug_Overdose_Deaths_per_100K',
 'Motor_Vehicle_Deaths_per_100K',
 '%_Children(under19)_w/o_Health_Insurance',
 'Other_Primary_Care_Providers(non_physician)_per_100K',
 'Median_Household_Income',
 '%_Children_Enrolled_in_Free/Reduced_Lunch',
 '#_Deaths_due_to_Homicide_per_100K',
 'Resident_Population',
 '%_under_18_years_of_age',
 '%_Age_65_and_over',
 '#_Non-Hispanic_Black_or_African_American',
 '%_Non-Hispanic_Black_or_African_American',
 '#_American_Indian_or_Alaskan_Native',
 '%_American_Indian_or_Alaskan_Native',
 '#_Asian',
 '%_Asian',
 '#_Native_Hawaiian/Other_Pacific_Islander',
 '%_Native_Hawaiian/Other_Pacific_Islander',
 '#_Hispanic',
 '%_Hispanic',
 '#_Non-Hispanic_White',
 '%_Non-Hispanic_White',
 '#_Not_Proficient_in_English',
 '%_Not_Proficient_in_English',
 '%_Female',
 '#_Pop_Living_in_Rural_Area',
 '%_Rural',
 'Health Outcomes_Rank',
 'Year']]

In [50]:
appended_data2015.head()

,FIPS,State,County/Borough/Parish,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,%_Low_Birthweight_Births,%_Adult_Smokers,%_Adult_Obesity,"Healthy_Food_Access(0=Worst,10=Best)",%_Adults_Physical_Inactivity,...,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Health Outcomes_Rank,Year
0,1000.0,Alabama,0,9507.90,20.5,10.296385,21.9,33.22678,6.7,29.371208,...,4.096615,3210595.0,66.420762,63069.0,1.402359,51.479481,1957932.0,40.963183,0,2015
1,1001.0,Alabama,Autauga,8405.21,22.8,9.085079,21.7,31.30000,7.2,27.900000,...,2.698838,41934.0,75.904138,245.0,0.476487,51.496941,22921.0,42.002162,10,2015
2,1003.0,Alabama,Baldwin,7456.89,12.7,8.670182,20.6,25.00000,7.6,25.200000,...,4.576557,162536.0,83.121612,2743.0,1.560030,51.167536,77060.0,42.279099,2,2015
3,1005.0,Alabama,Barbour,8900.51,23.4,11.877875,25.1,38.40000,5.0,32.300000,...,4.834540,12545.0,46.332545,574.0,2.230859,46.513518,18613.0,67.789635,22,2015
4,1007.0,Alabama,Bibb,10729.22,17.9,12.787437,25.9,37.30000,7.6,33.200000,...,2.070007,16786.0,74.564677,92.0,0.428625,45.993248,15663.0,68.352607,45,2015


### Water Violation data was included in 2015 data as a % of population impacted versus as a Yes/No indicator.  This code applies a map to convert any counties with a % of population impacted > 0 to Yes and all others as No to align with other data for use in analysis.

In [51]:
filter_method = lambda x: 'Yes' if x > 0 else 'No'
appended_data2015['County_Affected_by_Water_Violation(1=Yes,0=No)'] = appended_data2015['County_Affected_by_Water_Violation(1=Yes,0=No)'].apply(filter_method)
appended_data2015['County_Affected_by_Water_Violation(1=Yes,0=No)'].value_counts()

Yes    1993
No     1403
Name: County_Affected_by_Water_Violation(1=Yes,0=No), dtype: int64

In [52]:
# appended_data2015.to_csv(os.path.join("Cleaned_Files","2015_County_Health_Ranking_Data.csv"),index=False)

### Concatenate the annual dataframe sets in descending order 2010 > 2019 > 2018 > 2017> 2016 > 2015

In [53]:
full_data = pd.concat([appended_data2020, appended_data2019], ignore_index=True, sort=False)
full_data.head()

,FIPS,State,County/Borough/Parish,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,%_Low_Birthweight_Births,%_Adult_Smokers,%_Adult_Obesity,"Healthy_Food_Access(0=Worst,10=Best)",%_Adults_Physical_Inactivity,...,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Health Outcomes_Rank,Year
0,1000,Alabama,0,9942.794666,22.028703,10.254871,20.927353,35.5,5.8,29.8,...,4.443264,3197324,65.413428,48517,1.061048,51.633032,1957932.0,40.963183,0,2020
1,1001,Alabama,Autauga,8128.591190,20.882987,8.619529,18.081557,33.3,7.2,34.7,...,2.965774,41316,74.308016,426,0.820225,51.448715,22921.0,42.002162,6,2020
2,1003,Alabama,Baldwin,7354.122530,17.509134,8.345003,17.489033,31.0,8.0,26.5,...,4.646779,181201,83.111337,1068,0.543517,51.538377,77060.0,42.279099,2,2020
3,1005,Alabama,Barbour,10253.573403,29.591802,11.474559,21.999985,41.7,5.6,23.5,...,4.276355,11356,45.641252,398,1.631683,47.216752,18613.0,67.789635,45,2020
4,1007,Alabama,Bibb,11977.539484,19.439724,10.308710,19.114200,37.6,7.8,33.5,...,2.625000,16708,74.589286,57,0.268210,46.781250,15663.0,68.352607,34,2020


In [54]:
full_data = pd.concat([full_data, appended_data2018], ignore_index=True, sort=False)
full_data.head()

,FIPS,State,County/Borough/Parish,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,%_Low_Birthweight_Births,%_Adult_Smokers,%_Adult_Obesity,"Healthy_Food_Access(0=Worst,10=Best)",%_Adults_Physical_Inactivity,...,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Health Outcomes_Rank,Year
0,1000.0,Alabama,0,9942.794666,22.028703,10.254871,20.927353,35.5,5.8,29.8,...,4.443264,3197324.0,65.413428,48517.0,1.061048,51.633032,1957932.0,40.963183,0,2020
1,1001.0,Alabama,Autauga,8128.591190,20.882987,8.619529,18.081557,33.3,7.2,34.7,...,2.965774,41316.0,74.308016,426.0,0.820225,51.448715,22921.0,42.002162,6,2020
2,1003.0,Alabama,Baldwin,7354.122530,17.509134,8.345003,17.489033,31.0,8.0,26.5,...,4.646779,181201.0,83.111337,1068.0,0.543517,51.538377,77060.0,42.279099,2,2020
3,1005.0,Alabama,Barbour,10253.573403,29.591802,11.474559,21.999985,41.7,5.6,23.5,...,4.276355,11356.0,45.641252,398.0,1.631683,47.216752,18613.0,67.789635,45,2020
4,1007.0,Alabama,Bibb,11977.539484,19.439724,10.308710,19.114200,37.6,7.8,33.5,...,2.625000,16708.0,74.589286,57.0,0.268210,46.781250,15663.0,68.352607,34,2020


In [55]:
full_data = pd.concat([full_data, appended_data2017], ignore_index=True, sort=False)
full_data.head()

,FIPS,State,County/Borough/Parish,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,%_Low_Birthweight_Births,%_Adult_Smokers,%_Adult_Obesity,"Healthy_Food_Access(0=Worst,10=Best)",%_Adults_Physical_Inactivity,...,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Health Outcomes_Rank,Year
0,1000.0,Alabama,0,9942.794666,22.028703,10.254871,20.927353,35.5,5.8,29.8,...,4.443264,3197324.0,65.413428,48517.0,1.061048,51.633032,1957932.0,40.963183,0,2020
1,1001.0,Alabama,Autauga,8128.591190,20.882987,8.619529,18.081557,33.3,7.2,34.7,...,2.965774,41316.0,74.308016,426.0,0.820225,51.448715,22921.0,42.002162,6,2020
2,1003.0,Alabama,Baldwin,7354.122530,17.509134,8.345003,17.489033,31.0,8.0,26.5,...,4.646779,181201.0,83.111337,1068.0,0.543517,51.538377,77060.0,42.279099,2,2020
3,1005.0,Alabama,Barbour,10253.573403,29.591802,11.474559,21.999985,41.7,5.6,23.5,...,4.276355,11356.0,45.641252,398.0,1.631683,47.216752,18613.0,67.789635,45,2020
4,1007.0,Alabama,Bibb,11977.539484,19.439724,10.308710,19.114200,37.6,7.8,33.5,...,2.625000,16708.0,74.589286,57.0,0.268210,46.781250,15663.0,68.352607,34,2020


In [56]:
full_data = pd.concat([full_data, appended_data2016], ignore_index=True, sort=False)
full_data.head()

,FIPS,State,County/Borough/Parish,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,%_Low_Birthweight_Births,%_Adult_Smokers,%_Adult_Obesity,"Healthy_Food_Access(0=Worst,10=Best)",%_Adults_Physical_Inactivity,...,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Health Outcomes_Rank,Year
0,1000.0,Alabama,0,9942.794666,22.028703,10.254871,20.927353,35.5,5.8,29.8,...,4.443264,3197324.0,65.413428,48517.0,1.061048,51.633032,1957932.0,40.963183,0,2020
1,1001.0,Alabama,Autauga,8128.591190,20.882987,8.619529,18.081557,33.3,7.2,34.7,...,2.965774,41316.0,74.308016,426.0,0.820225,51.448715,22921.0,42.002162,6,2020
2,1003.0,Alabama,Baldwin,7354.122530,17.509134,8.345003,17.489033,31.0,8.0,26.5,...,4.646779,181201.0,83.111337,1068.0,0.543517,51.538377,77060.0,42.279099,2,2020
3,1005.0,Alabama,Barbour,10253.573403,29.591802,11.474559,21.999985,41.7,5.6,23.5,...,4.276355,11356.0,45.641252,398.0,1.631683,47.216752,18613.0,67.789635,45,2020
4,1007.0,Alabama,Bibb,11977.539484,19.439724,10.308710,19.114200,37.6,7.8,33.5,...,2.625000,16708.0,74.589286,57.0,0.268210,46.781250,15663.0,68.352607,34,2020


In [57]:
full_data = pd.concat([full_data, appended_data2015], ignore_index=True, sort=False)
full_data.head()

,FIPS,State,County/Borough/Parish,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,%_Low_Birthweight_Births,%_Adult_Smokers,%_Adult_Obesity,"Healthy_Food_Access(0=Worst,10=Best)",%_Adults_Physical_Inactivity,...,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Health Outcomes_Rank,Year
0,1000.0,Alabama,0,9942.794666,22.028703,10.254871,20.927353,35.5,5.8,29.8,...,4.443264,3197324.0,65.413428,48517.0,1.061048,51.633032,1957932.0,40.963183,0,2020
1,1001.0,Alabama,Autauga,8128.591190,20.882987,8.619529,18.081557,33.3,7.2,34.7,...,2.965774,41316.0,74.308016,426.0,0.820225,51.448715,22921.0,42.002162,6,2020
2,1003.0,Alabama,Baldwin,7354.122530,17.509134,8.345003,17.489033,31.0,8.0,26.5,...,4.646779,181201.0,83.111337,1068.0,0.543517,51.538377,77060.0,42.279099,2,2020
3,1005.0,Alabama,Barbour,10253.573403,29.591802,11.474559,21.999985,41.7,5.6,23.5,...,4.276355,11356.0,45.641252,398.0,1.631683,47.216752,18613.0,67.789635,45,2020
4,1007.0,Alabama,Bibb,11977.539484,19.439724,10.308710,19.114200,37.6,7.8,33.5,...,2.625000,16708.0,74.589286,57.0,0.268210,46.781250,15663.0,68.352607,34,2020


### Confirm no nulls in all year full county data

In [58]:
full_data.isnull().any()

FIPS                                                            False
State                                                           False
County/Borough/Parish                                           False
Yrs_Potential_Life_Lost_per100K(YPLL)                           False
%_Fair/Poor_Health                                              False
%_Low_Birthweight_Births                                        False
%_Adult_Smokers                                                 False
%_Adult_Obesity                                                 False
Healthy_Food_Access(0=Worst,10=Best)                            False
%_Adults_Physical_Inactivity                                    False
%_Access_to_Excercise_Opportunities                             False
%_Adults_Excessive_Drinking                                     False
%_Driving_Deaths_with_Alcohol_Involvement                       False
Chlamydia_Cases_per_100K                                        False
Teen_Birth_Rate     

### Due to some issues with D3, had to make changes to naming styles to remove symbols like % for chloropleth.
Was easier to insert here in the code versus redo in each individual year loop.  Please note for geojson there is still some slight renaming that had to take place even with this step but not as much as before.

In [59]:
list(full_data.columns.values)

['FIPS',
 'State',
 'County/Borough/Parish',
 'Yrs_Potential_Life_Lost_per100K(YPLL)',
 '%_Fair/Poor_Health',
 '%_Low_Birthweight_Births',
 '%_Adult_Smokers',
 '%_Adult_Obesity',
 'Healthy_Food_Access(0=Worst,10=Best)',
 '%_Adults_Physical_Inactivity',
 '%_Access_to_Excercise_Opportunities',
 '%_Adults_Excessive_Drinking',
 '%_Driving_Deaths_with_Alcohol_Involvement',
 'Chlamydia_Cases_per_100K',
 'Teen_Birth_Rate',
 '%_Ppl_Under65_w/o_Insurance',
 'PCP_per_100K',
 'Dentists_per_100K',
 'MHP_per_100K',
 'Preventable_Hospitalization_Rate(per100K_Medicare_Enrollees)',
 'Graduation_Rate',
 '%_Adults(25-44)_with_Some_Post-secondary_Ed',
 '%_Ppl(16+)_Unemployed_&_Looking_for_Work',
 '%_Children(under18)_Living_in_Poverty',
 '80th_Percentile_MHI',
 '20th_Percentile_MHI',
 '%_Children_Living_in_Single_Parent_Household',
 'Associations_per_10K',
 'Violent_Crimes_per_100K',
 'Injury_Death_Rate_per_100K',
 'Avg_Daily_Particulate_Matter_in_Micrograms_per_Cubic_Meter',
 'County_Affected_by_Water_V

In [60]:
full_data.rename(columns = {'FIPS':'FIPS',
 'State': 'State',
 'County/Borough/Parish': 'County/Borough/Parish',
 'Yrs_Potential_Life_Lost_per100K(YPLL)': 'Yrs_Potential_Life_Lost_per_100K',
 '%_Fair/Poor_Health': 'Percent_Fair_Poor_Health',
 '%_Low_Birthweight_Births': 'Percent_Low_Birthweight_Births',
 '%_Adult_Smokers': 'Percent_Adult_Smokers',
 '%_Adult_Obesity': 'Percent_Adult_Obesity',
 'Healthy_Food_Access(0=Worst,10=Best)': 'Healthy_Food_Access_Index',
 '%_Adults_Physical_Inactivity': 'Percent_Adults_Physical_Inactivity',
 '%_Access_to_Excercise_Opportunities': 'Percent_Access_to_Excercise_Opportunities',
 '%_Adults_Excessive_Drinking': 'Percent_Adults_Excessive_Drinking',
 '%_Driving_Deaths_with_Alcohol_Involvement': 'Percent_Driving_Deaths_with_Alcohol_Involvement',
 'Chlamydia_Cases_per_100K': 'Chlamydia_Cases_per_100K',
 'Teen_Birth_Rate': 'Teen_Birth_Rate',
 '%_Ppl_Under65_w/o_Insurance': 'Percent_Ppl_Under65_w/o_Insurance',
 'PCP_per_100K': 'PCP_per_100K',
 'Dentists_per_100K': 'Dentists_per_100K',
 'MHP_per_100K': 'MHP_per_100K',
 'Preventable_Hospitalization_Rate(per100K_Medicare_Enrollees)': 'Preventable_Hospitalization_Rate_per_100K_Medicare',
 'Graduation_Rate': 'Graduation_Rate',
 '%_Adults(25-44)_with_Some_Post-secondary_Ed': 'Percent_Adults_with_Some_Post-secondary_Ed',
 '%_Ppl(16+)_Unemployed_&_Looking_for_Work': 'Percent_Unemployed_&_Looking_for_Work',
 '%_Children(under18)_Living_in_Poverty': 'Percent_Children_Living_in_Poverty',
 '80th_Percentile_MHI': '80th_Percentile_MHI',
 '20th_Percentile_MHI': '20th_Percentile_MHI',
 '%_Children_Living_in_Single_Parent_Household': 'Percent_Children_Living_in_Single_Parent_Household',
 'Associations_per_10K': 'Associations_per_10K',
 'Violent_Crimes_per_100K': 'Violent_Crimes_per_100K',
 'Injury_Death_Rate_per_100K': 'Injury_Death_Rate_per_100K',
 'Avg_Daily_Particulate_Matter_in_Micrograms_per_Cubic_Meter': 'Avg_Daily_Particulate_Matter_in_Micrograms_per_Cubic_Meter',
 'County_Affected_by_Water_Violation(1=Yes,0=No)': 'County_Affected_by_Water_Violation',
 '%_Households_with_One_or_More Problems(overcrowding,high-housing-cost,lack-kitchen,lack-plumbing-facilities)': 'Percent_Households_with_One_or_More_Severe_Problems',
 '%_Lone_Commuters_who_Commute_More_Than_30Min': 'Percent_Lone_Commuters_who_Commute_More_Than_30Min',
 'Premature_Deaths': 'Premature_Deaths',
 'Premature(under75)_Age_Adjusted_Mortality_per_100K': 'Premature_Age_Adjusted_Mortality_per_100K',
 'Child(under18)_Mortality_per_100K': 'Child_Mortality_per_100K',
 'Infant(within1yr)_Mortality_per_1K_live_births': 'Infant_Mortality_per_1K_live_births',
 '%_Adults(20+)_Diagnosed_Diabetes': 'Percent_Adults_Diagnosed_Diabetes',
 '#_Ppl(Age13+)_diagnosed_HIV_per_100K': 'HIV_per_100K',
 '%_Pop_w/o_Adequate_Access_to_Food': 'Percent_Pop_w/o_Adequate_Access_to_Food',
 'Drug_Overdose_Deaths_per_100K': 'Drug_Overdose_Deaths_per_100K',
 'Motor_Vehicle_Deaths_per_100K': 'Motor_Vehicle_Deaths_per_100K',
 '%_Children(under19)_w/o_Health_Insurance': 'Percent_Children_w/o_Health_Insurance',
 'Other_Primary_Care_Providers(non_physician)_per_100K': 'Other_Primary_Care_Providers_per_100K',
 'Median_Household_Income': 'Median_Household_Income',
 '%_Children_Enrolled_in_Free/Reduced_Lunch': 'Percent_Children_Enrolled_in_Reduced_Lunch',
 '#_Deaths_due_to_Homicide_per_100K': 'Homicide_per_100K',
 'Resident_Population': 'Resident_Population',
 '%_under_18_years_of_age': 'Percent_under_18_years_of_age',
 '%_Age_65_and_over': 'Percent_Age_65_and_over',
 '#_Non-Hispanic_Black_or_African_American': 'Number_Non-Hispanic_Black_or_African_American',
 '%_Non-Hispanic_Black_or_African_American': 'Percent_Non-Hispanic_Black_or_African_American',
 '#_American_Indian_or_Alaskan_Native': 'Number_American_Indian_or_Alaskan_Native',
 '%_American_Indian_or_Alaskan_Native': 'Percent_American_Indian_or_Alaskan_Native',
 '#_Asian': 'Number_Asian',
 '%_Asian': 'Percent_Asian',
 '#_Native_Hawaiian/Other_Pacific_Islander': 'Number_Native_Hawaiian_Other_Pacific_Islander',
 '%_Native_Hawaiian/Other_Pacific_Islander': 'Percent_Native_Hawaiian_Other_Pacific_Islander',
 '#_Hispanic': 'Number_Hispanic',
 '%_Hispanic': 'Percent_Hispanic',
 '#_Non-Hispanic_White': 'Number_Non-Hispanic_White',
 '%_Non-Hispanic_White': 'Percent_Non-Hispanic_White',
 '#_Not_Proficient_in_English': 'Number_Not_Proficient_in_English',
 '%_Not_Proficient_in_English': 'Percent_Not_Proficient_in_English',
 '%_Female': 'Percent_Female',
 '#_Pop_Living_in_Rural_Area': 'Number_Pop_Living_in_Rural_Area',
 '%_Rural': 'Percent_Rural',
 'Year': 'Year',
 'Health Outcomes_Rank':'Health_Outcomes_Rank'},
                  inplace = True)

### Reordered columns to bring rank and year to front of dataframe with other Key County identifiers

In [61]:
cols_to_move = ['FIPS','State','County/Borough/Parish', 'Year','Health_Outcomes_Rank']
full_data_df = full_data[ cols_to_move + [ col for col in full_data.columns if col not in cols_to_move ] ]
full_data_df

,FIPS,State,County/Borough/Parish,Year,Health_Outcomes_Rank,Yrs_Potential_Life_Lost_per_100K,Percent_Fair_Poor_Health,Percent_Low_Birthweight_Births,Percent_Adult_Smokers,Percent_Adult_Obesity,...,Percent_Native_Hawaiian_Other_Pacific_Islander,Number_Hispanic,Percent_Hispanic,Number_Non-Hispanic_White,Percent_Non-Hispanic_White,Number_Not_Proficient_in_English,Percent_Not_Proficient_in_English,Percent_Female,Number_Pop_Living_in_Rural_Area,Percent_Rural
0,1000.0,Alabama,0,2020,0,9942.794666,22.028703,10.254871,20.927353,35.500000,...,0.106529,217181.0,4.443264,3197324.0,65.413428,48517.0,1.061048,51.633032,1957932.0,40.963183
1,1001.0,Alabama,Autauga,2020,6,8128.591190,20.882987,8.619529,18.081557,33.300000,...,0.111509,1649.0,2.965774,41316.0,74.308016,426.0,0.820225,51.448715,22921.0,42.002162
2,1003.0,Alabama,Baldwin,2020,2,7354.122530,17.509134,8.345003,17.489033,31.000000,...,0.066966,10131.0,4.646779,181201.0,83.111337,1068.0,0.543517,51.538377,77060.0,42.279099
3,1005.0,Alabama,Barbour,2020,45,10253.573403,29.591802,11.474559,21.999985,41.700000,...,0.184880,1064.0,4.276355,11356.0,45.641252,398.0,1.631683,47.216752,18613.0,67.789635
4,1007.0,Alabama,Bibb,2020,34,11977.539484,19.439724,10.308710,19.114200,37.600000,...,0.116071,588.0,2.625000,16708.0,74.589286,57.0,0.268210,46.781250,15663.0,68.352607
5,1009.0,Alabama,Blount,2020,24,11335.071134,21.745293,7.604563,19.208672,33.800000,...,0.121024,5536.0,9.571231,50255.0,86.886238,934.0,1.724520,50.726141,51562.0,89.951502
6,1011.0,Alabama,Bullock,2020,56,11679.558981,30.999102,15.688488,22.894664,37.200000,...,0.749655,807.0,7.960150,2151.0,21.217203,43.0,0.440754,45.482344,5607.0,51.374382
7,1013.0,Alabama,Butler,2020,64,14359.939103,27.910673,12.680288,21.765335,43.300000,...,0.050813,297.0,1.509146,10087.0,51.255081,93.0,0.494155,53.429878,14921.0,71.232157
8,1015.0,Alabama,Calhoun,2020,42,12078.616145,23.107733,9.161374,20.612560,38.500000,...,0.110258,4469.0,3.910673,82308.0,72.024992,1076.0,0.991376,51.946586,39955.0,33.696826
9,1017.0,Alabama,Chambers,2020,44,11112.536614,23.995474,12.213740,19.392927,40.100000,...,0.044623,861.0,2.561357,18566.0,55.231296,36.0,0.112938,52.125539,16816.0,49.148034


### Dropped commentary rows picked up from some states who added footnotes to certain columns.  The easiest way to identify was that those impacted rows reflected a FIPS of 0.

In [62]:
full_data_df = full_data_df[full_data_df.FIPS != 0].copy(deep=False)
full_data_df

,FIPS,State,County/Borough/Parish,Year,Health_Outcomes_Rank,Yrs_Potential_Life_Lost_per_100K,Percent_Fair_Poor_Health,Percent_Low_Birthweight_Births,Percent_Adult_Smokers,Percent_Adult_Obesity,...,Percent_Native_Hawaiian_Other_Pacific_Islander,Number_Hispanic,Percent_Hispanic,Number_Non-Hispanic_White,Percent_Non-Hispanic_White,Number_Not_Proficient_in_English,Percent_Not_Proficient_in_English,Percent_Female,Number_Pop_Living_in_Rural_Area,Percent_Rural
0,1000.0,Alabama,0,2020,0,9942.794666,22.028703,10.254871,20.927353,35.500000,...,0.106529,217181.0,4.443264,3197324.0,65.413428,48517.0,1.061048,51.633032,1957932.0,40.963183
1,1001.0,Alabama,Autauga,2020,6,8128.591190,20.882987,8.619529,18.081557,33.300000,...,0.111509,1649.0,2.965774,41316.0,74.308016,426.0,0.820225,51.448715,22921.0,42.002162
2,1003.0,Alabama,Baldwin,2020,2,7354.122530,17.509134,8.345003,17.489033,31.000000,...,0.066966,10131.0,4.646779,181201.0,83.111337,1068.0,0.543517,51.538377,77060.0,42.279099
3,1005.0,Alabama,Barbour,2020,45,10253.573403,29.591802,11.474559,21.999985,41.700000,...,0.184880,1064.0,4.276355,11356.0,45.641252,398.0,1.631683,47.216752,18613.0,67.789635
4,1007.0,Alabama,Bibb,2020,34,11977.539484,19.439724,10.308710,19.114200,37.600000,...,0.116071,588.0,2.625000,16708.0,74.589286,57.0,0.268210,46.781250,15663.0,68.352607
5,1009.0,Alabama,Blount,2020,24,11335.071134,21.745293,7.604563,19.208672,33.800000,...,0.121024,5536.0,9.571231,50255.0,86.886238,934.0,1.724520,50.726141,51562.0,89.951502
6,1011.0,Alabama,Bullock,2020,56,11679.558981,30.999102,15.688488,22.894664,37.200000,...,0.749655,807.0,7.960150,2151.0,21.217203,43.0,0.440754,45.482344,5607.0,51.374382
7,1013.0,Alabama,Butler,2020,64,14359.939103,27.910673,12.680288,21.765335,43.300000,...,0.050813,297.0,1.509146,10087.0,51.255081,93.0,0.494155,53.429878,14921.0,71.232157
8,1015.0,Alabama,Calhoun,2020,42,12078.616145,23.107733,9.161374,20.612560,38.500000,...,0.110258,4469.0,3.910673,82308.0,72.024992,1076.0,0.991376,51.946586,39955.0,33.696826
9,1017.0,Alabama,Chambers,2020,44,11112.536614,23.995474,12.213740,19.392927,40.100000,...,0.044623,861.0,2.561357,18566.0,55.231296,36.0,0.112938,52.125539,16816.0,49.148034


In [63]:
full_data_df.head()

,FIPS,State,County/Borough/Parish,Year,Health_Outcomes_Rank,Yrs_Potential_Life_Lost_per_100K,Percent_Fair_Poor_Health,Percent_Low_Birthweight_Births,Percent_Adult_Smokers,Percent_Adult_Obesity,...,Percent_Native_Hawaiian_Other_Pacific_Islander,Number_Hispanic,Percent_Hispanic,Number_Non-Hispanic_White,Percent_Non-Hispanic_White,Number_Not_Proficient_in_English,Percent_Not_Proficient_in_English,Percent_Female,Number_Pop_Living_in_Rural_Area,Percent_Rural
0,1000.0,Alabama,0,2020,0,9942.794666,22.028703,10.254871,20.927353,35.5,...,0.106529,217181.0,4.443264,3197324.0,65.413428,48517.0,1.061048,51.633032,1957932.0,40.963183
1,1001.0,Alabama,Autauga,2020,6,8128.591190,20.882987,8.619529,18.081557,33.3,...,0.111509,1649.0,2.965774,41316.0,74.308016,426.0,0.820225,51.448715,22921.0,42.002162
2,1003.0,Alabama,Baldwin,2020,2,7354.122530,17.509134,8.345003,17.489033,31.0,...,0.066966,10131.0,4.646779,181201.0,83.111337,1068.0,0.543517,51.538377,77060.0,42.279099
3,1005.0,Alabama,Barbour,2020,45,10253.573403,29.591802,11.474559,21.999985,41.7,...,0.184880,1064.0,4.276355,11356.0,45.641252,398.0,1.631683,47.216752,18613.0,67.789635
4,1007.0,Alabama,Bibb,2020,34,11977.539484,19.439724,10.308710,19.114200,37.6,...,0.116071,588.0,2.625000,16708.0,74.589286,57.0,0.268210,46.781250,15663.0,68.352607


### Convert County Affected by Water Violation metric to binary Yes = 1 and No = 0 to better enable use in our analyses

In [64]:
full_data_df['County_Affected_by_Water_Violation'].value_counts()

No     9780
Yes    8867
0       503
Name: County_Affected_by_Water_Violation, dtype: int64

In [65]:
filter_method2 = lambda x: '1' if x == 'Yes' else '0'
full_data_df['County_Affected_by_Water_Violation'] = full_data_df['County_Affected_by_Water_Violation'].apply(filter_method2)
full_data_df['County_Affected_by_Water_Violation'].value_counts()

0    10283
1     8867
Name: County_Affected_by_Water_Violation, dtype: int64

### Apply a filter to identify columns that are not of a float data type to ensure data types are properly aligned

In [66]:
msk = full_data_df.dtypes != np.float64
full_data_df.loc[:, msk]

,State,County/Borough/Parish,Year,Health_Outcomes_Rank,County_Affected_by_Water_Violation,Other_Primary_Care_Providers_per_100K
0,Alabama,0,2020,0,0,86.0293
1,Alabama,Autauga,2020,6,0,39.5676
2,Alabama,Baldwin,2020,2,0,55.9577
3,Alabama,Barbour,2020,45,0,52.2487
4,Alabama,Bibb,2020,34,0,111.607
5,Alabama,Blount,2020,24,0,22.4758
6,Alabama,Bullock,2020,56,0,88.7749
7,Alabama,Butler,2020,64,1,71.1382
8,Alabama,Calhoun,2020,42,0,56.0043
9,Alabama,Chambers,2020,44,0,14.8743


In [67]:
full_data_df.loc[:, msk].dtypes

State                                    object
County/Borough/Parish                    object
Year                                     object
Health_Outcomes_Rank                     object
County_Affected_by_Water_Violation       object
Other_Primary_Care_Providers_per_100K    object
dtype: object

### Correct Data Type for Other Primary Care Providers Column

In [68]:
full_data_df["Other_Primary_Care_Providers_per_100K"] = pd.to_numeric(full_data_df["Other_Primary_Care_Providers_per_100K"])

In [69]:
full_data_df["County_Affected_by_Water_Violation"] = pd.to_numeric(full_data_df["County_Affected_by_Water_Violation"])

### Convert FIPS identifier to String to ensure it does not get calculated as a metric in analysis

In [70]:
full_data_df['FIPS'] = full_data_df['FIPS'].astype(str)

In [71]:
# full_data_df['ColumnID'] = full_data_df['ColumnID'].astype(str)

### Export full county data file to csv based on all years and all counties ranked or not ranked

In [72]:
full_data_df.to_csv(os.path.join("Cleaned_Files","All_Years_County_Health_Ranking_Data.csv"),index=False)

### Remove metrics that may overstate analysis in machine learning models and drop unranked counties to create Training File of All US Counties in All Years

In [73]:
analysis_only_full_data_df = full_data_df.drop([
'Number_Non-Hispanic_Black_or_African_American',
'Number_American_Indian_or_Alaskan_Native',
'Number_Asian',
 'Number_Native_Hawaiian_Other_Pacific_Islander',
'Number_Hispanic',
'Number_Non-Hispanic_White',
'Number_Not_Proficient_in_English',
'Number_Pop_Living_in_Rural_Area']
                     ,axis=1)
analysis_only_full_data_df.head()

,FIPS,State,County/Borough/Parish,Year,Health_Outcomes_Rank,Yrs_Potential_Life_Lost_per_100K,Percent_Fair_Poor_Health,Percent_Low_Birthweight_Births,Percent_Adult_Smokers,Percent_Adult_Obesity,...,Percent_Age_65_and_over,Percent_Non-Hispanic_Black_or_African_American,Percent_American_Indian_or_Alaskan_Native,Percent_Asian,Percent_Native_Hawaiian_Other_Pacific_Islander,Percent_Hispanic,Percent_Non-Hispanic_White,Percent_Not_Proficient_in_English,Percent_Female,Percent_Rural
0,1000.0,Alabama,0,2020,0,9942.794666,22.028703,10.254871,20.927353,35.5,...,16.917263,26.497733,0.703271,1.496909,0.106529,4.443264,65.413428,1.061048,51.633032,40.963183
1,1001.0,Alabama,Autauga,2020,6,8128.591190,20.882987,8.619529,18.081557,33.3,...,15.562670,19.343177,0.480207,1.224798,0.111509,2.965774,74.308016,0.820225,51.448715,42.002162
2,1003.0,Alabama,Baldwin,2020,2,7354.122530,17.509134,8.345003,17.489033,31.0,...,20.443350,8.783976,0.772399,1.150343,0.066966,4.646779,83.111337,0.543517,51.538377,42.279099
3,1005.0,Alabama,Barbour,2020,45,10253.573403,29.591802,11.474559,21.999985,41.7,...,19.420441,48.032635,0.659137,0.454162,0.184880,4.276355,45.641252,1.631683,47.216752,67.789635
4,1007.0,Alabama,Bibb,2020,34,11977.539484,19.439724,10.308710,19.114200,37.6,...,16.473214,21.120536,0.437500,0.236607,0.116071,2.625000,74.589286,0.268210,46.781250,68.352607


In [74]:
train_df = analysis_only_full_data_df[analysis_only_full_data_df.Health_Outcomes_Rank != 'NR']
train_df

,FIPS,State,County/Borough/Parish,Year,Health_Outcomes_Rank,Yrs_Potential_Life_Lost_per_100K,Percent_Fair_Poor_Health,Percent_Low_Birthweight_Births,Percent_Adult_Smokers,Percent_Adult_Obesity,...,Percent_Age_65_and_over,Percent_Non-Hispanic_Black_or_African_American,Percent_American_Indian_or_Alaskan_Native,Percent_Asian,Percent_Native_Hawaiian_Other_Pacific_Islander,Percent_Hispanic,Percent_Non-Hispanic_White,Percent_Not_Proficient_in_English,Percent_Female,Percent_Rural
0,1000.0,Alabama,0,2020,0,9942.794666,22.028703,10.254871,20.927353,35.500000,...,16.917263,26.497733,0.703271,1.496909,0.106529,4.443264,65.413428,1.061048,51.633032,40.963183
1,1001.0,Alabama,Autauga,2020,6,8128.591190,20.882987,8.619529,18.081557,33.300000,...,15.562670,19.343177,0.480207,1.224798,0.111509,2.965774,74.308016,0.820225,51.448715,42.002162
2,1003.0,Alabama,Baldwin,2020,2,7354.122530,17.509134,8.345003,17.489033,31.000000,...,20.443350,8.783976,0.772399,1.150343,0.066966,4.646779,83.111337,0.543517,51.538377,42.279099
3,1005.0,Alabama,Barbour,2020,45,10253.573403,29.591802,11.474559,21.999985,41.700000,...,19.420441,48.032635,0.659137,0.454162,0.184880,4.276355,45.641252,1.631683,47.216752,67.789635
4,1007.0,Alabama,Bibb,2020,34,11977.539484,19.439724,10.308710,19.114200,37.600000,...,16.473214,21.120536,0.437500,0.236607,0.116071,2.625000,74.589286,0.268210,46.781250,68.352607
5,1009.0,Alabama,Blount,2020,24,11335.071134,21.745293,7.604563,19.208672,33.800000,...,18.236515,1.462656,0.653527,0.319848,0.121024,9.571231,86.886238,1.724520,50.726141,89.951502
6,1011.0,Alabama,Bullock,2020,56,11679.558981,30.999102,15.688488,22.894664,37.200000,...,16.383902,69.540343,0.838430,0.187414,0.749655,7.960150,21.217203,0.440754,45.482344,51.374382
7,1013.0,Alabama,Butler,2020,64,14359.939103,27.910673,12.680288,21.765335,43.300000,...,20.299797,44.557927,0.376016,1.316057,0.050813,1.509146,51.255081,0.494155,53.429878,71.232157
8,1015.0,Alabama,Calhoun,2020,42,12078.616145,23.107733,9.161374,20.612560,38.500000,...,17.717476,20.850215,0.539916,0.964324,0.110258,3.910673,72.024992,0.991376,51.946586,33.696826
9,1017.0,Alabama,Chambers,2020,44,11112.536614,23.995474,12.213740,19.392927,40.100000,...,19.521047,39.565670,0.306411,1.326789,0.044623,2.561357,55.231296,0.112938,52.125539,49.148034


In [75]:
train_df.to_csv(os.path.join("Cleaned_Files","Training_Data_All_Years_All_Counties.csv"),index=False)

### Create Testing File of just Not Ranked County data across all states and years

In [76]:
test_df = analysis_only_full_data_df[analysis_only_full_data_df.Health_Outcomes_Rank == 'NR']
test_df

,FIPS,State,County/Borough/Parish,Year,Health_Outcomes_Rank,Yrs_Potential_Life_Lost_per_100K,Percent_Fair_Poor_Health,Percent_Low_Birthweight_Births,Percent_Adult_Smokers,Percent_Adult_Obesity,...,Percent_Age_65_and_over,Percent_Non-Hispanic_Black_or_African_American,Percent_American_Indian_or_Alaskan_Native,Percent_Asian,Percent_Native_Hawaiian_Other_Pacific_Islander,Percent_Hispanic,Percent_Non-Hispanic_White,Percent_Not_Proficient_in_English,Percent_Female,Percent_Rural
69,2013.0,Alaska,Aleutians East,2020,NR,0.0,16.834521,0.000000,16.646811,50.5,...,9.217639,10.412518,19.687055,43.357041,1.194879,14.708393,8.591750,13.084112,32.460882,100.0
73,2060.0,Alaska,Bristol Bay,2020,NR,0.0,15.385269,0.000000,18.137644,41.8,...,14.481186,0.798176,32.611174,1.368301,0.342075,7.525656,43.443558,0.000000,46.636260,100.0
92,2230.0,Alaska,Skagway,2020,NR,0.0,11.604994,0.000000,15.584385,32.6,...,14.547038,0.696864,5.139373,2.526132,0.348432,5.487805,81.358885,0.402010,47.996516,0.0
96,2282.0,Alaska,Yakutat,2020,NR,0.0,16.437249,0.000000,19.879947,31.9,...,19.536424,1.986755,39.569536,6.788079,1.324503,4.635762,32.615894,0.797448,44.536424,100.0
277,8053.0,Colorado,Hinsdale,2020,NR,0.0,12.127612,0.000000,12.894843,24.4,...,30.332922,0.616523,1.356350,0.739827,0.000000,6.041924,89.025894,0.000000,48.458693,100.0
279,8057.0,Colorado,Jackson,2020,NR,0.0,13.964569,19.767442,14.462952,28.0,...,22.230164,0.071480,2.644746,0.714796,0.000000,12.580415,84.703360,1.731245,48.248749,100.0
290,8079.0,Colorado,Mineral,2020,NR,0.0,11.949737,0.000000,12.747824,21.1,...,31.185567,0.257732,1.159794,0.515464,0.000000,6.443299,90.206186,0.246609,50.386598,100.0
306,8111.0,Colorado,San Juan,2020,NR,0.0,15.051907,0.000000,16.019892,24.8,...,23.359580,0.131234,0.918635,0.787402,0.000000,12.335958,83.858268,0.756144,43.963255,100.0
560,15005.0,Hawaii,Kalawao,2020,NR,0.0,14.529876,0.000000,12.195113,17.8,...,42.045455,0.000000,0.000000,7.954545,48.863636,1.136364,26.136364,2.816901,53.409091,100.0
576,16025.0,Idaho,Camas,2020,NR,0.0,16.906713,0.000000,15.222347,24.8,...,22.448980,0.443656,1.419698,0.443656,0.000000,6.477374,88.110027,0.972053,49.423248,100.0


In [77]:
test_df.to_csv(os.path.join("Cleaned_Files","Test_Data_All_Years_All_Counties.csv"),index=False)

### Create Texas Only Subset of Full Data, Training Data and Test Data files

In [78]:
full_tx_data = full_data_df[full_data_df.State == 'Texas']
full_tx_data

,FIPS,State,County/Borough/Parish,Year,Health_Outcomes_Rank,Yrs_Potential_Life_Lost_per_100K,Percent_Fair_Poor_Health,Percent_Low_Birthweight_Births,Percent_Adult_Smokers,Percent_Adult_Obesity,...,Percent_Native_Hawaiian_Other_Pacific_Islander,Number_Hispanic,Percent_Hispanic,Number_Non-Hispanic_White,Percent_Non-Hispanic_White,Number_Not_Proficient_in_English,Percent_Not_Proficient_in_English,Percent_Female,Number_Pop_Living_in_Rural_Area,Percent_Rural
2566,48000.0,Texas,0,2020,0,6650.619481,20.806624,8.328265,15.709603,30.1,...,0.147492,11368849.0,39.610168,11912849.0,41.505516,1957944.0,7.563623,50.316413,3847522.0,15.300999
2567,48001.0,Texas,Anderson,2020,194,10927.038043,19.727723,7.307041,17.386321,36.5,...,0.158465,10430.0,17.965103,34059.0,58.664760,1388.0,2.529523,38.773964,39204.0,67.063533
2568,48003.0,Texas,Andrews,2020,27,7137.509901,19.851002,6.292750,13.436300,34.9,...,0.022065,10268.0,56.641659,7229.0,39.877538,1306.0,8.028524,48.841571,2440.0,16.502097
2569,48005.0,Texas,Angelina,2020,182,8913.812861,20.900430,9.050049,15.856966,40.7,...,0.070041,19589.0,22.492307,52523.0,60.307491,3586.0,4.394823,51.214807,37383.0,43.082366
2570,48007.0,Texas,Aransas,2020,176,8770.440897,20.550541,9.263521,14.157452,40.5,...,0.092468,6526.0,27.429388,16063.0,67.514291,571.0,2.424011,50.609449,6313.0,27.260558
2571,48009.0,Texas,Archer,2020,58,8253.250524,13.891398,7.977737,14.062607,26.8,...,0.034145,717.0,8.160710,7757.0,88.288186,79.0,0.945089,50.762577,8057.0,88.988292
2572,48011.0,Texas,Armstrong,2020,53,0.000000,12.287990,8.609272,11.749550,29.3,...,0.000000,151.0,7.980973,1685.0,89.059197,18.0,1.002786,51.479915,1901.0,100.000000
2573,48013.0,Texas,Atascosa,2020,102,7753.554711,23.678388,7.989096,14.394470,36.7,...,0.127211,32529.0,64.657126,16669.0,33.132578,1839.0,4.064627,50.141125,27266.0,60.711184
2574,48015.0,Texas,Austin,2020,44,7371.154523,17.116889,7.949791,13.560554,31.5,...,0.043349,8265.0,27.560105,18497.0,61.679282,1449.0,5.215044,50.135050,18853.0,66.344090
2575,48017.0,Texas,Bailey,2020,180,9061.655502,26.991708,7.457627,16.253688,31.3,...,0.128077,4504.0,64.095631,2319.0,33.001281,881.0,13.346463,50.206347,2060.0,28.750872


In [79]:
full_tx_data.to_csv(os.path.join("Cleaned_Files","Texas_All_Years_County_Health_Ranking_Data.csv"),index=False)

In [80]:
analysis_only_full_tx_data = full_tx_data.drop([
'Number_Non-Hispanic_Black_or_African_American',
'Number_American_Indian_or_Alaskan_Native',
'Number_Asian',
 'Number_Native_Hawaiian_Other_Pacific_Islander',
'Number_Hispanic',
'Number_Non-Hispanic_White',
'Number_Not_Proficient_in_English',
'Number_Pop_Living_in_Rural_Area']
                     ,axis=1)
analysis_only_full_tx_data.head()

,FIPS,State,County/Borough/Parish,Year,Health_Outcomes_Rank,Yrs_Potential_Life_Lost_per_100K,Percent_Fair_Poor_Health,Percent_Low_Birthweight_Births,Percent_Adult_Smokers,Percent_Adult_Obesity,...,Percent_Age_65_and_over,Percent_Non-Hispanic_Black_or_African_American,Percent_American_Indian_or_Alaskan_Native,Percent_Asian,Percent_Native_Hawaiian_Other_Pacific_Islander,Percent_Hispanic,Percent_Non-Hispanic_White,Percent_Not_Proficient_in_English,Percent_Female,Percent_Rural
2566,48000.0,Texas,0,2020,0,6650.619481,20.806624,8.328265,15.709603,30.1,...,12.550831,11.991020,1.013736,5.178294,0.147492,39.610168,41.505516,7.563623,50.316413,15.300999
2567,48001.0,Texas,Anderson,2020,194,10927.038043,19.727723,7.307041,17.386321,36.5,...,14.797527,20.958713,0.697590,0.930120,0.158465,17.965103,58.664760,2.529523,38.773964,67.063533
2568,48003.0,Texas,Andrews,2020,27,7137.509901,19.851002,6.292750,13.436300,34.9,...,10.359665,1.334951,1.522507,0.772286,0.022065,56.641659,39.877538,8.028524,48.841571,16.502097
2569,48005.0,Texas,Angelina,2020,182,8913.812861,20.900430,9.050049,15.856966,40.7,...,16.325265,14.833739,0.758968,1.077022,0.070041,22.492307,60.307491,4.394823,51.214807,43.082366
2570,48007.0,Texas,Aransas,2020,176,8770.440897,20.550541,9.263521,14.157452,40.5,...,29.627606,1.260928,1.235709,1.996469,0.092468,27.429388,67.514291,2.424011,50.609449,27.260558


In [81]:
Texas_train_df = analysis_only_full_tx_data[analysis_only_full_tx_data.Health_Outcomes_Rank != 'NR']
Texas_train_df

,FIPS,State,County/Borough/Parish,Year,Health_Outcomes_Rank,Yrs_Potential_Life_Lost_per_100K,Percent_Fair_Poor_Health,Percent_Low_Birthweight_Births,Percent_Adult_Smokers,Percent_Adult_Obesity,...,Percent_Age_65_and_over,Percent_Non-Hispanic_Black_or_African_American,Percent_American_Indian_or_Alaskan_Native,Percent_Asian,Percent_Native_Hawaiian_Other_Pacific_Islander,Percent_Hispanic,Percent_Non-Hispanic_White,Percent_Not_Proficient_in_English,Percent_Female,Percent_Rural
2566,48000.0,Texas,0,2020,0,6650.619481,20.806624,8.328265,15.709603,30.1,...,12.550831,11.991020,1.013736,5.178294,0.147492,39.610168,41.505516,7.563623,50.316413,15.300999
2567,48001.0,Texas,Anderson,2020,194,10927.038043,19.727723,7.307041,17.386321,36.5,...,14.797527,20.958713,0.697590,0.930120,0.158465,17.965103,58.664760,2.529523,38.773964,67.063533
2568,48003.0,Texas,Andrews,2020,27,7137.509901,19.851002,6.292750,13.436300,34.9,...,10.359665,1.334951,1.522507,0.772286,0.022065,56.641659,39.877538,8.028524,48.841571,16.502097
2569,48005.0,Texas,Angelina,2020,182,8913.812861,20.900430,9.050049,15.856966,40.7,...,16.325265,14.833739,0.758968,1.077022,0.070041,22.492307,60.307491,4.394823,51.214807,43.082366
2570,48007.0,Texas,Aransas,2020,176,8770.440897,20.550541,9.263521,14.157452,40.5,...,29.627606,1.260928,1.235709,1.996469,0.092468,27.429388,67.514291,2.424011,50.609449,27.260558
2571,48009.0,Texas,Archer,2020,58,8253.250524,13.891398,7.977737,14.062607,26.8,...,20.009105,0.865013,1.479627,0.523560,0.034145,8.160710,88.288186,0.945089,50.762577,88.988292
2572,48011.0,Texas,Armstrong,2020,53,0.000000,12.287990,8.609272,11.749550,29.3,...,23.942918,0.845666,1.479915,0.211416,0.000000,7.980973,89.059197,1.002786,51.479915,100.000000
2573,48013.0,Texas,Atascosa,2020,102,7753.554711,23.678388,7.989096,14.394470,36.7,...,14.678990,0.741403,1.313854,0.622143,0.127211,64.657126,33.132578,4.064627,50.141125,60.711184
2574,48015.0,Texas,Austin,2020,44,7371.154523,17.116889,7.949791,13.560554,31.5,...,19.003635,8.619827,0.796959,0.770282,0.043349,27.560105,61.679282,5.215044,50.135050,66.344090
2575,48017.0,Texas,Bailey,2020,180,9061.655502,26.991708,7.457627,16.253688,31.3,...,14.586595,1.252313,3.358474,0.868080,0.128077,64.095631,33.001281,13.346463,50.206347,28.750872


In [82]:
Texas_train_df.to_csv(os.path.join("Cleaned_Files","Training_Data_All_Years_Texas_Only.csv"),index=False)

In [83]:
Texas_test_df = analysis_only_full_tx_data[analysis_only_full_tx_data.Health_Outcomes_Rank == 'NR']
Texas_test_df

,FIPS,State,County/Borough/Parish,Year,Health_Outcomes_Rank,Yrs_Potential_Life_Lost_per_100K,Percent_Fair_Poor_Health,Percent_Low_Birthweight_Births,Percent_Adult_Smokers,Percent_Adult_Obesity,...,Percent_Age_65_and_over,Percent_Non-Hispanic_Black_or_African_American,Percent_American_Indian_or_Alaskan_Native,Percent_Asian,Percent_Native_Hawaiian_Other_Pacific_Islander,Percent_Hispanic,Percent_Non-Hispanic_White,Percent_Not_Proficient_in_English,Percent_Female,Percent_Rural
2583,48033.0,Texas,Borden,2020,NR,0.0,12.579258,0.000000,10.643092,27.1,...,24.691358,0.925926,0.771605,0.154321,0.000000,18.672840,78.240741,0.000000,47.993827,100.0
2653,48173.0,Texas,Glasscock,2020,NR,0.0,20.098770,0.000000,15.073828,23.9,...,15.201729,1.512968,0.432277,0.144092,0.144092,38.472622,59.221902,15.198238,44.884726,100.0
2684,48235.0,Texas,Irion,2020,NR,0.0,15.936694,10.377358,12.589311,25.2,...,21.484888,1.051248,1.182654,0.262812,0.000000,26.281209,70.367937,0.000000,48.685940,100.0
2697,48261.0,Texas,Kenedy,2020,NR,0.0,30.347698,0.000000,16.071325,23.7,...,15.610860,2.036199,1.583710,0.678733,0.000000,73.303167,20.814480,19.607843,48.416290,100.0
2698,48263.0,Texas,Kent,2020,NR,0.0,16.817128,0.000000,13.049828,26.0,...,27.134986,1.239669,1.652893,0.137741,0.137741,19.696970,76.308540,0.837989,50.275482,100.0
2701,48269.0,Texas,King,2020,NR,0.0,17.414647,0.000000,14.322325,22.7,...,17.689531,0.000000,1.444043,0.000000,0.361011,15.523466,80.866426,3.317536,49.458484,100.0
2717,48301.0,Texas,Loving,2020,NR,0.0,15.312154,0.000000,14.343420,28.9,...,9.868421,4.605263,0.657895,0.000000,0.000000,16.447368,78.289474,7.228916,44.078947,100.0
2722,48311.0,Texas,McMullen,2020,NR,0.0,18.195807,0.000000,12.279316,26.4,...,24.032043,2.403204,0.667557,0.667557,0.000000,42.990654,52.736983,3.241491,45.126836,100.0
2763,48393.0,Texas,Roberts,2020,NR,0.0,13.020938,0.000000,12.588106,23.2,...,20.930233,0.332226,0.996678,0.221484,0.000000,11.184939,85.825028,0.000000,50.387597,100.0
2782,48431.0,Texas,Sterling,2020,NR,0.0,21.379731,0.000000,15.259653,24.9,...,14.416476,1.830664,2.059497,0.533944,0.076278,40.655988,54.767353,0.279851,49.046529,100.0


In [84]:
Texas_train_df.to_csv(os.path.join("Cleaned_Files","Test_Data_All_Years_Texas_Only.csv"),index=False)